# 导入模块

In [1]:
from tokenize import group

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy.dialects.mssql.information_schema import columns
import hashlib

plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine, column
import json
import ast
import datetime
from datetime import timedelta ,time, datetime
import re
import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')

sys.path.append(module_dir)

from All_Class import All_Model, Week_Model, Data_Clean, Risk_Data

In [2]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()
risk_data = Risk_Data()

# 定义数据库连接

In [3]:
# sql1 = ''' -- 订单&风控信息  近10日数据   
# SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
# ,om.status, om.user_id
# ,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
# when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
# when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
# ,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
# ,tod.sku_attributes,tod.product_name,tod.new_actual_money
# ,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租完归还' end as back_type
# ,om.user_mobile,tmu.true_name,tmu.id_card_num
# ,top.total_describes,tor.decision_result,om.cancel_reason
# ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
# ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
# ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
# ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips
# ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
# ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
# ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
# ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result  
# ,cc.name as channel_name         -- 来源渠道
# ,cc.channel_type_id              -- 渠道id
# ,pa.name as activity_name        -- 活动名称
# ,om.merchant_id,om.merchant_name, tomt.merchant_id merchant_id_t
# ,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
# ,om.buy_service_product,tso.status as service_status 
# ,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
# , tor.update_time, tomt.reason, tp.classify_id, tso.price as 碎屏险价格
# , tprm.commodity_premium as 溢价费
# ,tprm.all_deposit, tprm.all_rental, tprm.total_receivable, tprm.purchase_amount, tprm.contract_price
# ,topay.payer_user_id, tprm.model_number, tprm.specification, om.relet_status, om.has_actual
# ,tprm.overdue_type, ym.name zf_name, ym.company_name, om.rent_grade, om.all_sesame_promo_money '总差异化定价优惠金额'
# , tprm.liquidated_damages 违约金, om.auto_pay_tag, topc.policy_status, topc.money 保费, tprm.rent_received, topc.policy_url
# ,rarr.high_risk_models, rarr.extremely_low_risk_models, om.order_type, tol.delivery_username 收货人名称
# , tmao.alipay_order_id, tmao.zfb_open_id, tmu.create_time create_time_tmu, too.capital_type 资金类目, too.money 订单违约金, too.status 支付状态
# , toou.discount_money, toou.pay_status, om.channel, tod.product_id, om.alipay_order_id alipay_order_id_o, tpr.rent_discount_money
# ,toe.expressage
# from  db_digua_business.t_order  om -- 3660.02MB
# -- 渠道名称
# left join db_digua_business.t_channel cc on om.channel = cc.scene -- 0.53MB
# -- 活动名称 
# left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id -- 1.72MB
# -- 订单其他账单表
# left join db_digua_business.t_order_other too on too.order_id=om.id -- 16.83MB
# -- 订单买断表
# left join db_digua_business.t_order_out toou on toou.order_id=om.id
# -- 租后表
# left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id -- 23.53MB
# -- 投保订单表
# left join db_digua_business.t_order_picc topc on topc.order_id=om.id -- 38.56MB
# -- 支付宝-租赁交易共租风险查询
# left join db_credit.risk_alipay_rent_risk rarr on rarr.out_biz_no=om.order_number -- 47.59MB
# -- 订单快递表
# left join db_digua_business.t_order_express toe on toe.order_id=om.id
# -- 交易组件订单关联表
# left join db_digua_business.t_mini_ali_order tmao on tmao.order_id=om.id -- 254.67MB
# -- 服务信息
# left join  db_digua_business.t_service_order tso  on om.id = tso.order_id -- 280.39MB
# -- 资方表
# left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id -- 383.31MB
# -- 备注信息合并 
# left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top on om.id = top.order_id -- 697.31MB
# -- 商家订单转移表
# left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id -- 1237.63MB
# -- 订单物流表
# left join db_digua_business.t_order_logistics tol on tol.order_id=om.id -- 1284.52MB
# -- 免押信息  
# left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
# from db_digua_business.t_order_pay t 
# where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
# on topay.order_id=om.id   and  topay.rn = 1  -- 1434.08MB
# left join db_digua_business.t_order_risk tor on om.id = tor.order_id -- 2425.14MB
# -- 商品信息
# left join db_digua_business.t_order_details tod on om.id = tod.order_id -- 4037.08MB
# -- 支付宝预授权记录表
# left join db_digua_business.t_pay_record tpr on tpr.order_id=om.id -- 6735.94MB
# -- 商品表
# left join db_digua_business.t_product tp on tp.id=tod.product_id -- 8942.97MB
# -- 用户信息 
# left join db_digua_business.t_member_user tmu on om.user_id = tmu.id -- 9676.28MB
# where om.user_mobile is not null 
# and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
# ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
# ,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')

# -- and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -1 day)              
# -- and om.create_time >= '2024-10-30 00:00:00' 
# -- and  om.create_time < DATE_ADD(CURRENT_DATE,INTERVAL -0 day )
# -- and  HOUR(om.create_time)< 16
# -- and  YEAR(om.create_time) = 2024
# and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2024-12-01'
# and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-06-30'
# ;
# '''

# df_order = clean.query(sql1)
# df_order.shape

# 导入订单数据

In [4]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status, om.user_id, om.overdue, om.alipay_order_id
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money, tod.product_id
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name, om.has_actual
,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
,om.buy_service_product
,tso.status as service_status 
,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days, tprm.total_receivable
, tor.update_time, tprm.purchase_amount, tprm.liquidated_damages 违约金
, tomt.reason
, om.order_type, om.union_rent_tag, tprm.rent_received 已收租金 , tprm.paid_buyout_balance_payment 已付买断尾款, tprm.relet_days
, tprm.all_rental, tprm.commodity_premium as 溢价费, tprm.all_deposit
, tolog.status 物流状态, tojo.app_type

, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.rejected_ato_risk') end,'"','') as rejected_ato_risk  # 联合拒量

, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.union_rent_rejected') end,'"','') as union_rent_rejected   # 联合运营

, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_241011') end,'"','') as strategy_241011                    # 拒量
, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_241205_rejected') end,'"','') as strategy_241205_rejected  # 拒量
, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_241212_rejected') end,'"','') as strategy_241212_rejected  # 拒量
, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_250120_rejected') end,'"','') as strategy_250120_rejected  # 拒量
, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_250330_rejected') end,'"','') as strategy_250330_rejected  # 拒量

-- , ymos.money, ymos.part_payment, ymos.refund_date, ymos.reality_refund_date, ymos.sort, ymos.status 分期租金支付状态
from  db_digua_business.t_order  om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 

where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
-- 京东外部订单关联表
left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=om.id
-- 订单物流表
left join db_digua_business.t_order_logistics tolog on tolog.order_id=om.id
-- 租后分期表
-- left join db_rent.ya_merchant_order_stages ymos on ymos.order_id=om.id
left join db_credit.risk risk on risk.order_id=om.id


where om.user_mobile is not null 
-- 曙光计划订单会在去重逻辑中剔除，这里不做过滤，否则存在非线下小店订单被剔除的情况
-- and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
-- ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
-- and pa.type!=4 号卡活动，已过期，不统计
-- and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )               -- 近20天数据
-- and  om.create_time <= DATE_ADD(CURRENT_DATE,INTERVAL -0 day )
and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-12-15'
and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-12-21'
-- and pa.name='支付宝直播商品'

'''

df_order = clean.query(sql1)
df_order.shape

(73588, 67)

In [5]:
sql_modelname = '''
-- 商品表
select
tp.id, tp.classify_id, tp.id 商品ID, tpmn.name phone_name
from db_digua_business.t_product tp
left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id
'''
df_modelname = clean.query(sql_modelname)
df_order = df_order.merge(df_modelname, left_on='product_id', right_on='id',how='left')

In [6]:
sql_risk = ''' -- risk等级表
select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r from db_credit.risk
'''
df_risk = clean.query(sql_risk)
df_risk.loc[:, 'time_r'] = pd.to_datetime(df_risk.time.dt.date,errors="coerce")
df_risk = df_risk.sort_values(by='time', ascending=False).groupby('id_card_r').head(1)

In [7]:
sql3 = '''
SELECT id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = clean.query(sql3)
df_re = df_risk_examine.copy()
df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)
df_re = df_re.sort_values(by='time', ascending=False).groupby('id_card_re').head(1)

In [8]:
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
df_ra = clean.query(sql_ra)
df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

In [9]:
# trace关联
# sql_trace='''-- trace关联表 
# SELECT order_trace_id, risk_trace_id, trace_id, order_id
# from db_credit.order_association
# ;
# '''
# trace_df = query(sql_trace)
# trace_df.shape

In [10]:
# # 火眸分
# sql1='''-- 风控-冰鉴-火眸分
# SELECT trace_id,  rs
# from db_credit.risk_icekredit_cds
# ;
# '''
# bj_hm_df =query(sql1)
# bj_hm_df['bj_hm_score'] = bj_hm_df['rs'].apply(lambda x: json.loads(x).get('result', {}).get('card_device_score22', {}).get('bj_score', None))  
# bj_hm_df['bj_hm_score'] = bj_hm_df['bj_hm_score'].astype(int)
# df_rule = trace_df.merge(bj_hm_df[['trace_id', 'bj_hm_score']], on='trace_id', how='outer')

In [11]:
sql_examine = '''
select risk_trace_id, time
,replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.strategy_241205') end,'"','') as strategy_241205 
from db_credit.risk_examine
'''
df_examine = clean.query(sql_examine)
df_examine['time'] = pd.to_datetime(df_examine['time'].dt.date)

In [12]:
# sql_base = ''' -- 风控-自有模型-2501
# select trace_id, score as score_base from db_credit.risk_private_model_2501_base
# '''
# df_base = query(sql_base)

In [13]:
# sql_pro = ''' -- 风控-自有模型-2501
# select order_id, score as score_pro from db_credit.risk_private_model_2501_pro
# '''
# df_pro = query(sql_pro)
# df_pro['order_id'] = df_pro.order_id.astype(int)

In [14]:
sql_risk = '''
select id_card, trace_id as trace_id_risk, time as time_risk 
,replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.strategy_250210') end, '"', '') as strategy_250210
from db_credit.risk
'''
df_risks = clean.query(sql_risk)
df_risks.loc[:, '请求日期'] = pd.to_datetime(df_risks.time_risk.dt.date)
df_risks = df_risks.sort_values('time_risk', ascending=False).groupby(['id_card', '请求日期']).head(1)

In [15]:
# sql_stages = '''
# select 
# ymos.order_id, ymos.sort, ymos.refund_date, ymos.reality_refund_date, ymos.part_payment
# from db_rent.ya_merchant_order_stages ymos
# -- where ymos.sort=2
# '''
# df_stages = clean.query(sql_stages)

In [16]:
# sql_stages2 = '''
# select 
# ymos.order_id, ymos.sort, ymos.refund_date, ymos.reality_refund_date, ymos.real_pay_money
# from db_digua_business.t_order_stages ymos
# where ymos.sort=2
# '''
# df_stages2 = clean.query(sql_stages2)

In [17]:
sql_mini = '''
select
order_id, alipay_order_id
from db_digua_business.t_mini_ali_order
'''
df_mini = clean.query(sql_mini)

# 提取订单风控数据

# 合并数据

In [18]:
# df = df_order.merge(df_order_risk,left_on='订单id',right_on='order_id',how='left')
df = df_order.copy()
df = df[df.type!=4]
df.order_id.notnull().sum()

np.int64(73588)

# 处理日期

In [19]:
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month
df["年份"]=df["下单日期"].dt.year
df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')
df['hour'] = df['create_time'].dt.hour

In [20]:
# 机审强拒策略
# 提取拒绝理由字段并清理特殊字符
# df['拒绝理由'] = df["rejected"].fillna('').str.replace(r'[\[\]"]', '', regex=True)
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')

df['拒绝理由_risk']= df['rejected'].fillna('')+df['strategy_241205_rejected'].fillna('')+df['strategy_241212_rejected'].fillna('')+df['strategy_241011'].fillna('')+df['strategy_250120_rejected'].fillna('')+df['strategy_250330_rejected'].fillna('')+df['union_rent_rejected'].fillna('')+df['rejected_ato_risk'].fillna('')

df.loc[:, '拒绝理由'] = np.where(df['拒绝理由'] == '', df['拒绝理由_risk'], df['拒绝理由'])
df['拒绝理由'] = df["拒绝理由"].str.replace("[","").str.replace("]","").str.replace('"','')

# 渠道补充

In [21]:
# qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
# qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)
# qudao_name_df.rename(columns={'订单号': '订单号1'}, inplace=True)
# df =  df.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号1',how = 'left')
# df["channel_name"] = np.where(df.渠道名称.notnull(),df.渠道名称,df["channel_name"])
df.下单日期.unique()

<DatetimeArray>
['2025-12-15 00:00:00', '2025-12-16 00:00:00', '2025-12-17 00:00:00',
 '2025-12-18 00:00:00', '2025-12-19 00:00:00', '2025-12-20 00:00:00',
 '2025-12-21 00:00:00']
Length: 7, dtype: datetime64[ns]

# 处理备注信息

In [22]:
def extract_reason(desc, marker):
    """
    从描述中提取特定标记后的原因
    
    参数:
    desc -- 要处理的描述文本
    marker -- 用于分割的标记字符串（如"客户申请取消："或"用户无法联系："）
    
    返回:
    提取的原因字符串，如果不符合格式则返回None
    """
    try:
        # 确保是字符串类型
        desc = str(desc)
        # 按分割符分割并提取所需部分
        parts = desc.split(marker)
        if len(parts) > 1:
            reason = parts[1].split("$")[0].strip()
            # 如果是取消原因，还需要去掉引号部分
            if marker == "客户申请取消：":
                reason = reason.split('"')[0]
            return reason
        return None
    except (IndexError, AttributeError):
        return None

In [23]:
df = df[df['sku_attributes'].notnull()]
# df["取消原因"] = df["total_describes"].apply(lambda x: extract_reason(x, "客户申请取消："))
# df["取消原因"]=df["total_describes"].astype(str).str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip().str.split('"').str[0]
# df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip().str.split('"').str[0]
# df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"] = df["total_describes"].apply(lambda x: extract_reason(x, "用户无法联系："))

# df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip().str.split('"').str[0]
df["取消原因"] = df["total_describes"].fillna("").astype(str).str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip().str.split('"').str[0]
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip().str.split('"').str[0]
# df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()

df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  

# %%
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]

df.loc[:,"内存"]=df.apply(lambda x: getneicun(x["sku_attributes"]),axis=1)

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")

df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
df.loc[:, '商品ID'] = df.商品ID.astype(str)+'_'+df.phone_name
# 物流状态=5，退货拒签
df.loc[:,"退货拒签"] = np.where(df['物流状态']==5, 1, 0)
df.loc[:,'机型内存'] = df['phone_name'] + df['内存']

# 渠道归属

In [24]:


df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"],x['channel_type_id'],x['order_type']),axis=1)

# 订单去重

In [25]:
# 保留不去重的数据,京东不去重
df_no_drop = df.copy()
df = clean.order_drop_duplicates(df)

# 定义状态

In [26]:
# note 2025-02-24添加
df = df.merge(df_risk[['id_card_r', 'time_r', 'status_r']], left_on=['id_card_num', '下单日期'], right_on=['id_card_r', 'time_r'], how='left').merge(df_re[['id_card_re', 'time_re', 'status_re']], left_on=['id_card_num', '下单日期'], right_on=['id_card_re', 'time_re'], how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')

# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
# df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
# df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)
df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)
df.loc[:, '是否拒量'] = np.where(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)

In [27]:

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)
df['取消原因2'] = df['cancel_reason'].str.split('：')
df['取消原因2'] = df['取消原因2'].apply(lambda x: x[-1] if x is not None else x)

# 保留商家的数据

In [28]:
df_contain = df.copy()

# 剔除商家数据

In [29]:
# 调用Data_Clean类中的drop_merchant函数
df = clean.drop_merchant(df)

# 去重订单明细导出

In [30]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')

# 导入服务订单

In [31]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

df_j = df[df["进件"]=="进件"]

In [32]:
# 定义各个节点的状态
df = clean.status_node(df)
# df.下单日期.unique()

In [33]:
# df[df['order_number']=='A20250812134154794']

# 导入台账数据

In [34]:
#读取导入出库台账数据
# f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
# df_ck = pd.read_excel(f_path_ck,sheet_name="2023")  

f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2025")  

In [35]:
dfck = pd.merge(df_ck,df,left_on="订单号", right_on="order_number")
dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)
# 删除 露营设备 出库 
try:
    dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)
except:
    dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)
# 将序列号转换为日期
# base_date = pd.Timestamp('1899-12-30')  # Excel的基准日期
# dfck['日期'] = base_date + pd.to_timedelta(dfck['日期'], unit='D')


# 按转化日期看每日各渠道出库单数

In [36]:
#按转化日期看每日各渠道出库单数
df_weekday_zh = pd.crosstab(dfck["下单日期"],dfck["归属渠道"],margins=True)   #下单月份
# df_weekday_zh['All'][:-1]

# 按下单日期看每日各渠道出库单数

In [37]:
#按下单日期看每日各渠道出库单数
df_weekday = pd.crosstab(dfck["下单日期"],dfck["归属渠道"],margins=True)
# df_weekday
# df_weekday['All'][:-1]

In [38]:
#按下单日期看每日各渠道出库单数（纯租物）
df_ly_ck = pd.crosstab(dfck["下单日期"],dfck["来源渠道"],margins=True)
# df_ly_ck['芝麻租物'][:-1]



# 按下单日期看每日支付宝直播商品出库单数

In [39]:
#按下单日期看每日支付宝直播商品出库单数
df_zfb_ck = pd.crosstab(dfck["下单日期"],dfck["activity_name"],margins=True)

# df_zfb_ck['支付宝直播商品'][:-1]

# 按下单日期看 免押 出库单数

In [40]:
df_my = pd.crosstab(dfck["下单日期"],dfck["押金类型"],margins=True)
# df_my['全免押'][:-1]

# 按月份看商家的免押出库单数

In [41]:
# df_contain[(df_contain["merchant_name"] == '小蚂蚁租机')&(df['是否出库']==1)].head()

In [42]:
# path = 'F:\需求\总需求\商家免押.xlsx'

# # 筛选小蚂蚁商家
# df_xmy_ck = df_contain[(df_contain["merchant_name"] == '小蚂蚁租机')&(df_contain['是否出库']==1)]
# df_xmy_tab = pd.crosstab(df_xmy_ck["下单月份"],df_xmy_ck["押金类型"],margins=True)

# # 筛选人人享租商家
# df_rrx_ck = df_contain[(df_contain["merchant_name"] == '人人享租')&(df_contain['是否出库']==1)]
# df_rrx_tab = pd.crosstab(df_rrx_ck["下单月份"],df_rrx_ck["押金类型"],margins=True)
# # 筛选喜卓灵租机
# df_xzl_ck = df_contain[(df_contain["merchant_name"] == '喜卓灵租机')&(df_contain['是否出库']==1)]
# df_xzl_tab = pd.crosstab(df_xzl_ck["下单月份"],df_xzl_ck["押金类型"],margins=True)
# # 筛选喜卓灵新租机
# df_xzl_new_ck = df_contain[(df_contain["merchant_name"] == '喜卓灵新租机')&(df_contain['是否出库']==1)]
# df_xzl_new_tab = pd.crosstab(df_xzl_new_ck["下单月份"],df_xzl_new_ck["押金类型"],margins=True)
# # 写入excel
# with pd.ExcelWriter(path, engine='xlsxwriter') as writer:
#     df_xmy_tab.to_excel(writer, sheet_name='小蚂蚁商家免押')
#     df_rrx_tab.to_excel(writer, sheet_name='人人享租商家免押')
#     df_xzl_tab.to_excel(writer, sheet_name='喜卓灵商家免押')
#     df_xzl_new_tab.to_excel(writer, sheet_name='喜卓灵新商家免押')
    

In [43]:
# df_rrx_tab

# 残值计算（需要更改）

In [44]:
# def ygcz(row): #已过时
#     if 'iPhone' in row['product_name']:
#         if 'iPhone15' in row['product_name']:
#             return row['采购金额'] * 0.6
#         else:
#             return row['采购金额'] * 0.4
#     else:
#         return row['采购金额'] * 0.425


# # 2025年8月及以前的残值计算方式
def getrecanzhi(s):
    """
    计算设备残值
    :param s: 包含产品信息的Series对象
    :return: 残值金额
    """
    # 租完即送方案残值为0
    if s["租赁方案"]=="租完即送":
        return  0
    elif 'iPhone' in s['product_name']:
        if '16' in s['product_name']:
            return s['成本'] * 0.6
        elif '15' in s['product_name']:
            return s['成本'] * 0.47
        elif '14' in s['product_name']:
            return s['成本'] * 0.4
        else:
            return s['成本'] * 0.375
    # 其他设备使用固定残值率
    else:
        return s['成本'] * 0.425
# 2025年9月以后的残值计算方式
# def getrecanzhi(s):
#     if s["租赁方案"]=="租完即送":
#         return  0
#     elif 'iPhone' in s['product_name']:
#         if '17' in s['product_name']:
#             return s['all_deposit'] * 0.6
#         elif 'Air' in s['product_name']:
#             return s['all_deposit'] * 0.6
#         elif '16' in s['product_name']:
#             return s['成本'] * 0.45
#         elif '15' in s['product_name']:
#             return s['成本'] * 0.47
#         elif '14' in s['product_name']:
#             return s['成本'] * 0.4
#         else:
#             return s['成本'] * 0.375
#     # 其他设备使用固定残值率
#     else:
#         return s['成本'] * 0.425



A20250301093707114 正常被拒订单  拒绝理由:命中自有模型强拒, 命中融安分低于680强拒, 评分等级低于D,命中拒绝  审核状态：机审强拒  是否机审强拒：1  是否出库前风控强拒: 0

A20250821195141539 拒量订单（回捞241205） 拒绝理由:null  审核状态：人审拒绝  是否机审强拒：0, 是否出库前风控强拒: 0   日报会回加到机审强拒中

A20250916223149153 联合拒量订单 拒绝理由:null  审核状态：人审拒绝  是否机审强拒：0  是否出库前风控强拒：0

A20250807124004660 联合运营被拒订单(支付宝联合运营订单) 拒绝理由:null  审核状态：人审拒绝  是否机审强拒：0  是否出库前风控强拒：0  [命中银联模型及Fico联合规则强拒(青云分豁免), 已有在租, 支付宝联合运营订单, 用户下单次数:2]   日报不会回加到机审强拒中


# 联合拒量出库订单

In [45]:
# 按下单日期与身份证号去重
# df.drop_duplicates(subset=["id_card_num", "下单日期"], keep="last", inplace=True)# 75
# df[(df.tips.str.contains(r'联合拒量', regex=True, na=False)==True)&(df['是否出库'] == 1)][['order_number', '下单日期', '是否出库']]

# 总体转化

## 剔除小蚂蚁租机和已退款订单

In [46]:
# df.loc[:, '内存'] = df.specification.str.split('：').str[1].str.split(' ').str[0]
# df.loc[:, '型号内存'] = df.model_number+ '_' + df.内存
# df[df.classify_id.isin([125, 126, 127, 185])].机型.unique()

In [47]:
iPhone_str = 'iPhone ?\d+ ?(ProMax|Pro|Plus|mini)?'
Xiaomi_str = '(小米|Xiaomi) ?(\d+|Note|Civi|MIX)(Pro|Ultra|Fold|\d+)(\d+)?|Redmi ?(K|Note|Turbo)(\d+)(Pro)?'
# df.product_name.str.extract(fr'({iPhone_str}|{Xiaomi_str})')[0].str.replace(' ', '').drop_duplicates()
# df.product_name.str.extract(fr'({iPhone_str})')[0].str.replace(' ', '').drop_duplicates()

In [48]:
df2 = df.copy()
df2 = clean.drop_rejected_merchant(df2)
df2 = df2[df2.是否拒量==0]

In [49]:
# df_es = df[(df.是否二手==1)&(df.product_name.str.contains('iPhone', regex=True)==True)]
# df2_es = df2[(df2.是否二手==1)&(df2.product_name.str.contains('iPhone', regex=True)==True)]
# all_models.data_group(df_es, df2_es, df_risk_examine, '下单日期')[['去重订单数', '进件数', '出库']]

In [50]:
# df_mobile = pd.read_excel('F:/需求/总需求/手机号.xlsx')
# df_mobile.loc[:, '手机号码'] = df_mobile.手机号码.astype(str)
# df_mobile_list = df_mobile.手机号码.to_list()
# df_mobile.dtypes

In [51]:
# df2_1 = df2[(df2.下单日期>='2025-06-01')&(df2.下单日期<='2025-06-10')]
# df2_2 = df2[(df2.下单日期>='2025-07-07')&(df2.下单日期<='2025-07-16')]
# df2_3 = df2[(df2.下单日期>='2025-07-15')&(df2.下单日期<='2025-07-16')]
# df_reject1 = df2_1.groupby('电审拒绝原因').agg({'order_id': 'count'}).sort_values(by='order_id', ascending=False).rename(columns={'order_id': '数量'})
# df_reject1.loc['合计', :] = df_reject1.sum(axis=0)
# df_reject1.loc[:, '比例'] = (df_reject1.数量/df_reject1.loc['合计', '数量']).map(lambda x: format(x, '.2%'))
# df_reject1.loc[:, 'nan'] = np.nan
# df_reject1 = df_reject1.reset_index(names=['电审拒绝原因_0601-0610'])
# df_reject2 = df2_2.groupby('电审拒绝原因').agg({'order_id': 'count'}).sort_values(by='order_id', ascending=False).rename(columns={'order_id': '数量'})
# df_reject2.loc['合计', :] = df_reject2.sum(axis=0)
# df_reject2.loc[:, '比例'] = (df_reject2.数量/df_reject2.loc['合计', '数量']).map(lambda x: format(x, '.2%'))
# df_reject2 = df_reject2.reset_index(names=['电审拒绝原因_0707-0716'])
# df2_3 = df2_3[df2_3.电审拒绝原因.isin(['已在其他平台租赁"]', '验证资料与电审不符且差距大"]'])][['order_number', '电审拒绝原因']]
# df_reject = pd.concat([df_reject1,df_reject2], axis=1)
# # with pd.ExcelWriter('F:/需求/总需求/取消拒绝原因.xlsx', engine='xlsxwriter') as writer:
# #     df_reject.to_excel(writer, sheet_name='电审拒绝原因')
# df2_3.to_excel('F:/需求/总需求/拒绝原因.xlsx', index=False)
# df2_3 = df2[(df2.下单日期>='2025-07-15')&(df2.下单日期<='2025-07-16')]

## 赛租机取样

In [52]:
# # 已电销
# df_ydx = pd.read_excel(f'F:/需求/总需求/赛租机数据.xlsx')
# df_ydx['手机号码'] = df_ydx['手机号码'].astype('str')
# mobile = df_ydx.手机号码.to_list()
# df2 = df2[~df2.user_mobile.isin(mobile)]
# df2= df2[df2.product_name != '测试商品请勿下单']

In [53]:
# 获取excel表中进件前取消的订单数据
# df_jjq = df2[(df2.进件前取消==1)]
# # df_jjq = df_jjq[df_jjq.user_mobile.isin(df_mobile_list)][['create_time_tmu', 'user_mobile', 'id_card_num', 'true_name']].rename(columns={'user_mobile': '手机号码', 'create_time_tmu': '注册时间'})
# 从 df_jjq DataFrame 中随机抽取 500 条记录（sample(n=500)），并确保每次抽取的结果都是一致的（通过设置 random_state=42）
# df_jjq = df_jjq.sample(n=500, random_state=42)[['create_time_tmu', 'user_mobile', 'id_card_num', 'true_name','product_name','颜色','内存', '下单日期']].rename(columns={'user_mobile': '手机号码', 'create_time_tmu': '注册时间','product_name':'产品'})
# df_jjq.loc[:, 'gender'] = df_jjq.id_card_num.apply(lambda x: clean.get_gender(x))
# df_jjq.loc[:, '姓名'] = np.where(df_jjq.gender=='男', df_jjq.true_name.str[0]+'先生', df_jjq.true_name.str[0]+'女士')
# df_jjq = df_jjq[['注册时间','下单日期','手机号码', '姓名','产品','颜色','内存']]
# with pd.ExcelWriter(f'F:/需求/力哥需求/赛租机数据_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_jjq.to_excel(writer, index=False)

In [54]:
# # # 进件后取消的
# df_jjq = df2[(df2.进件 == '进件')&(df2.审核状态 == '客户取消')]

# # df_jjq = df_jjq[df_jjq.user_mobile.isin(df_mobile_list)][['create_time_tmu', 'user_mobile', 'id_card_num', 'true_name']].rename(columns={'user_mobile': '手机号码', 'create_time_tmu': '注册时间'})
# df_jjq = df_jjq.sort_values(by = '下单日期')
# df_jjq = df_jjq[['下单日期','create_time_tmu', 'user_mobile', 'id_card_num', 'true_name','product_name','颜色','内存']].rename(columns={'user_mobile': '手机号码', 'create_time_tmu': '注册时间','product_name':'产品'})
# df_jjq.loc[:, 'gender'] = df_jjq.id_card_num.apply(lambda x: clean.get_gender(x))
# df_jjq.loc[:, '姓名'] = np.where(df_jjq.gender=='男', df_jjq.true_name.str[0]+'先生', df_jjq.true_name.str[0]+'女士')
# df_jjq = df_jjq[['注册时间','下单日期','手机号码', '姓名','产品','颜色','内存']]
# df_jjq = df_jjq[:50]
# with pd.ExcelWriter(f'F:/需求/力哥需求/赛租机数据_进件后取消{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_jjq.to_excel(writer, index=False)


## 疑似欺诈

In [55]:
# 资信作假/虚假资料、非本人申请、第三方指导电审、命中高风险地址
# df2_qx = df2[df2.取消原因2.str.contains(r'资信作假/虚假资料|非本人申请|第三方指导电审|命中高风险地址')==True]
# df2_qx.merge(df_mini, on='order_id', how='left')
# df2_qx = df2_qx[['下单日期', 'order_number', 'alipay_order_id', '取消原因2']]
# with pd.ExcelWriter('F:/需求/总需求/疑似欺诈.xlsx', engine='xlsxwriter') as writer:
#     df2_qx.to_excel(writer, sheet_name='疑似欺诈数据', index=False)

In [56]:
# df2_group = df2.groupby('下单月份').agg({'是否出库': 'sum', 'purchase_amount': 'sum', 'new_actual_money': 'sum'}).rename(columns={'是否出库': '出库数', 'purchase_amount': '采购金额', 'new_actual_money': '签约价'})
# df2_group.to_excel('F:/需求/七月需求/24年出库数据1.xlsx', sheet_name='24年出库数据')

In [57]:
# df_examine
# pd.pivot_table(df_examine, values='risk_trace_id', index='time', columns='strategy_241205', aggfunc='count')

In [58]:
# df_examine_merge = df.merge(df_examine, left_on='trace_id', right_on='risk_trace_id', how='right')
# # df_examine_merge = df_examine_merge[(df_examine_merge.strategy_241205.notna())|(df_examine_merge.strategy_241205!='None')]
# df_examine_merge['strategy_241205'].drop_duplicates()
# pd.pivot_table(df_examine_merge, values='order_id', index='下单日期', columns='strategy_241205', aggfunc='count')#.to_excel('F:/需求/总需求/241205.xlsx')

In [59]:
# trace关联
sql1='''-- trace关联表 
SELECT order_trace_id, risk_trace_id, trace_id, order_id
from db_credit.order_association
;
'''
trace_df = clean.query(sql1)
trace_df.shape

(3744930, 4)

In [162]:
sql_rent = '''
select trace_id, time, rentOnlineDetailLevelOverall -- 综合风险等级
, out_order_no from db_credit.risk_alipay_auth_query
'''
df_rent = clean.query(sql_rent)
df_rule_rent = trace_df.merge(df_rent, on='trace_id', how='left')
# df_rent_new = df[df.是否进件==1].merge(df_rule_rent, on='order_id', how='left')
# df_rent_new = df_rent_new[['下单日期', 'order_id', '机审强拒', 'rentOnlineDetailLevelOverall', '是否进件', '是否出库']]

In [163]:
# df_rent_new_jj = pd.pivot_table(df_rent_new, values='是否进件', index='下单日期', columns='rentOnlineDetailLevelOverall', aggfunc='sum')
# df_rent_new_js = pd.pivot_table(df_rent_new, values='机审强拒', index='下单日期', columns='rentOnlineDetailLevelOverall', aggfunc='sum')
# df_rent_new_ck = pd.pivot_table(df_rent_new, values='是否出库', index='下单日期', columns='rentOnlineDetailLevelOverall', aggfunc='sum')
# with pd.ExcelWriter('F:/需求/总需求/细化综合风险等级.xlsx', engine='xlsxwriter') as writer:
#     df_rent_new_jj.to_excel(writer, sheet_name='进件')
#     df_rent_new_js.to_excel(writer, sheet_name='机审强拒')
#     df_rent_new_ck.to_excel(writer, sheet_name='出库')

In [164]:
# df2_ck2023 = df2[(df2.是否出库==1)&(df2.年份==2023)].contract_price.sum()
# df2_ck2024 = df2[(df2.是否出库==1)&(df2.年份==2024)].contract_price.sum()
# df2_ck2024

## 筛选手机品类

In [ ]:
# df_phone1 = df[(df.classify_id==116)|(df.classify_id==165)]
# df_phone1.loc[:, '机型'] = df_phone1.product_name.str.replace(' ', '').str.extract(r'(iPhone?(\d+)(ProMax|Pro|Plus|mini)?)')[0].str.replace(' ', '').drop_duplicates()
# df_phone1.loc[:, '机型']  = df_phone1.product_name.str.replace(' ', '').str.extract(fr'({iPhone_str}|华为 ?(nova|Mate|P) ?\d+ ?|Mate ?\d+ ?(Pro)?)|Redmi ?K ?(\d+) ?|REDMI ?(K) ?\d+ ?|vivo ?(S|X) ?\d+ ?(Pro|e)?|OPPO ?(Reno|K|Find) ?(N3|X5|\d+) ?(Pro+|Pro)?|小米 ?(\d+)?(Pro|Ultra)?|oppo ?(一加) ?(\d+) ?(Pro)?|iQOO ?(Neo|\d+) ?(Pro)? |一加Ace ?(\d+)?|真我  ?(realme|GT) ?(V) ?\d+ ?|荣耀 ?(Magic|\d+) ?(\d+|Pro) ?(Ultra)?')[0].drop_duplicates()
# df_phone1[df_phone1.机型.isna()].product_name.drop_duplicates()
# df_phone2 = df2[(df2.classify_id==116)|(df.classify_id==165)]
# df_group = all_models.data_group(df_phone1, df_phone2, df_risk_examine, '型号内存')[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# df_group_20 = df_group.sort_values('出库', ascending=False).head(20)
# df_group_20.to_excel('F:/宏生/SKU出库前20.xlsx')

In [ ]:
# df_phone1[~df_phone1.product_name.str.contains(r'iPhone|华为|vivo|OPPO|Mate|REDMI|小米|Redmi|荣耀|一加|真我|iQOO')].product_name.drop_duplicates()

In [ ]:
# df2_ck = df2[df2.是否出库==1]
# pd.pivot_table(df2_ck, values='order_number', index='下单月份', columns='overdue_type', aggfunc='count')

In [ ]:
# 总体数据

df_all = all_models.data_group(df, df2, df_risk_examine, '下单日期')
df_all['出库（按下单时间）'] = df_all['出库']
df_all['出库（按转化时间）'] = df_weekday_zh['All'][:-1]

# df_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                 "出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]#.to_excel('F:/需求/七月需求/24年转化数据.xlsx')


In [ ]:
# 取出二手订单
# df_js = df2[df2.机审通过件==1][['order_number', '是否二手']]
# # 拒量
# df_241205 = df[df.tips.str.contains(r'策略241205|策略241212|命中自有模型回捞策略', regex=True)==True]
# df_241205 = df_241205[df_241205.merchant_name!='小蚂蚁租机']
# df_241205.loc[:, '出库前强拒_拒量'] = np.where(((df_241205.qvt_risk == '0')|(df_241205.qvt_risk.isna()))&(df_241205.qvt_result == '1'), 1, 0)
# df_241205_js = df_241205[df_241205.出库前强拒_拒量==1][['order_number', '是否二手']]
# # 小蚂蚁
# df_xmy_new, df_xmy_new_g = all_models.xmy(df, df_risk_examine, '下单日期')
# df_xmy_new_js = df_xmy_new[df_xmy_new.小蚂蚁人审拒绝==1][['order_number', '是否二手']]
# # 匹配小蚂蚁数据
# df_xmy = df[(df['merchant_name'] == '小蚂蚁租机')|(df['merchant_name'] == '兴鑫兴通讯')]
# # 出库前强拒数据重命名
# df_risk_examine.rename(columns={'time': 'time_risk_ex', 'status': 'status_risk_ex'}, inplace=True)
# # 对小蚂蚁数据和出库前强拒数据进行拼接
# df_risk_examine_all = pd.merge(df_xmy, df_risk_examine, left_on='id_card_num', right_on='id_card', how='inner')
# # 计算出库前强拒的订单数
# df_risk_examine_all2 = df_risk_examine_all[(df_risk_examine_all['time_risk_ex'] < df_risk_examine_all['update_time'])]
# # 进行排序并取到最近的一个订单
# df_risk_examine_all2 = df_risk_examine_all2.sort_values(['order_id', 'time_risk_ex'], ascending=[True, False]).groupby('order_id').head(1)
# df_risk_examine_all2 = df_risk_examine_all2[df_risk_examine_all2['status_risk_ex'] == '1']
# df_risk_examine_all2_js = df_risk_examine_all2[['order_number', '是否二手']]

In [ ]:
# df_concat = pd.concat([df_js, df_241205_js, df_xmy_new_js, df_risk_examine_all2_js])
# with pd.ExcelWriter('F:/需求/总需求/机审通过.xlsx', engine='xlsxwriter') as writer:
#     df_all[['去重订单数','机审通过件']].to_excel(writer, sheet_name='汇总')
#     df_concat.to_excel(writer, sheet_name='二手手机', index=False)

In [ ]:
# df_all[["去重订单数","进件数",'出库（按下单时间）','进件出库率']][-10:]

# df_all[['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审强拒','强拒比例','机审通过件','人审拒绝','风控通过件','风控通过率', '客户取消','无法联系',
#         '出库前风控强拒','待审核','出库']]

In [ ]:
# df_xmy_all, df_xmy = all_models.xmy(df, df_risk_examine, '下单日期')
# df_xmy_all[(df_xmy_all.status_risk_ex=='1')&(df_xmy_all.下单日期=='2025-02-28')]['order_number'].shape

In [ ]:

# df_241205 = df[df.tips.str.contains(r'策略241205|策略241212|命中自有模型回捞策略', regex=True)==True]
# 出库前强拒_拒量状态设置
# df_241205.loc[:, '出库前强拒_拒量'] = np.where((df_241205.qvt_risk == '0')&(df_241205.qvt_result == '1'), 1, 0)
# df_241205_o = df_241205[(df_241205.出库前强拒_拒量==1)&(df_241205.下单日期=='2025-02-28')][['下单日期', 'order_number']]

In [ ]:
# df2_o = df2[(df2.机审通过件==1)&(df2.下单日期=='2025-02-28')][['下单日期', 'order_number']]
# df_new = pd.concat([df_241205_o, df2_o])
# df_new.to_excel('F:/需求/总需求/机审通过件.xlsx', index=False)

In [ ]:
# df_all2 = all_models.data_group_hour(df, df2, df_risk_examine, ['下单日期', 'hour'], 14)
# df_all2 = df_all2[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# with pd.ExcelWriter(f'F:/需求/瑞彤需求/渠道转化_{t_date}.xlsx', engine='openpyxl') as writer:
#     df_all2[-7:].to_excel(writer, sheet_name='总体')

## 二手手机

In [ ]:
# # 获取二手数据
# df_two = df[df['是否二手']==1]
# df_two2 = df2[df2['是否二手']==1]
# # 提取机型
# df_two['机型'] = df_two['product_name'].str.extract(r'(iPhone ?\d+(ProMax|Pro|Plus|mini)?|OPPO ?(Reno|K|Find) ?(N3|X5|\d+) ?(Pro+|Pro)?|华为 ?(nova|Mate|P) ?\d+ ?|Mate ?\d+ ?(Pro)?|真我  ?(realme) ?(V) ?\d+ ?|vivo ?(S|X) ?\d+ ?(Pro|e)? |Redmi ?(K) ?\d+ ?)')[0]
# df_two2['机型'] = df_two2['product_name'].str.extract(r'(iPhone ?\d+(ProMax|Pro|Plus|mini)?|OPPO ?(Reno|K|Find) ?(N3|X5|\d+) ?(Pro+|Pro)?|华为 ?(nova|Mate|P) ?\d+ ?|Mate ?\d+ ?(Pro)?|真我  ?(realme) ?(V) ?\d+ ?|vivo ?(S|X) ?\d+ ?(Pro|e)? |Redmi ?(K) ?\d+ ?)')[0]


In [ ]:
# # 日期分组
# df_two_all = all_models.data_group(df_two, df_two2, df_risk_examine, '下单日期')
# df_two_all = df_two_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# # 机型分组
# df_two_all_model = all_models.data_group(df_two, df_two2, df_risk_examine, '机型').sort_values('进件数', ascending=False)
# df_two_all_model = df_two_all_model[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]



## 总体自然周周数据

In [ ]:
# week_models.week_data_group_all(df_all[["去重订单数","前置拦截",'进件前取消', "进件数","机审强拒","机审通过件","人审拒绝","风控通过件","客户取消","无法联系","出库前风控强拒","待审核",'出库']], 7)

## 租售比（下次跑时需验证逻辑是否无误）

In [ ]:
# df2_ck[df2_ck['成本'].notna()][['订单号','租赁方案','product_name', '成本', '残值']]

In [49]:
dfck2 = pd.merge(df_ck[['订单号', '成本', '类型']],df2,left_on="订单号", right_on="order_number")
dfck2.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)
# # 删除 露营设备 出库 
dfck2.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)
dfck2.loc[:, '残值'] = dfck2.apply(getrecanzhi, axis=1)
dfck2.loc[:, '总应收'] = dfck2.all_rental+dfck2.残值+dfck2.溢价费
# 筛选purchase_amount成本大于0是因为存在退款的订单，导致成本为负数
df2_ck = dfck2[(dfck2.是否出库==1)&(dfck2.purchase_amount>0)]
# contract_price 签约价 2024-09后 总应收:total_receivable 
# df2_ck.loc[:, '残值估算毛利'] = np.where(df2_ck.下单月份<='2024-09', df2_ck.总应收 - df2_ck.成本, df2_ck.total_receivable - df2_ck.成本)
df2_ck.loc[:, '残值估算毛利'] = df2_ck.总应收 - df2_ck.成本
df2_ck.loc[:, '买断估算毛利'] = df2_ck.new_actual_money - df2_ck.成本+df2_ck.溢价费
df2_ck_group = df2_ck.groupby('年份').agg({'all_deposit': 'mean', 'all_rental': 'mean', '成本': 'mean', '残值估算毛利': 'mean', '买断估算毛利': 'mean'}).rename(columns={'all_rental': '总租金均值', 'all_deposit': '总押金均值', '成本': '采购成本均值'})
df2_ck_group.loc[:, '租售比'] = (df2_ck_group.总租金均值/df2_ck_group.总押金均值).map(lambda x: format(x,".2%"))
df2_ck_group.loc[:, '买断估算毛利率'] = (df2_ck_group.买断估算毛利/df2_ck_group.采购成本均值).map(lambda x: format(x,".2%"))
df2_ck_group.loc[:, '残值估算毛利率'] = (df2_ck_group.残值估算毛利/df2_ck_group.采购成本均值).map(lambda x: format(x,".2%"))

with pd.ExcelWriter('F:/需求/瑞彤需求/租售比_11.13.xlsx', engine='openpyxl') as writer:
    df2_ck_group.to_excel(writer, sheet_name='总体租售比')#2945.949547,3970.444563417388
df2_ck_group

,总押金均值,总租金均值,采购成本均值,残值估算毛利,买断估算毛利,租售比,买断估算毛利率,残值估算毛利率
年份,,,,,,,,
2025,9858.198474,6524.117642,8435.683518,2999.174843,4223.360565,66.18%,50.07%,35.55%


In [ ]:
# df2_ck[['all_rental','残值','溢价费']]

# 搜索渠道转化

In [ ]:
df_ss = df[df['归属渠道']=='搜索渠道']
df_ss2 = df2[df2['归属渠道']=='搜索渠道']
# df_ss_group = all_models.data_group(df_ss, df_ss2, df_risk_examine, '下单日期')

# df_ss_group['出库（按下单时间）'] = df_ss_group['出库']
# df_ss_group['出库（按转化时间）'] = df_weekday_zh['搜索渠道'][:-1]
# df_ss_groups = df_ss_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# df_ss_groups = df_ss_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# df_ss_groups
# with pd.ExcelWriter('F:/需求/丹琦需求/转化数据.xlsx', engine='openpyxl') as writer:
#     df_ss_groups.to_excel(writer, sheet_name='搜索渠道')
# df_ss_group[['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审强拒','强拒比例','机审通过件','人审拒绝','风控通过件','风控通过率', '客户取消','无法联系',
#         '出库前风控强拒','待审核','出库']]
# df_ss_groups

In [ ]:
# df_ss2_5 = df_ss2[(df_ss2.下单月份=='2025-05')&(df_ss2.是否进件==1)]
# df_ss2_6 = df_ss2[(df_ss2.下单月份=='2025-06')&(df_ss2.是否进件==1)]
# df_5_group = df_ss2_5.groupby('取消原因2').agg({'order_id': 'count'}).rename(columns={'order_id': '数量'})
# df_6_group = df_ss2_6.groupby('取消原因2').agg({'order_id': 'count'}).rename(columns={'order_id': '数量'})
# df_ss_5 = df_ss[(df_ss.下单月份=='2025-05')&(df_ss2.人审拒绝==1)]
# df_ss_6 = df_ss[(df_ss.下单月份=='2025-06')&(df_ss2.人审拒绝==1)]
# df_5_group1 = df_ss_5.groupby('取消原因2').agg({'order_id': 'count'}).rename(columns={'order_id': '数量'})
# df_6_group1 = df_ss_6.groupby('取消原因2').agg({'order_id': 'count'}).rename(columns={'order_id': '数量'})
# with pd.ExcelWriter('F:/需求/丹琦需求/取消_拒绝原因.xlsx', engine='xlsxwriter') as writer:
#     df_5_group.to_excel(writer, sheet_name='5月份取消原因')
#     df_6_group.to_excel(writer, sheet_name='6月份取消原因')
#     df_5_group1.to_excel(writer, sheet_name='5月份拒绝理由')
#     df_6_group1.to_excel(writer, sheet_name='6月份拒绝理由')

## 按周分组，搜索渠道非机审拒绝的订单(瑞彤需求)

In [ ]:
# df_ss = df[df['归属渠道']=='搜索渠道']
# # 将下单周转换为时间范围格式，例如"2025.8.20-2025.8.24"
# def get_week_range(date):
#     # 获取该日期所在周的周一
#     monday = date - pd.Timedelta(days=date.weekday())
#     # 获取该日期所在周的周日
#     sunday = monday + pd.Timedelta(days=6)
#     # 格式化为"年.月.日-年.月.日"格式
#     return f"{monday.year}.{monday.month}.{monday.day}-{sunday.year}.{sunday.month}.{sunday.day}"

# df_ss_quyy = df_ss[((df_ss['机审强拒'] == 0)&((df_ss['人审拒绝']==1)|(df_ss['客户取消']==1)|(df_ss['无法联系']==1)|(df_ss['出库前风控强拒']==1)))]
# # 按星期设置列：下单周，周一至周日为一周，未满一周按一周算，eg:8月20到8月24，9月1日到9月4日都算一周
# df_ss_quyy['下单周范围'] = df_ss_quyy['下单日期'].apply(get_week_range)

# df_ss_quyy['取消原因2'] = df_ss_quyy['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_ss_quyy['取消原因2'].drop_duplicates()
# df_ss_quyy_group = df_ss_quyy.groupby(['下单周范围', '取消原因2']).agg({'order_number': 'count'}).rename(columns={'order_number': '取消数'})
# df_ss_quyy_group = df_ss_quyy_group.sort_values(['下单周范围', '取消数'], ascending=[True, False])
# df_ss_quyy_group['取消率'] = df_ss_quyy_group['取消数']/(df_ss_quyy['是否进件'].sum())
# df_ss_quyy_group['取消率'] = df_ss_quyy_group['取消率'].apply(lambda x: format(x, '.2%'))
# # df_ss_quyy_group.to_excel('F:/需求/瑞彤需求/搜索非机审拒绝的取消原因.xlsx', sheet_name='订单取消原因')
# df_ss_quyy_group

In [ ]:
# df_ss_jg = df_ss_quyy[df_ss_quyy.取消原因2=='不接受监管设备'][['下单日期','order_number','机型内存']]
# df_ss_jg.to_excel('F:/需求/瑞彤需求/搜索不接受监管设备.xlsx', sheet_name='订单明细',index=False)

## 租售比(瑞彤需求)

In [50]:
df_ss2 = dfck2[dfck2.归属渠道=='搜索渠道']
df_ss2_ck = df_ss2[(df_ss2.是否出库==1)&(df_ss2.purchase_amount>0)]

# df_ss2_ck.loc[:, '残值估算毛利'] = np.where(df_ss2_ck.下单月份<='2024-09', df_ss2_ck.总应收 - df_ss2_ck.成本, df_ss2_ck.total_receivable - df_ss2_ck.采购金额)
# df_ss2_ck.loc[:, '买断估算毛利'] = df_ss2_ck.contract_price - df_ss2_ck.采购金额
df_ss2_ck.loc[:, '残值估算毛利'] = df_ss2_ck.总应收 - df_ss2_ck.成本
df_ss2_ck.loc[:, '买断估算毛利'] = df_ss2_ck.new_actual_money - df_ss2_ck.成本+df_ss2_ck.溢价费

df_ss2_ck_ck_group = df_ss2_ck.groupby('年份').agg({'all_deposit': 'mean', 'all_rental': 'mean', '成本': 'mean', '残值估算毛利': 'mean', '买断估算毛利': 'mean'}).rename(columns={'all_rental': '总租金均值', 'all_deposit': '总押金均值', '成本': '采购成本均值'})
df_ss2_ck_ck_group.loc[:, '租售比'] = (df_ss2_ck_ck_group.总租金均值/df_ss2_ck_ck_group.总押金均值).map(lambda x: format(x,".2%"))
df_ss2_ck_ck_group.loc[:, '买断估算毛利率'] = (df_ss2_ck_ck_group.买断估算毛利/df_ss2_ck_ck_group.采购成本均值).map(lambda x: format(x,".2%"))
df_ss2_ck_ck_group.loc[:, '残值估算毛利率'] = (df_ss2_ck_ck_group.残值估算毛利/df_ss2_ck_ck_group.采购成本均值).map(lambda x: format(x,".2%"))

with pd.ExcelWriter('F:/需求/瑞彤需求/租售比_11.13.xlsx', engine='openpyxl', mode='a') as writer:
    df_ss2_ck_ck_group.to_excel(writer, sheet_name='搜索渠道租售比')
df_ss2_ck_ck_group

,总押金均值,总租金均值,采购成本均值,残值估算毛利,买断估算毛利,租售比,买断估算毛利率,残值估算毛利率
年份,,,,,,,,
2025,10033.694863,7521.752509,8603.1067,3885.306201,4272.580746,74.96%,49.66%,45.16%


In [60]:
df_ss = df[df['归属渠道']=='搜索渠道']
df_ss2 = df2[df2['归属渠道']=='搜索渠道']
# df_ss_group = all_models.data_group(df_ss, df_ss2, df_risk_examine, '下单日期')

# df_ss_group['出库（按下单时间）'] = df_ss_group['出库']
# df_ss_group['出库（按转化时间）'] = df_weekday_zh['搜索渠道'][:-1]
# df_ss_groups = df_ss_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# df_ss_groups = df_ss_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# df_ss_groups
# with pd.ExcelWriter('F:/需求/丹琦需求/转化数据.xlsx', engine='openpyxl') as writer:
#     df_ss_groups.to_excel(writer, sheet_name='搜索渠道')
# df_ss_group[['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审强拒','强拒比例','机审通过件','人审拒绝','风控通过件','风控通过率', '客户取消','无法联系',
#         '出库前风控强拒','待审核','出库']]
# df_ss_groups

In [61]:
df_ss2_5 = df_ss2[(df_ss2.下单月份=='2025-09')&(df_ss2.是否进件==1)]# 
df_ss2_6 = df_ss2[(df_ss2.下单月份=='2025-10')&(df_ss2.是否进件==1)]
df_5_group = df_ss2_5.groupby('取消原因2').agg({'order_id': 'count'}).rename(columns={'order_id': '数量'})
df_6_group = df_ss2_6.groupby('取消原因2').agg({'order_id': 'count'}).rename(columns={'order_id': '数量'})
df_ss_5 = df_ss2[(df_ss2.下单月份=='2025-09')&(df_ss2.人审拒绝==1)]# 
df_ss_6 = df_ss2[(df_ss2.下单月份=='2025-10')&(df_ss2.人审拒绝==1)]
df_5_group1 = df_ss_5.groupby('取消原因2').agg({'order_id': 'count'}).rename(columns={'order_id': '数量'})
df_6_group1 = df_ss_6.groupby('取消原因2').agg({'order_id': 'count'}).rename(columns={'order_id': '数量'})
with pd.ExcelWriter(f'F:/需求/丹琦需求/取消_拒绝原因_{t_date}.xlsx', engine='xlsxwriter') as writer:
    df_5_group.to_excel(writer, sheet_name='9月份取消原因')
    df_6_group.to_excel(writer, sheet_name='10月份取消原因')
    df_5_group1.to_excel(writer, sheet_name='9月份拒绝理由')
    df_6_group1.to_excel(writer, sheet_name='10月份拒绝理由')

## iPhone16ProMax512G

In [ ]:
# df_ss_iPhone = df_ss[(df_ss.product_name.str.replace(' ', '').str.contains('iPhone16ProMax', regex=True))&(df_ss.内存.str.contains('512', regex=True))]
# df_ss_iPhone2 = df_ss2[(df_ss2.product_name.str.replace(' ', '').str.contains('iPhone16ProMax', regex=True))&(df_ss2.内存.str.contains('512', regex=True))]
# df_ss_iPhone_group = all_models.data_group(df_ss_iPhone, df_ss_iPhone2, df_risk_examine, '下单日期')[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                 "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# with pd.ExcelWriter('F:/需求/瑞彤需求/iPhone16ProMax512.xlsx', engine='xlsxwriter') as writer:
#     df_ss_iPhone_group.to_excel(writer, sheet_name='搜索渠道')
#     df_ss_iPhone[['order_number', '取消原因']].to_excel(writer, sheet_name='搜索渠道取消原因')

In [ ]:
# df_ss_group[["去重订单数","进件数",'出库（按下单时间）','进件出库率']][-10:]
# data_group2(df_ss, '下单日期')[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库']]

In [ ]:
# df_ss_group2 = all_models.data_group_hour(df_ss, df_ss2, df_risk_examine, ['下单日期', 'hour'], 14)
# df_ss_group2 = df_ss_group2[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# 
# with pd.ExcelWriter(f'F:/需求/瑞彤需求/渠道转化_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:
#     df_ss_group2[-7:].to_excel(writer, sheet_name='搜索渠道')

In [ ]:
# df_ss2_ck = df_ss2[df_ss2.是否出库==1]
# 续租：relet_status=2；买断：has_actual=1&status2='已完成'；应归还：has_actual=0&status2='已完成'；
# df_ss2_ck.loc[:, '续租'] = np.where(df_ss2_ck.relet_status==2, 1, 0)
# df_ss2_ck.loc[:, '买断'] = np.where((df_ss2_ck.has_actual==1)&(df_ss2_ck.status2=='已完成'), 1, 0)
# df_ss2_ck.loc[:, '归还'] = np.where((df_ss2_ck.has_actual==0)&(df_ss2_ck.status2=='已完成'), 1, 0)
# df_ss2_ck_group = df_ss2_ck.groupby('下单月份').agg({'order_number': 'count', '续租': 'sum', '买断': 'sum', '归还': 'sum'}).rename(columns={'order_number': '出库数'})
# df_ss2_ck_group.loc[:, '续租率'] = (df_ss2_ck_group.续租/df_ss2_ck_group.出库数).map(lambda x: format(x, '.2%'))
# df_ss2_ck_group.loc[:, '买断率'] = (df_ss2_ck_group.买断/df_ss2_ck_group.出库数).map(lambda x: format(x, '.2%'))
# df_ss2_ck_group.loc[:, '归还率'] = (df_ss2_ck_group.归还/df_ss2_ck_group.出库数).map(lambda x: format(x, '.2%'))
# 
# with pd.ExcelWriter('F:/需求/瑞彤需求/续租买断归还占比.xlsx', engine='openpyxl') as writer:
#     df_ss2_ck_group.to_excel(writer, sheet_name='搜索渠道')

## 支付宝UID

In [ ]:
# df_ss2_uid = df_ss2[(df_ss2.是否出库==1)&(df_ss2.碎屏险价格.notna())]
# df_ss2_uid_new = df_ss2_uid[['下单日期', 'order_number', 'payer_user_id', 'product_name', '内存', '租赁方案']].rename(columns={'order_number': '订单号', 'payer_user_id': '支付宝uid', 'product_name': '产品名称'})
# df_ss2_uid_new.to_excel('F:/需求/瑞彤需求/搜索渠道购买碎屏险出库订单.xlsx', index=False)

## 二手手机

In [ ]:
# df_ss_2 = df_ss[df_ss['是否二手']==1]
# df_ss2_2 = df_ss2[df_ss2['是否二手']==1]
# df_ss_2_group = all_models.data_group(df_ss_2, df_ss2_2, df_risk_examine, '下单日期')
# df_ss_2_group = df_ss_2_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# df_ss_2_group

## 搜索渠道自然周周数据

In [ ]:
# week_ss_group = week_models.week_data_group_all(df_ss_group[["去重订单数","前置拦截",'进件前取消', "进件数","机审强拒","机审通过件","人审拒绝","风控通过件","客户取消","无法联系","出库前风控强拒","待审核",'出库']], 7)[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","机审强拒比例","机审通过件","人审拒绝","风控通过件","风控通过件比例","客户取消","无法联系", "出库前风控强拒","待审核",'出库','进件出库率','客户取消比例','人审拒绝比例','出库前风控强拒比例','无法联系比例','订单出库率']]

# 支付宝社群（婷玉需求）

In [49]:
# TO_DO婷玉，支付宝需求# TO_DO日常
df_zfbsq = df[df['activity_name'].str.contains('群')==True]
df_zfbsq2 = df2[df2['activity_name'].str.contains('群')==True]
df_zfbsq_group = all_models.data_group_contain_hl(df_zfbsq, df_zfbsq2, df_risk_examine, '下单日期')

# df_ss_group['出库（按下单时间）'] = df_weekday['搜索渠道'][:-1]
df_zfbsq_group['出库（按下单时间）'] = df_zfbsq_group['出库']
# df_zfbsq_group['出库（按转化时间）'] = df_weekday_zh['支付宝社群'][:-1]
df_zfbsq_group = df_zfbsq_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# 支付宝来源渠道转化    
df_zfb_ly = df[df['来源渠道'].str.contains('群')==True]
df_zfb_ly2 = df2[df2['来源渠道'].str.contains('群')==True]
df_zfb_ly_group = all_models.data_group_contain_hl(df_zfb_ly, df_zfb_ly2, df_risk_examine, '下单日期')
df_zfb_ly_group['出库（按下单时间）'] = df_zfb_ly_group['出库']
df_zfb_ly_group = df_zfb_ly_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]


with pd.ExcelWriter(f'F:/需求/崔婷玉需求/支付宝社群_{t_date}.xlsx', engine='openpyxl') as writer:
    df_zfbsq_group.to_excel(writer, sheet_name='支付宝社群')
    df_zfb_ly_group.to_excel(writer, sheet_name='来源渠道')


In [51]:
# df_zfbsq[df_zfbsq['是否出库']==1][['order_number', 'merchant_name']]
# df_zfbsq[(df_zfbsq2['order_number']=='A20250811004758200')&(df_zfbsq["是否出库"]==1)]
df_zfbsq_group

,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
下单日期,,,,,,,,,,,,,,,,,,,,,,
2025-11-24,1,1,100.00%,0,0.00%,0.0,nan%,0.0,0.0,0.0,nan%,0,0,0.0,0,0,nan%,nan%,nan%,nan%,nan%,0.00%
2025-11-25,3,1,33.33%,1,33.33%,1.0,100.00%,0.0,0.0,0.0,0.00%,0,0,0.0,0,0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
2025-11-26,2,0,0.00%,0,0.00%,0.0,nan%,0.0,0.0,0.0,nan%,0,0,0.0,0,0,nan%,nan%,nan%,nan%,nan%,0.00%
2025-11-27,2,1,50.00%,1,50.00%,0.0,0.00%,1.0,0.0,1.0,100.00%,0,0,1.0,0,0,0.00%,0.00%,0.00%,100.00%,0.00%,0.00%
2025-11-28,2,0,0.00%,0,0.00%,0.0,nan%,0.0,0.0,0.0,nan%,0,0,0.0,0,0,nan%,nan%,nan%,nan%,nan%,0.00%


In [52]:
# df_zfb_ly2[(df_zfb_ly2['是否出库']==1)]
df_zfb_ly_group

,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
下单日期,,,,,,,,,,,,,,,,,,,,,,
2025-11-24,5,3,60.00%,0,0.00%,0.0,nan%,0.0,0.0,0.0,nan%,0,0,0.0,0,0,nan%,nan%,nan%,nan%,nan%,0.00%
2025-11-25,7,3,42.86%,1,14.29%,1.0,100.00%,0.0,0.0,0.0,0.00%,0,0,0.0,0,0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
2025-11-26,6,0,0.00%,1,16.67%,1.0,100.00%,0.0,0.0,0.0,0.00%,0,0,0.0,0,0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
2025-11-27,6,1,16.67%,0,0.00%,0.0,nan%,0.0,0.0,0.0,nan%,0,0,0.0,0,0,nan%,nan%,nan%,nan%,nan%,0.00%
2025-11-28,3,0,0.00%,0,0.00%,0.0,nan%,0.0,0.0,0.0,nan%,0,0,0.0,0,0,nan%,nan%,nan%,nan%,nan%,0.00%
2025-11-29,2,0,0.00%,0,0.00%,0.0,nan%,0.0,0.0,0.0,nan%,0,0,0.0,0,0,nan%,nan%,nan%,nan%,nan%,0.00%
2025-11-30,5,0,0.00%,0,0.00%,0.0,nan%,0.0,0.0,0.0,nan%,0,0,0.0,0,0,nan%,nan%,nan%,nan%,nan%,0.00%


## 来源归属渠道转化

In [ ]:
# df_zfb_ly = df[df['来源渠道'].str.contains('群')==True]
# df_zfb_ly2 = df2[df2['来源渠道'].str.contains('群')==True]
# df_zfb_ly_group = all_models.data_group(df_zfb_ly, df_zfb_ly2, df_risk_examine, '下单日期')
# df_zfb_ly_group['出库（按下单时间）'] = df_zfb_ly_group['出库']
# df_zfb_ly_group = df_zfb_ly_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]

# df_zfb_gs = df[df['归属渠道'].str.contains('群')==True]
# df_zfb_gs2 = df2[df2['归属渠道'].str.contains('群')==True]
# df_zfb_gs_group = all_models.data_group(df_zfb_gs, df_zfb_gs2, df_risk_examine, '下单日期')
# df_zfb_gs_group['出库（按下单时间）'] = df_zfb_gs_group['出库']
# df_zfb_gs_group = df_zfb_gs_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# # 写入excel
# with pd.ExcelWriter(f'F:/需求/崔婷玉需求/来源归属渠道转化_{t_date}.xlsx', engine='openpyxl') as writer:
#     df_zfb_ly_group.to_excel(writer, sheet_name='来源渠道')
#     df_zfb_gs_group.to_excel(writer, sheet_name='归属渠道')

In [ ]:
# df_zfb_ly['来源渠道'].unique()

In [ ]:
# df_zfb_gs['归属渠道'].unique()

# 单人聊天 转化

In [ ]:
df_dr = df[df['来源渠道']=='单人聊天会话中的小程序消息卡片（分享）']
df_dr2 = df2[df2['来源渠道']=='单人聊天会话中的小程序消息卡片（分享）']
# df_dr_group = all_models.data_group(df_dr, df_dr2, df_risk_examine,  '下单日期')
# 
# # df_dr_group['出库（按下单时间）'] = df_weekday['单人聊天会话中的小程序消息卡片（分享）'][:-1]
# df_dr_group['出库（按下单时间）'] = df_dr_group['出库']
# df_dr_group['出库（按转化时间）'] = df_weekday_zh['单人聊天会话中的小程序消息卡片（分享）'][:-1]
# df_dr_groups = df_dr_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例",'出库（按下单时间）','出库（按转化时间）','进件出库率','订单出库率']]

In [ ]:
# df_dr2_id = df_dr2[df_dr2.是否进件==1][['下单日期', 'zfb_open_id']]
# df2_id = df2[(df2.expressage.str.contains('暂无法联系到收方客户', regex=False)==True)&(df2.order_type=='ZFB_ORDER')][['下单日期', 'zfb_open_id']]
# with pd.ExcelWriter('F:/需求/总需求/异常-中介数据.xlsx', engine='xlsxwriter') as writer:
#     df2_id.to_excel(writer, sheet_name='派送异常订单', index=False)
#     df_dr2_id.to_excel(writer, sheet_name='中介订单', index=False)

## 单人会话自然周周数据

In [ ]:
# week_dr_group = week_models.week_data_group_all(df_dr_group[["去重订单数","前置拦截",'进件前取消', "进件数","机审强拒","机审通过件","人审拒绝","风控通过件","客户取消","无法联系","出库前风控强拒","待审核",'出库',]], 7)[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","机审强拒比例",'出库','进件出库率','订单出库率']]


# 芝麻渠道转化

In [ ]:
# df_zm = df[df['归属渠道']=='芝麻租物']
# df_zm2 = df2[df2['归属渠道']=='芝麻租物']
# df_zm_group = all_models.data_group(df_zm, df_zm2, df_risk_examine, '下单日期')
# df_zm_group['出库（按下单时间）'] = df_zm_group['出库']
# df_zm_group['出库（按转化时间）'] = df_weekday_zh['芝麻租物'][:-1]
# df_zm_groups = df_zm_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒", "待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# df_zm_groups
# df_zm_group[['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审强拒','强拒比例','机审通过件','人审拒绝','风控通过件','风控通过率', '客户取消','无法联系',
#         '出库前风控强拒','待审核','出库']]

In [ ]:
# df_zm2_ck = df_zm2[df_zm2.是否出库==1]
# pd.pivot_table(df_zm2_ck, values='order_number', index='下单月份', columns='overdue_type', aggfunc='count')


In [ ]:
# df_rent_new = df_zm[df_zm.是否进件==1].merge(df_rule_rent, on='order_id', how='left')
# df_rent_new = df_rent_new[['下单日期', 'order_id', 'rentOnlineDetailLevelOverall', '是否进件','机审强拒', '出库前风控强拒', '是否出库']]
# df_rent_new['rentOnlineDetailLevelOverall'] = df_rent_new.rentOnlineDetailLevelOverall.fillna('-2')
# df_rent_new_jj = pd.pivot_table(df_rent_new, values='是否进件', index='下单日期', columns='rentOnlineDetailLevelOverall', aggfunc='sum')
# df_rent_new_js = pd.pivot_table(df_rent_new, values='机审强拒', index='下单日期', columns='rentOnlineDetailLevelOverall', aggfunc='sum')
# df_rent_new_fk = pd.pivot_table(df_rent_new, values='出库前风控强拒', index='下单日期', columns='rentOnlineDetailLevelOverall', aggfunc='sum')
# df_rent_new_ck = pd.pivot_table(df_rent_new, values='是否出库', index='下单日期', columns='rentOnlineDetailLevelOverall', aggfunc='sum')
# with pd.ExcelWriter('F:/需求/总需求/细化综合风险等级_all.xlsx', engine='xlsxwriter') as writer:
#     df_rent_new_jj.to_excel(writer, sheet_name='进件')
#     df_rent_new_js.to_excel(writer, sheet_name='机审强拒')
#     df_rent_new_fk.to_excel(writer, sheet_name='出库前风控强拒')
#     df_rent_new_ck.to_excel(writer, sheet_name='出库')
# df_rent_new_jj

## iPhone16ProMax 512

In [ ]:
# df_zm_iPhone = df_zm[(df_zm.product_name.str.replace(' ', '').str.contains('iPhone16ProMax', regex=True))&(df_zm.内存.str.contains('512', regex=True))]
# df_zm_iPhone2 = df_zm2[(df_zm2.product_name.str.replace(' ', '').str.contains('iPhone16ProMax', regex=True))&(df_zm2.内存.str.contains('512', regex=True))]
# df_zm_iPhone_group = all_models.data_group(df_zm_iPhone, df_zm_iPhone2, df_risk_examine, '下单日期')[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                 "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# with pd.ExcelWriter('F:/需求/瑞彤需求/iPhone16ProMax512.xlsx', engine='openpyxl', mode='a') as writer:
#     df_zm_iPhone_group.to_excel(writer, sheet_name='芝麻租物')
#     df_zm_iPhone[['order_number', '取消原因']].to_excel(writer, sheet_name='芝麻租物取消原因')

In [ ]:
# df_zm_group[["去重订单数","进件数",'出库（按下单时间）','进件出库率']][-10:]
# data_group2(df_zm, '下单日期')[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒",
#         "待审核",'出库']]

In [ ]:
# df_zm_group2 = all_models.data_group_hour(df_zm, df_zm2, df_risk_examine, ['下单日期', 'hour'], 14)
# df_zm_group2 = df_zm_group2[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# with pd.ExcelWriter(f'F:/需求/瑞彤需求/渠道转化_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:
#     df_zm_group2[-7:].to_excel(writer, sheet_name='芝麻租物')

## 二手手机

In [ ]:
# df_zm_2 = df_zm[df_zm['是否二手']==1]
# df_zm2_2 = df_zm2[df_zm2['是否二手']==1]
# df_zm_2_group = all_models.data_group(df_zm_2, df_zm2_2, df_risk_examine, '下单日期')
# df_zm_2_group = df_zm_2_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# df_zm_2_group

## 芝麻租物自然周周数据

In [ ]:
# week_zm_group = week_models.week_data_group_all(df_zm_group[["去重订单数","前置拦截",'进件前取消', "进件数","机审强拒","机审通过件","人审拒绝","风控通过件","客户取消","无法联系","出库前风控强拒","待审核",'出库',]], 7)[['week_group',"去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","机审强拒比例","机审通过件","人审拒绝","风控通过件","风控通过件比例","客户取消","无法联系","出库前风控强拒", "待审核",'出库','进件出库率','客户取消比例','人审拒绝比例','出库前风控强拒比例','无法联系比例','订单出库率']]


## 16PM转化

In [ ]:
# df_zm_16 = df_zm[df_zm['product_name'].str.contains('iPhone16')]
# df_zm_16['机型'] = df_zm_16['product_name'].str.extract(r'(iPhone ?\d+(ProMax|Pro|Plus)?)')[0]
# df_zm_162 = df_zm2[df_zm2['product_name'].str.contains('iPhone16')]
# df_zm_162['机型'] = df_zm_162['product_name'].str.extract(r'(iPhone ?\d+(ProMax|Pro|Plus)?)')[0]
# df_zm_16_group = all_models.data_group(df_zm_16, df_zm_162, df_risk_examine, '机型')
# df_zm_16_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒", "待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].to_excel('F:/宏生/16PM转化数据.xlsx')

## 租售比

In [51]:
df_zm2 = dfck2[dfck2.归属渠道=='芝麻租物']
df_zm2_ck = df_zm2[(df_zm2.是否出库==1)&(df_zm2.purchase_amount>0)]

# df_zm2_ck.loc[:, '残值估算毛利'] = np.where(df_zm2_ck.下单月份<='2024-09', df_zm2_ck.总应收 - df_zm2_ck.成本, df_zm2_ck.total_receivable - df_zm2_ck.成本)
# df_zm2_ck.loc[:, '买断估算毛利'] = df_zm2_ck.contract_price - df_zm2_ck.成本
df_zm2_ck.loc[:, '残值估算毛利'] = df_zm2_ck.总应收 - df_zm2_ck.成本
df_zm2_ck.loc[:, '买断估算毛利'] = df_zm2_ck.new_actual_money - df_zm2_ck.成本+df_zm2_ck.溢价费
df_zm2_ck_group = df_zm2_ck.groupby('年份').agg({'all_deposit': 'mean', 'all_rental': 'mean', '成本': 'mean', '残值估算毛利': 'mean', '买断估算毛利': 'mean'}).rename(columns={'all_rental': '总租金均值', 'all_deposit': '总押金均值', '成本': '采购成本均值'})
df_zm2_ck_group.loc[:, '租售比'] = (df_zm2_ck_group.总租金均值/df_zm2_ck_group.总押金均值).map(lambda x: format(x,".2%"))
df_zm2_ck_group.loc[:, '买断估算毛利率'] = (df_zm2_ck_group.买断估算毛利/df_zm2_ck_group.采购成本均值).map(lambda x: format(x,".2%"))
df_zm2_ck_group.loc[:, '残值估算毛利率'] = (df_zm2_ck_group.残值估算毛利/df_zm2_ck_group.采购成本均值).map(lambda x: format(x,".2%"))

with pd.ExcelWriter('F:/需求/瑞彤需求/租售比_11.13.xlsx', engine='openpyxl', mode='a') as writer:
    df_zm2_ck_group.to_excel(writer, sheet_name='芝麻租物租售比')
df_zm2_ck_group

,总押金均值,总租金均值,采购成本均值,残值估算毛利,买断估算毛利,租售比,买断估算毛利率,残值估算毛利率
年份,,,,,,,,
2025,9842.406261,5835.763053,8462.228025,2405.990541,4244.010595,59.29%,50.15%,28.43%


In [ ]:
# df_zm2_ck = df_zm2[df_zm2.是否出库==1]
# df_zm2_ck.loc[:, '续租'] = np.where(df_zm2_ck.relet_status==2, 1, 0)
# df_zm2_ck.loc[:, '买断'] = np.where((df_zm2_ck.status2=='已完成')&(df_zm2_ck.has_actual==1), 1, 0)
# df_zm2_ck.loc[:, '归还'] = np.where((df_zm2_ck.status2=='已完成')&(df_zm2_ck.has_actual==0), 1, 0)
# df_zm2_ck_group = df_zm2_ck.groupby('下单月份').agg({'order_number': 'count', '续租': 'sum', '买断': 'sum', '归还': 'sum'}).rename(columns={'order_number': '出库数'})
# df_zm2_ck_group.loc[:, '续租率'] = (df_zm2_ck_group.续租/df_zm2_ck_group.出库数).map(lambda x: format(x, '.2%'))
# df_zm2_ck_group.loc[:, '买断率'] = (df_zm2_ck_group.买断/df_zm2_ck_group.出库数).map(lambda x: format(x, '.2%'))
# df_zm2_ck_group.loc[:, '归还率'] = (df_zm2_ck_group.归还/df_zm2_ck_group.出库数).map(lambda x: format(x, '.2%'))
# 
# with pd.ExcelWriter('F:/需求/瑞彤需求/续租买断归还占比.xlsx', engine='openpyxl', mode='a') as writer:
#     df_zm2_ck_group.to_excel(writer, sheet_name='芝麻租物')

# 纯租物 转化

In [ ]:
#逻辑是否需要优化   
# df_zw = df[df['来源渠道']=='芝麻信用']
# df_zw2 = df2[df2['来源渠道']=='芝麻信用']
# df_zw_group = all_models.data_group(df_zw, df_zw2, df_risk_examine, '下单日期')
# 
# # df_zw_group['出库（按下单时间）'] = df_ly_ck['芝麻租物'][:-1]
# df_zw_group['出库（按下单时间）'] = df_zw_group['出库']
# df_zw_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例",'出库（按下单时间）','进件出库率','订单出库率']]

# 抖音转化

In [ ]:
df_dy = df[df['归属渠道']=='抖音渠道']
df_dy2 = df2[df2['归属渠道']=='抖音渠道']
# df_dy_group = all_models.data_group(df_dy, df_dy2, df_risk_examine, '下单日期')
# 
# # df_dy_group = df_dy_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","客户取消","取消率","人审拒绝","人审拒绝率","待审核","出库前风控强拒"]]  
# 
# df_dy_group['出库（按下单时间）'] = df_dy_group['出库']
# try:
#     # df_dy_group['出库（按下单时间）'] = df_weekday_zh['抖音渠道'][:-1]
#     df_dy_group['出库（按转化时间）'] = df_weekday_zh['抖音渠道'][:-1]
# except:
#     # df_dy_group['出库（按下单时间）'] = 0
#     df_dy_group['出库（按转化时间）'] = 0
# df_dy_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","客户取消","取消率","人审拒绝","人审拒绝率","待审核","出库前风控强拒",'出库（按下单时间）','出库（按转化时间）','进件出库率','订单出库率']]  
# df_dy

In [ ]:
# df_dy2_ck = df_dy2[df_dy2.是否出库==1]
# df_dy2_ck = df_dy2_ck[['下单日期', 'order_number', 'zf_name', 'company_name']].rename(columns={'order_number': '订单号', 'zf_name': '资方名称', 'company_name': '企业名称'})
# df_dy2_ck.to_excel('F:/需求/七月需求/抖音资方数据.xlsx', index=False)

In [ ]:
# df_dy_new = df_dy[(df_dy.下单月份=='2025-02')&(df_dy.客户取消==1)]

# df_dy_new['取消原因2'] = df_dy_new['cancel_reason'].str.split('：')
# df_dy_new['取消原因2'] = df_dy_new['取消原因2'].apply(lambda x: x[-1] if x is not None else x)
# df_dy_new.取消原因2.drop_duplicates()
# df_dy_new[['order_number', '取消原因2']].to_excel('F:/需求/廖志彧需求/抖音取消原因.xlsx', index=False)

In [ ]:
# 计算抖音历史出库数，毛利率和预收（含押金）
# df_dy2_ck = df_dy2[(df_dy2.是否出库==1)&(df_dy2.purchase_amount>1)][['下单月份', 'order_id', '是否出库', 'status2', 'purchase_amount', 'new_actual_money', 'all_deposit']]
# df_dy2_ck[df_dy2_ck.status2=='已完成']
# df_stages_1 = df_stages[df_stages.sort==1]
# df_dy_merge = df_dy2_ck.merge(df_stages_1, on='order_id', how='left')
# df_dy_merge.loc[:, '预收'] = df_dy_merge.part_payment+df_dy_merge.all_deposit
# df_dy_merge_g = df_dy_merge.groupby('下单月份').agg({'order_id': 'count', 'purchase_amount': 'sum', 'new_actual_money': 'sum', '预收': 'sum', 'all_deposit': 'sum'}).rename(columns={'order_id': '出库数', 'all_deposit': '押金'})
# df_dy_merge_g.loc[:, '应收毛利率'] = ((df_dy_merge_g.new_actual_money-df_dy_merge_g.purchase_amount)/df_dy_merge_g.purchase_amount).map(lambda x: format(x, '.2%'))
# df_dy_merge_g = df_dy_merge_g[['出库数', '预收', '押金', '应收毛利率']]
# with pd.ExcelWriter('F:/需求/廖志彧需求/抖音历史出库数据.xlsx', engine='xlsxwriter') as writer:
#     df_dy_merge_g.to_excel(writer, sheet_name='抖音数据')

# 京享租

## 租售比（瑞彤需求）

In [52]:
df_jd2 = dfck2[dfck2.归属渠道=='京东渠道']
df_jd2_ck = df_jd2[(df_jd2.是否出库==1)&(df_jd2.purchase_amount>0)]

# df_jd2_ck.loc[:, '残值估算毛利'] = np.where(df_jd2_ck.下单月份<='2024-09', df_jd2_ck.总应收 - df_jd2_ck.成本, df_jd2_ck.total_receivable - df_jd2_ck.采购金额)
# df_jd2_ck.loc[:, '买断估算毛利'] = df_jd2_ck.contract_price - df_jd2_ck.采购金额
df_jd2_ck.loc[:, '残值估算毛利'] = df_jd2_ck.总应收 - df_jd2_ck.成本
df_jd2_ck.loc[:, '买断估算毛利'] = df_jd2_ck.new_actual_money - df_jd2_ck.成本+df_jd2_ck.溢价费

df_jd2_ck_ck_group = df_jd2_ck.groupby('年份').agg({'all_deposit': 'mean', 'all_rental': 'mean', '成本': 'mean', '残值估算毛利': 'mean', '买断估算毛利': 'mean'}).rename(columns={'all_rental': '总租金均值', 'all_deposit': '总押金均值', '成本': '采购成本均值'})
df_jd2_ck_ck_group.loc[:, '租售比'] = (df_jd2_ck_ck_group.总租金均值/df_jd2_ck_ck_group.总押金均值).map(lambda x: format(x,".2%"))
df_jd2_ck_ck_group.loc[:, '买断估算毛利率'] = (df_jd2_ck_ck_group.买断估算毛利/df_jd2_ck_ck_group.采购成本均值).map(lambda x: format(x,".2%"))
df_jd2_ck_ck_group.loc[:, '残值估算毛利率'] = (df_jd2_ck_ck_group.残值估算毛利/df_jd2_ck_ck_group.采购成本均值).map(lambda x: format(x,".2%"))

with pd.ExcelWriter('F:/需求/瑞彤需求/租售比_11.13.xlsx', engine='openpyxl', mode='a') as writer:
    df_jd2_ck_ck_group.to_excel(writer, sheet_name='京东渠道租售比')
df_jd2_ck_ck_group
# 2025-06	9817.505976	6205.173015	8008.828685	2658.548247	4417.115405	63.21%	55.15%	33.20%

,总押金均值,总租金均值,采购成本均值,残值估算毛利,买断估算毛利,租售比,买断估算毛利率,残值估算毛利率
年份,,,,,,,,
2025,9843.473546,6858.017202,8391.814248,3370.485756,4222.375945,69.67%,50.32%,40.16%


In [ ]:
# df_jd = df[df.归属渠道=='京东渠道']
# df_jd.loc[:, '碎屏险价格'] = df_jd.碎屏险价格.fillna(0)
# df_jd[(df_jd.碎屏险价格>0)&(df_jd.是否出库==1)][['order_number', '碎屏险价格']]

## 瑞彤姐需求（日报格式需修改+总租金+买断尾款+买断价）

In [ ]:
# 日报转化数据加工
def data_group_trans(self, df, df2, df_risk_examine, model):
    '''
    计算df按model分组的数据
    :param df: 包含小蚂蚁租机的df
    :param df2: 不含小蚂蚁租机的df
    :param df_risk_examine: 包含出库前风控强拒的df
    :param model: 需要分组的类型
    :return: 返回分组后的数据
    '''
    # 包含拒量的数据
    df_group_1 = df.groupby(model).agg({'order_id': 'size', '是否进件': 'sum', '前置拦截': 'sum'})  # , '机审通过件': 'sum', '风控通过件': 'sum'
    df_group_1.rename(columns={'order_id': '去重订单数', '是否进件': '进件数'}, inplace=True)
    # 策略241205,策略241212,自有模型回捞策略
    df_241205 = df[df.tips.str.contains(r'策略241205|策略241212|命中自有模型回捞策略|回捞策略250330命中', regex=True)==True]
    # 拒绝
    df_241205 = df_241205[~df_241205.merchant_name.isin(['小蚂蚁租机', '兴鑫兴通讯', '人人享租', '崇胜数码', '喜卓灵租机', '喜卓灵新租机'])]
    df_241205.loc[:, '机审强拒_拒量'] = np.where(df_241205.qvt_risk=='1', 1, 0)
    df_241205.loc[:, '出库前强拒_拒量'] = np.where(((df_241205.qvt_risk == '0')|(df_241205.qvt_risk.isna()))&(df_241205.qvt_result == '1'), 1, 0)
    df_241205_group = df_241205.groupby(model).agg({'机审强拒_拒量': 'sum', '出库前强拒_拒量': 'sum', '是否出库': 'sum'})
    df_241205_group.rename(columns={'是否出库': '拒量出库'}, inplace=True)

    # 不包含拒量数据
    df_group_2 = df2.groupby(model).agg({'机审强拒': 'sum', '机审通过件': 'sum', '进件前取消': 'sum', '风控通过件': 'sum'
            , '人审拒绝': 'sum', '客户取消': 'sum', '已退款': 'sum', '无法联系': 'sum', '出库前风控强拒': 'sum', '待审核': 'sum', '是否出库': 'sum'}) # '已退款': 'sum',
    df_group_2.rename(columns={'是否出库': '出库'}, inplace=True)

    df_group_3 = pd.merge(df_group_1, df_group_2, on=model, how='inner')
    # 拼接下拒量的被拒数据
    df_xmy_all, df_xmy = self.xmy(df, df_risk_examine, model)
    df_group = df_group_3.merge(df_xmy, on=model, how='left').merge(df_241205_group, on=model, how='left')#.merge(df_241212_group, on=model, how='left')
    df_group = df_group.fillna(0)
    df_group['机审强拒'] = df_group['机审强拒'] + df_group['小蚂蚁机审强拒'] + df_group['机审强拒_拒量'] #+ df_group['机审通过件_241212']
    df_group['机审通过件'] = df_group['机审通过件'] + df_group['小蚂蚁人审拒绝'] + df_group['order_risk_ex'] + df_group['出库前强拒_拒量']  # +除机审强拒的单
    df_group['风控通过件'] = df_group['风控通过件'] + df_group['order_risk_ex'] + df_group['出库前强拒_拒量']  # +出库前强拒的单
    df_group['人审拒绝'] = df_group['人审拒绝'] + df_group['小蚂蚁人审拒绝']
    df_group['出库前风控强拒'] = df_group['出库前风控强拒'] + df_group['order_risk_ex'] + df_group['出库前强拒_拒量']
    df_group['客户取消'] = df_group['客户取消']+df_group['已退款']

    df_group["拦截率"] = df_group["前置拦截"] / df_group["去重订单数"]
    df_group["拦截率"] = df_group["拦截率"].apply(lambda x: format(x, ".2%"))

    df_group["进件前取消率"] = df_group["进件前取消"] / df_group["去重订单数"]
    df_group["进件前取消率"] = df_group["进件前取消率"].apply(lambda x: format(x, ".2%"))

    df_group['预授权通过率'] = df_group["进件数"] / df_group["去重订单数"]
    df_group["预授权通过率"] = df_group["预授权通过率"].apply(lambda x: format(x, ".2%"))

    df_group["强拒比例"] = df_group["机审强拒"] / df_group["进件数"]
    df_group["强拒比例"] = df_group["强拒比例"].apply(lambda x: format(x, ".2%"))

    df_group["风控通过率"] = df_group["风控通过件"] / df_group["进件数"]
    df_group["风控通过率"] = df_group["风控通过率"].apply(lambda x: format(x, ".2%"))

    df_group["取消率"] = df_group["客户取消"] / df_group["进件数"]
    df_group["取消率"] = df_group["取消率"].apply(lambda x: format(x, ".2%"))

    df_group["无法联系占比"] = df_group["无法联系"] / df_group["进件数"]
    df_group["无法联系占比"] = df_group["无法联系占比"].apply(lambda x: format(x, ".2%"))

    df_group["人审拒绝率"] = df_group["人审拒绝"] / df_group["进件数"]
    df_group["人审拒绝率"] = df_group["人审拒绝率"].apply(lambda x: format(x, ".2%"))

    df_group["出库前强拒比例"] = df_group["出库前风控强拒"] / df_group["进件数"]
    df_group["出库前强拒比例"] = df_group["出库前强拒比例"].apply(lambda x: format(x, ".2%"))

    df_group["进件出库率"] = df_group["出库"] / df_group["进件数"]
    df_group['总体进件出库率（含拒量）'] = (df_group['出库'] + df_group['拒量出库'])/df_group["进件数"]
    df_group['拒量进件出库率增加'] = df_group['总体进件出库率（含拒量）'] - df_group['进件出库率']
    df_group["进件出库率"] = df_group["进件出库率"].apply(lambda x: format(x, ".2%"))
    df_group["总体进件出库率（含拒量）"] = df_group["总体进件出库率（含拒量）"].apply(lambda x: format(x, ".2%"))
    df_group["拒量进件出库率增加"] = df_group["拒量进件出库率增加"].apply(lambda x: format(x, ".2%"))

    df_group["订单出库率"] = df_group["出库"] / df_group["去重订单数"]
    df_group["订单出库率"] = df_group["订单出库率"].apply(lambda x: format(x, ".2%"))

    df_group["人审转化率"] = df_group["出库"] / df_group["机审通过件"]
    df_group["人审转化率"] = df_group["人审转化率"].apply(lambda x: format(x, ".2%"))
    
    # 

    return df_group

In [ ]:
# 下周一输出的数据需求：
# 渠道：京东总体、京东app、京东金融app；
# 字段：日报格式+总租金+买断尾款+买断价；
# 数据时间：0522-0528、0529-0604、0605-0611、06112-0618、0619-0625、0626-0702、0703-0716、0717-0723、0724-0730
df_no_drop_jd = df_no_drop[df_no_drop.归属渠道=='京东渠道']
df_no_drop_jd = df_no_drop_jd[df_no_drop_jd.下单日期>='2025-05-20']
# 将下单日期按0522-0528、0529-0604、0605-0611、06112-0618、0619-0625、0626-0702、0703-0716、0717-0723、0724-0730分组
df_no_drop_jd['下单日期'] = pd.to_datetime(df_no_drop_jd['下单日期'])

# 定义日期分组的边界
date_bins = [
    '2025-05-22', '2025-05-29', '2025-06-05', '2025-06-12', '2025-06-19', '2025-06-26', '2025-07-03', '2025-07-10', '2025-07-17', '2025-07-24', '2025-07-31'
]
# 将日期字符串转换为datetime对象
date_bins = pd.to_datetime(date_bins)

# 定义分组的标签
date_labels = ['0522-0528', '0529-0604', '0605-0611', '0612-0618', '0619-0625', '0626-0702', '0703-0709', '0710-0716','0717-0723', '0724-0730']

# 使用cut函数进行分组
df_no_drop_jd['下单日期分组'] = pd.cut(df_no_drop_jd['下单日期'], bins=date_bins, labels=date_labels, right=False)
df['下单日期分组'] = pd.cut(df['下单日期'], bins=date_bins, labels=date_labels, right=False)
df2['下单日期分组'] = pd.cut(df2['下单日期'], bins=date_bins, labels=date_labels, right=False)
df_no_drop_jd.loc[:, '创建进件数'] = np.where(df_no_drop_jd.进件=='进件', 1, 0)
df_no_drop_jd_g = df_no_drop_jd.groupby('下单日期分组').agg({'order_id': 'count', '创建进件数': 'sum'}).rename(columns={'order_id': '创建订单数'})


df_no_drop_jdsc_g = df_no_drop_jd[df_no_drop_jd.app_type=='JD_APP'].groupby('下单日期分组').agg({'order_id': 'count'}).rename(columns={'order_id': '创建订单数'})
df_no_drop_jdjr_g = df_no_drop_jd[df_no_drop_jd.app_type=='JR_APP'].groupby('下单日期分组').agg({'order_id': 'count'}).rename(columns={'order_id': '创建订单数'})


In [ ]:
df.下单日期分组.unique()

In [ ]:
df_jd = df[df['归属渠道']=='京东渠道']
df_jd2 = df2[df2['归属渠道']=='京东渠道']
df_jd = df_jd[df_jd.下单日期>='2025-05-20']
df_jd2 = df_jd2[df_jd2.下单日期>='2025-05-20']
df_jd_group = all_models.data_group(df_jd, df_jd2, df_risk_examine, '下单日期分组')
# 获取按下单日期分组汇总的数据：总租金tprm.all_rental、买断尾款(tod.new_actual_money-tprm.all_rental) 、买断价tod.new_actual_money
df_jd.loc[:, '买断尾款'] = df_jd['new_actual_money']- df_jd['all_rental']
df_jd_md = df_jd.groupby('下单日期分组').agg({'all_rental': 'sum', 'new_actual_money': 'sum', '买断尾款': 'sum'}).rename(columns={'all_rental': '总租金', 'new_actual_money': '买断价'})

df_jd_group['出库（按下单时间）'] = df_jd_group['出库']
try:
    df_jd_group['出库（按转化时间）'] = df_weekday_zh['京东渠道'][:-1]
except:
    df_jd_group['出库（按转化时间）'] = 0
df_jd_group = df_jd_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_jd_group_new = df_jd_group.reset_index()
df_jd_group_new = df_no_drop_jd_g.merge(df_jd_group_new, on='下单日期分组', how='inner').merge(df_jd_md, on='下单日期分组', how='inner')
df_jd_group_new

### 京东商城

In [ ]:
df_jdsc = df_jd[df_jd.app_type=='JD_APP']
df_jdsc2 = df_jd2[df_jd2.app_type=='JD_APP']
df_jdsc_group = all_models.data_group(df_jdsc, df_jdsc2, df_risk_examine, '下单日期分组')

df_jdsc.loc[:, '买断尾款'] = df_jdsc['new_actual_money']- df_jdsc['all_rental']
df_jdsc_md = df_jdsc.groupby('下单日期分组').agg({'all_rental': 'sum', 'new_actual_money': 'sum', '买断尾款': 'sum'}).rename(columns={'all_rental': '总租金', 'new_actual_money': '买断价'})



df_jdsc_group['出库（按下单时间）'] = df_jdsc_group['出库']

df_jdsc_group = df_jdsc_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_jdsc_group_new = df_jdsc_group.reset_index()
df_jdsc_group_new = df_no_drop_jdsc_g.merge(df_jdsc_group_new, on='下单日期分组', how='left').merge(df_jdsc_md, on='下单日期分组', how='inner')
df_jdsc_group_new

### 京东金融

In [ ]:
df_jdjr = df_jd[df_jd.app_type=='JR_APP']
df_jdjr2 = df_jd2[df_jd2.app_type=='JR_APP']
df_jdjr_group = all_models.data_group(df_jdjr, df_jdjr2, df_risk_examine, '下单日期分组')

df_jdjr.loc[:, '买断尾款'] = df_jdjr['new_actual_money']- df_jdjr['all_rental']
df_jdjr_md = df_jdjr.groupby('下单日期分组').agg({'all_rental': 'sum', 'new_actual_money': 'sum', '买断尾款': 'sum'}).rename(columns={'all_rental': '总租金', 'new_actual_money': '买断价'})



df_jdjr_group['出库（按下单时间）'] = df_jdjr_group['出库']

df_jdjr_group = df_jdjr_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_jdjr_group_new = df_jdjr_group.reset_index()
df_jdjr_group_new = df_no_drop_jdjr_g.merge(df_jdjr_group_new, on='下单日期分组', how='left').merge(df_jdjr_md, on='下单日期分组', how='left')
df_jdjr_group_new

In [ ]:
# 写入excel
with pd.ExcelWriter('F:/需求/瑞彤需求/京东渠道周数据.xlsx', engine='xlsxwriter') as writer:
    df_jd_group_new.to_excel(writer, sheet_name='京东总体数据', index=False)
    df_jdsc_group_new.to_excel(writer, sheet_name='京东商城数据', index=False)
    df_jdjr_group_new.to_excel(writer, sheet_name='京东金融数据', index=False)

## 机型转化

In [ ]:
# sql_jx = '''
# select
# tp.id, tpmn.name 机型
# from db_digua_business.t_product tp
# left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id
# '''
# df_jx = clean.query(sql_jx)

In [ ]:
# df_jx1 = df.merge(df_jx, left_on='product_id', right_on='id', how='left')
# df_jx1.loc[:, '机型内存'] = df_jx1.机型 + '_' + df_jx1.内存
# df2_jx1 = df2.merge(df_jx, left_on='product_id', right_on='id', how='left')
# df2_jx1.loc[:, '机型内存'] = df2_jx1.机型 + '_' + df2_jx1.内存
# df_jx_group = all_models.data_group(df_jx1, df2_jx1, df_risk_examine, '机型内存')
# df_jx_group = df_jx_group.sort_values(by='出库', ascending=False).groupby('机型内存').head(10)

In [ ]:
# jx_top10 = df_jx_group[:10].index.to_list()
# df2_jx1_group = df2_jx1[(df2_jx1.是否出库==1)&(df2_jx1.机型内存.isin(jx_top10))&(df2_jx1.purchase_amount>0)].groupby('机型内存').agg({'purchase_amount': 'mean', 'all_rental': 'mean', 'all_deposit':'mean', 'new_actual_money': 'mean'}).rename(columns={'purchase_amount': '采购价', 'all_rental': '总租金', 'all_deposit': '总押金', 'new_actual_money': '签约价'})#官网价  采购价、总租金、买断尾款、签约价
# df2_jx1_group.loc[:, '买断尾款'] = df2_jx1_group.签约价-df2_jx1_group.总租金

# with pd.ExcelWriter('F:/需求/瑞彤需求/top10机型转化.xlsx', engine='xlsxwriter') as writer:
#     df2_jx1_group.to_excel(writer, sheet_name='机型转化')

In [ ]:
# df_jd = df[df.归属渠道=='京东渠道']
# df_jd_new= df_jd.merge(df_jx, left_on='product_id', right_on='id', how='left')
# df_jd_new.loc[:, '机型内存'] = df_jd_new.机型 + '_' + df_jd_new.内存
# df_jd_new_g = df_jd_new.groupby('机型内存').agg({'是否进件': 'sum', '是否出库': 'sum'}).rename(columns={'是否进件': '进件数', '是否出库': '出库数'})
# df_jd_news = df_jd_new[(df_jd_new.是否出库==1)&(df_jd_new.purchase_amount>0)]
# df_jd_news['all_deposit'] = df_jd_news.all_deposit.astype(float)
# df_jd_news_g = df_jd_news.groupby('机型内存').agg({'purchase_amount': 'sum', 'all_deposit': 'sum', 'new_actual_money': 'sum', 'all_rental': 'sum'}).rename(columns={'purchase_amount': '采购金额', 'all_deposit': '总押金', 'new_actual_money': '签约价', 'all_rental': '总租金'})
# df_jd_new_g = df_jd_new_g.merge(df_jd_news_g, on='机型内存', how='left').fillna(0).sort_values(by='进件数', ascending=False)
# df_jd_new_g.loc['合计', :] = df_jd_new_g.sum(axis=0)
# df_jd_new_g.loc[:, '机型占比_出库'] = (df_jd_new_g.出库数/df_jd_new_g.loc['合计', '出库数']).map(lambda x: format(x, '.2%'))
# df_jd_new_g.loc[:, '租金/押金'] = (df_jd_new_g.总租金/df_jd_new_g.总押金).map(lambda x: format(x, '.2%'))
# df_jd_new_g.loc[:, '签约价/押金'] = (df_jd_new_g.签约价/df_jd_new_g.总押金).map(lambda x: format(x, '.2%'))
# df_jd_new_g.to_excel('F:/京东SKU.xlsx')

## 临时需求

### 京东二手商品转化数据及取消原因，含商家

In [ ]:
# df_jd_es = df[(df['归属渠道']=='京东渠道')&(df['商品类型']=='二手')]
# df_jd_es.shape
# df_jd_es2 = df2[(df2['归属渠道']=='京东渠道')&(df2['商品类型']=='二手')&(df2['商品ID'].isin([7979, 7975]))]
# # df_jd_es.loc[:, '日期范围'] = '0809-0934'
# # df_jd_es2.loc[:, '日期范围'] = '0809-0934'
# df_jdes_group = all_models.data_group(df_jd_es, df_jd_es2, df_risk_examine, '下单日期')

# df_jdes_group = df_jdes_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# # 添加汇总行，仅计算数值型
# df_jdes_group.loc['汇总', :] = df_jdes_group.select_dtypes(include=[np.number]).sum()
# df_jdes_group_new = df_jdes_group.reset_index()


# df_jdes_group_new.tail()#.to_excel('F:/需求/崔婷玉需求/京东二手商品总体转化数据.xlsx')


In [ ]:
# df_jdes_jj = df_jd_es[(df_jd_es['是否进件']==1)]
# df_jdes_jj['取消原因2'] = df_jdes_jj['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_jdes_jj['取消原因2'].drop_duplicates()
# df_jdes_qx = df_jdes_jj.groupby('取消原因2').agg({'order_number': 'count'}).rename(columns={'order_number': '取消数'})
# df_jdes_qx['取消率'] = df_jdes_qx['取消数']/(df_jdes_jj['是否进件'].sum())
# df_jdes_qx['取消率'] = df_jdes_qx['取消率'].apply(lambda x: format(x, '.2%'))
# df_jdes_qx#.to_excel('F:/需求/崔婷玉需求/京东二手商品总体机审强拒理由.xlsx')（强拒：拒绝理由）
# with pd.ExcelWriter('F:/需求/崔婷玉需求/京东二手特定商品ID转化数据及取消原因.xlsx', engine='xlsxwriter') as writer:
#     df_jdes_group_new.to_excel(writer, sheet_name='京东二手商品转化数据', index=False)
#     df_jdes_qx.to_excel(writer, sheet_name='京东二手商品取消原因')

### 京东出库订单号

In [ ]:
# df_jd_qs = df[(df['归属渠道']=='京东渠道')&(df['是否出库']==1)]
# df_jd_qs.shape
# df_jd_qs.loc[:, '订单状态'] = np.where(df_jd_qs['退货拒签']==1, '已退货',np.where((df_jd_qs['status']==4)&(df_jd_qs.overdue==1), '逾期', df_jd_qs['status2']))
# # df_jd_qs.loc[:, '是否逾期'] = np.where((df_jd_qs['status']==4)&(df_jd_qs.overdue==1), '逾期', '未逾期')#(df['退货拒签']==0)&
# # df_jd_qs.loc[:, '是否买断'] = np.where(df_jd_qs.has_actual==1, '买断', '未买断')
# df_jd_qs[['order_number', '订单状态']].to_excel('F:/需求/崔婷玉需求/京东渠道8月出库订单号.xlsx', index=False)

### 京东618活动机型出库数据

In [ ]:
# df_no_drop_jd = df_no_drop[df_no_drop.来源渠道 == '京东618活动']
# df_no_drop_jd.loc[:, '创建进件数'] = np.where(df_no_drop_jd.进件 == '进件', 1, 0)
# df_no_drop_jd_g = df_no_drop_jd.groupby('机型内存').agg({'order_id': 'count', '创建进件数': 'sum'}).rename(columns={'order_id': '创建订单数'})
# df_jd_yk = df[df['来源渠道'] == '京东618活动']
# df_jd2_yk = df2[df2['来源渠道'] == '京东618活动']
# df_jd_group = all_models.data_group(df_jd_yk, df_jd2_yk, df_risk_examine, '机型内存')
# df_jd_group = df_jd_group[
#     ["去重订单数", "前置拦截", "拦截率", "进件数", "预授权通过率", "机审强拒", "强拒比例", "机审通过件",
#         "人审拒绝", "风控通过件", "风控通过率", "客户取消", "无法联系", "出库前风控强拒", "待审核",
#         '出库', '进件出库率', '取消率', '人审拒绝率', '出库前强拒比例',
#         '无法联系占比', '订单出库率']].fillna(0)
# df_jd_group_new = df_jd_group.reset_index()
# df_jd_group_new2 = df_no_drop_jd_g.merge(df_jd_group_new, on='机型内存', how='inner')

# df_jd_group_new2.to_excel('F:\需求\瑞彤需求\京东618活动机型出库数据.xlsx', index=False)


### 按下单周拉取转化数据

In [ ]:
# # 按下单日期所在本周的周四到下周三为一周，eg:，8月7日到8月13日为一周，8月7日到8月13日的下单周范围都为2025.8.7-2025.8.13，由此逻辑设置列：下单周范围
# def get_week_range_v2(date):
#     # 获取该日期所在周的周四
#     # weekday(): 周一=0, 周二=1, ..., 周四=3, ..., 周日=6
#     days_since_thursday = (date.weekday() - 3) % 7
#     thursday = date - pd.Timedelta(days=days_since_thursday)
#     # 获取下周的周三（周四+6天）
#     next_wednesday = thursday + pd.Timedelta(days=6)
#     # 格式化为"年.月.日-年.月.日"格式
#     return f"{thursday.year}.{thursday.month}.{thursday.day}-{next_wednesday.year}.{next_wednesday.month}.{next_wednesday.day}"

# df_no_drop_jd = df_no_drop[df_no_drop.归属渠道=='京东渠道']
# df_jd = df[df['归属渠道']=='京东渠道']
# df_jd2 = df2[df2['归属渠道']=='京东渠道']

# df_no_drop_jd.loc[:, '创建进件数'] = np.where(df_no_drop_jd.进件=='进件', 1, 0)

# df_no_drop_jd.loc[:, '下单周范围'] = df_no_drop_jd['下单日期'].apply(get_week_range_v2)
# df_jd.loc[:, '下单周范围'] = df_jd['下单日期'].apply(get_week_range_v2)
# df_jd2.loc[:, '下单周范围'] = df_jd2['下单日期'].apply(get_week_range_v2)

# # 京东商城
# df_no_drop_jdsc_g = df_no_drop_jd[df_no_drop_jd.app_type=='JD_APP'].groupby('下单日期').agg({'order_id': 'count','创建进件数': 'sum'}).rename(columns={'order_id': '创建订单数'})
# df_jdsc = df_jd[df_jd.app_type=='JD_APP']
# df_jdsc2 = df_jd2[df_jd2.app_type=='JD_APP']
# df_jdsc_group = all_models.data_group(df_jdsc, df_jdsc2, df_risk_examine, '下单日期')
# df_jdsc_group['出库（按下单时间）'] = df_jdsc_group['出库']

# df_jdsc_group = df_jdsc_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_jdsc_group_new = df_jdsc_group.reset_index()
# df_jdsc_group_new = df_no_drop_jdsc_g.merge(df_jdsc_group_new, on='下单日期', how='left')

# # 京东金融
# df_no_drop_jdjr_g = df_no_drop_jd[df_no_drop_jd.app_type=='JR_APP'].groupby('下单日期').agg({'order_id': 'count','创建进件数': 'sum'}).rename(columns={'order_id': '创建订单数'})
# df_jdjr = df_jd[df_jd.app_type=='JR_APP']
# df_jdjr2 = df_jd2[df_jd2.app_type=='JR_APP']
# df_jdjr_group = all_models.data_group(df_jdjr, df_jdjr2, df_risk_examine, '下单日期')
# df_jdjr_group['出库（按下单时间）'] = df_jdjr_group['出库']

# df_jdjr_group = df_jdjr_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_jdjr_group_new = df_jdjr_group.reset_index()
# df_jdjr_group_new = df_no_drop_jdjr_g.merge(df_jdjr_group_new, on='下单日期', how='left')
# df_jdsc_group_new

In [ ]:
# path = 'F:\需求\崔婷玉需求/'
# with pd.ExcelWriter(path+f'京东渠道各子渠道出库数据_{t_date}.xlsx', engine='openpyxl') as writer:
#     df_jdsc_group_new.to_excel(writer, sheet_name='京东商城', index=False)
#     df_jdjr_group_new.to_excel(writer, sheet_name='京东金融', index=False)

## 客户取消数据

### 取消原因1：创建订单后处于订单取消状态

In [ ]:
# df_no_drop_jd = df_no_drop[(df_no_drop.归属渠道=='京东渠道')&(df_no_drop['product_name'].str.contains(r'iPhone 17|iPhone Air'))]
# df_no_drop_jd.loc[:, '取消1'] = np.where((df_no_drop_jd['status2'] == '订单取消')&(df_no_drop_jd['order_id'].notna()), 1, 0)
# df_no_drop_jd_group = df_no_drop_jd.groupby('下单日期').agg({'取消1':'sum', 'order_id':'count'}).rename(columns={'取消1':'取消数', 'order_id':'创建订单数'})
# df_no_drop_jd_group['取消率'] = (df_no_drop_jd_group['取消数']/df_no_drop_jd_group['创建订单数']).apply(lambda x: format(x, '.2%'))
# df_no_drop_jd_group

### 取消原因2：进件后，存在机审强拒、人审拒绝、客户取消、无法联系、出库前风控强拒中至少有一个为1

In [ ]:
# df_jd = df[(df['归属渠道']=='京东渠道')&(df['product_name'].str.contains(r'iPhone 17|iPhone Air'))]
# df_jd.loc[:, '取消2'] = np.where(((df_jd['机审强拒'] == 1)|(df_jd['人审拒绝']==1)|(df_jd['客户取消']==1)|(df_jd['无法联系']==1)|(df_jd['出库前风控强拒']==1)), 1, 0)
# df_jd_group = df_jd.groupby('下单日期').agg({'取消2':'sum', '是否进件':'sum'}).rename(columns={'取消2':'取消数', '是否进件':'进件数'})
# df_jd_group['取消率'] = (df_jd_group['取消数']/df_jd_group['进件数']).apply(lambda x: format(x, '.2%'))
# df_jd_group

### 取消原因4：审核通过后或有支付流水且状态为已退款

In [ ]:
# df_jd.loc[:, '审核通过'] = np.where(df_jd['total_describes'].str.contains('msg=审核通过'), 1, 0)
# df_jd.loc[:, '取消4'] = np.where((df_jd['审核通过']==1)&(df_jd['status2']=='已退款'), 1, 0)
# df_jd_group_sh = df_jd.groupby('下单日期').agg({'取消4':'sum', '审核通过':'sum'}).rename(columns={'取消4':'取消数', '审核通过':'审核通过数'})
# df_jd_group_sh.loc[:, '取消率'] = (df_jd_group_sh['取消数']/df_jd_group_sh['审核通过数']).apply(lambda x: format(x, '.2%'))
# df_jd_group_sh

In [ ]:
# # 写入Excel
# with pd.ExcelWriter('F:/需求/瑞彤需求/京东取消原因订单_月份.xlsx', engine='openpyxl') as writer:
#     df_no_drop_jd_group.to_excel(writer, sheet_name='取消原因1')
#     df_jd_group.to_excel(writer, sheet_name='取消原因2')
#     df_jd_group_sh.to_excel(writer, sheet_name='取消原因4')

### 发货前订单取消原因

In [ ]:
# df_jd_shtg = df_jd#[(df_jd['风控通过件']==1)&(df_jd['status2']=='已退款')]
# df_jd_shtg.loc[:,'发货前取消'] = np.where((df_jd['status2']=='已退款')&(df_jd['风控通过件']==1)&(df_jd['出库前风控强拒']==0),1,0)
# # 物流状态=5，退货拒签
# # df.loc[:,"退货拒签"] = np.where(df['物流状态']==5, 1, 0)
# df_jd_shtg.loc[:,'发货拒签'] = np.where((df_jd['物流状态']==5)&(df_jd['是否出库']==1),1,0)
# df_jd_shtg_group = df_jd_shtg.groupby('下单月份').agg({'发货前取消':'sum', '发货拒签':'sum', '是否进件':'sum'})#.rename(columns={'发货前取消':'发货前取消数', '发货拒签':'发货拒签数'})
# df_jd_shtg_group['发货前取消率'] = (df_jd_shtg_group['发货前取消']/df_jd_shtg_group['是否进件']).apply(lambda x: format(x, ".2%"))
# df_jd_shtg_group['发货拒签率'] = (df_jd_shtg_group['发货拒签']/df_jd_shtg_group['是否进件']).apply(lambda x: format(x, ".2%"))
# df_jd_shtg_group

In [ ]:
# # [df_jd['product_name'].str.contains(r'iPhone 17|iPhone Air')]
# df_jd = df[df['归属渠道']=='京东渠道']
# df_jd = df_jd[df_jd['product_name'].str.contains(r'iPhone 17|iPhone Air')]
# # df_jd_shtg = df_jd[(df_jd['风控通过件']==1)&(df_jd['status2']=='已退款')&(df_jd['出库前风控强拒']==0)]
# # df_jd_shtg = df_jd[(df_jd['是否进件'] == 1) & (df_jd['机审强拒'] == 0) & (df_jd['人审拒绝'] == 0)&(df_jd['status2']=='已退款')&(df_jd['出库前风控强拒']==0)]
# df_jd_shtg = df_jd[(df_jd['是否进件'] == 1)]
# df_jd_shtg['取消原因2'] = df_jd_shtg['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_jd_shtg['取消原因2'].drop_duplicates()
# df_jd_shtg_group = df_jd_shtg.groupby(['下单日期', 'rejected']).agg({'order_number': 'count'}).rename(columns={'order_number': '取消数'})
# df_jd_shtg_group['取消率'] = df_jd_shtg_group['取消数']/(df_jd_shtg['是否进件'].sum())
# df_jd_shtg_group['取消率'] = df_jd_shtg_group['取消率'].apply(lambda x: format(x, '.2%'))
# df_jd_shtg_group.shape#.to_excel('F:/需求/崔婷玉需求/iPhone17取消原因.xlsx', sheet_name='取消原因')

In [ ]:
# # 写入Excel
# with pd.ExcelWriter('F:/需求/瑞彤需求/京东取消订单_月份.xlsx', engine='openpyxl') as writer:
#     df_tkjd_kh_group.to_excel(writer, sheet_name='客户取消')
#     df_jdqj_group.to_excel(writer, sheet_name='机审人审拒绝')
#     df_jd_shtg_group.to_excel(writer, sheet_name='审核通过取消订单')

# 支付宝直播转化

In [203]:
# # 支付宝直播
df_zfb = df[df['activity_name']=='支付宝直播商品']
df_zfb2 = df2[df2['activity_name']=='支付宝直播商品']
# df_zfb_group = all_models.data_group(df_zfb, df_zfb2, df_risk_examine, '下单日期')


# # df_zfb_group['出库（按下单时间）'] = df_zfb_ck['支付宝直播商品'][:-1]
# df_zfb_group['出库（按下单时间）'] = df_zfb_group['出库']
# df_zfb_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","人审拒绝率","风控通过件","客户取消","取消率",'无法联系',"出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','订单出库率']]#.to_excel('F:/需求/总需求/支付宝直播转化.xlsx')


## 租售比

In [205]:
df_zfb2_ck = df_zfb[df_zfb.是否出库==1]
df_zfb2_ck_group = df_zfb2_ck.groupby('月份').agg({'是否出库': 'sum', 'all_rental': 'mean', 'new_actual_money': 'mean', 'all_deposit': 'mean'}).rename(columns={'是否出库': '出库', 'all_deposit': '总押金均值', 'all_rental': '总租金均值', 'new_actual_money': '签约价均值'})
df_zfb2_ck_group.loc[:, '租售比'] = (df_zfb2_ck_group.总租金均值/df_zfb2_ck_group.总押金均值).map(lambda x: format(x, '.2%'))
df_zfb2_ck_group#.to_excel('F:/需求/崔婷玉需求/支付宝直播租售比.xlsx')

,出库,总租金均值,签约价均值,总押金均值,租售比
月份,,,,,
10,4,6498.955,13646.6,10499.0,61.90%


In [ ]:
# df1_ck = df_zfb[df_zfb['是否出库']==1]
# Today_zfb = datetime.now().strftime('%Y%m%d%H')
# df1_ck[['下单日期','order_number', 'product_name', '内存', '租赁方案']].to_excel(f'F:/需求/支付宝直播数据/支付宝直播数据_{Today_zfb}.xlsx')

In [ ]:
# df1_j = df_zfb[df_zfb['是否出库']!=1]
# df1_j = df1_j[['下单日期', 'order_number', 'product_name', 'cancel_reason']]
# df1_j.rename(columns={'order_number':'订单号', 'product_name': '产品名称', 'cancel_reason':'取消原因'}, inplace=True)
# df1_j.to_excel(f'F:/需求/支付宝直播数据/支付宝直播取消数据_{Today_zfb}.xlsx')

# 快捷下单转化

In [ ]:
# df_kj = df[(df['order_method']==1)&(df['classify_id'].isin([116, 165]))]
# df_kj_group = df_kj.groupby('下单日期').agg({'order_id': 'size', '是否进件': 'sum', '是否出库': 'sum'})
# df_kj_group.rename(columns={'order_id': '去重订单数'}, inplace=True)
# df_kj_group['进件转化率'] = df_kj_group['是否进件'] / df_kj_group['去重订单数'] * 100
# df_kj_group['进件转化率'] = df_kj_group['进件转化率'].apply(lambda x: f'{x: .2f}%')
# 
# df_kj_group['进件出库率'] = df_kj_group['是否出库'] / df_kj_group['是否进件'] * 100
# df_kj_group['进件出库率'] = df_kj_group['进件出库率'].apply(lambda x: f'{x: .2f}%')
# df_kj_group.to_excel('F:/需求/瑞彤需求/快捷下单出库率.xlsx')


# 无快捷下单的租物转化

In [ ]:
# df_wkj = df[df['order_method']!=1]
# df_wkj_zm = df_wkj[df_wkj['归属渠道']=='芝麻租物']
# df_wkj_zm_group = df_wkj_zm.groupby('下单日期').agg({'order_id': 'size', '是否进件': 'sum'})
# df_wkj_zm_group.rename(columns={'order_id': '去重订单数'}, inplace=True)
# df_wkj_zm_group['进件转化率'] = df_wkj_zm_group['是否进件'] / df_wkj_zm_group['去重订单数'] * 100
# df_wkj_zm_group['进件转化率'] = df_wkj_zm_group['进件转化率'].apply(lambda x: f'{x: .2f}%')


In [ ]:
# with pd.ExcelWriter(f'F:/需求/快捷下单/快捷下单转化_{Today_zfb}.xlsx', engine='openpyxl') as writer:
#     df_kj_group.to_excel(writer, sheet_name='快捷下单转化')
# with pd.ExcelWriter(f'F:/需求/快捷下单/快捷下单转化_{Today_zfb}.xlsx', engine='openpyxl', mode='a') as writer:
#     df_wkj_zm_group.to_excel(writer, sheet_name='无快捷下单租物转化')

# 机型内存转化

In [ ]:
# 机型内存转化
def product_model( df, df2, keys):
    if keys == 'All':
        df = df.copy()
        df2 = df2.copy()
    elif keys == '芝麻租物':
        df = df[df["归属渠道"] == "芝麻租物"]
        df2 = df2[df2["归属渠道"] == "芝麻租物"]
    elif keys == '全新':
        df = df[(df["归属渠道"] == "芝麻租物") & (df["商品类型"] == "全新")]
        df2 = df2[(df2["归属渠道"] == "芝麻租物") & (df2["商品类型"] == "全新")]
    # df["型号内存"] = df["product_name"] + "_" + df["内存"]
    # df2["型号内存"] = df2["product_name"] + "_" + df2["内存"]
    df["机型内存"] = df["机型"] + "_" + df["内存"]
    df2["机型内存"] = df2["机型"] + "_" + df2["内存"]

    df_group1 = df.groupby(['机型内存']).agg({'order_id': 'size', '是否进件': 'sum'})
    df_group2 = df2.groupby(['机型内存']).agg(
        {'前置拦截': 'sum', '机审强拒': 'sum', '人审拒绝': 'sum', '客户取消': 'sum'
            , '无法联系': 'sum', '出库前风控强拒': 'sum', '待审核': 'sum', '是否出库': 'sum'})
    df_group1.rename(columns={'order_id': '去重订单数'}, inplace=True)

    df_group = pd.merge(df_group1, df_group2, on='机型内存', how='inner')

    df_group["拦截率"] = df_group["前置拦截"] / df_group["去重订单数"]
    df_group["拦截率"] = df_group["拦截率"].apply(lambda x: format(x, ".2%"))

    df_group['预授权通过率'] = df_group["是否进件"] / df_group["去重订单数"]
    df_group["预授权通过率"] = df_group["预授权通过率"].apply(lambda x: format(x, ".2%"))

    df_group["强拒比例"] = df_group["机审强拒"] / df_group["是否进件"]
    df_group["强拒比例"] = df_group["强拒比例"].apply(lambda x: format(x, ".2%"))

    df_group["机审通过件"] = df_group["是否进件"] - df_group["机审强拒"]

    df_group["风控通过件"] = df_group["是否进件"] - df_group["机审强拒"] - df_group["人审拒绝"]
    df_group["风控通过率"] = df_group["风控通过件"] / df_group["是否进件"]
    df_group["风控通过率"] = df_group["风控通过率"].apply(lambda x: format(x, ".2%"))

    df_group["取消率"] = df_group["客户取消"] / df_group["是否进件"]
    df_group["取消率"] = df_group["取消率"].apply(lambda x: format(x, ".2%"))

    # df_group["无法联系占比"]=df_group["无法联系"]/df_group["是否进件"]
    # df_group["无法联系占比"]=df_group["无法联系占比"].apply(lambda x:format(x,".2%"))

    df_group["人审拒绝率"] = df_group["人审拒绝"] / df_group["是否进件"]
    df_group["人审拒绝率"] = df_group["人审拒绝率"].apply(lambda x: format(x, ".2%"))

    df_group["进件出库率"] = df_group["是否出库"] / df_group["是否进件"]
    df_group["进件出库率"] = df_group["进件出库率"].apply(lambda x: format(x, ".2%"))

    df_group["订单出库率"] = df_group["是否出库"] / df_group["去重订单数"]
    df_group["订单出库率"] = df_group["订单出库率"].apply(lambda x: format(x, ".2%"))

    df_group = df_group[
        ["去重订单数", "前置拦截", "拦截率", "是否进件", "预授权通过率", "机审强拒", "强拒比例", "机审通过件", "人审拒绝", "风控通过件", "风控通过率"
            , "客户取消", "无法联系", "出库前风控强拒", "待审核", "是否出库", "进件出库率", "订单出库率", "取消率", "人审拒绝率"]]

    return df_group

In [ ]:
# 所有机型内存的全流程转化统计
# df_all_jxnc = all_models.product_model(df, df2, 'All')
df_all_jxnc = product_model(df, df2, 'All')
df_all_jxnc.loc['all', :] = df_all_jxnc.sum(numeric_only=True)

df_all_jxnc.sort_values(by = '去重订单数', ascending=False)#.to_excel(f"F:/myfile/业务相关数据/所有机型内存数据转化_{t_date}.xlsx") 

In [ ]:
# 租物所有机型内存的全流程转化统计
# df_zw = all_models.product_model(df, df2, '芝麻租物')
# df_zw.to_excel(f"F:/myfile/业务相关数据/宏生租物所有机型内存数据转化-{t_date}.xlsx") 

In [ ]:

# #  全新   租物所有机型内存的全流程转化统计
# df_zw_qx = all_models.product_model(df, df2, '全新')
# df_zw_qx.to_excel(f"F:/myfile/业务相关数据/全新-租物所有机型内存数据转化-{t_date}.xlsx") 

# 免审转化

In [ ]:
# 免审：is_vip = 1&status_result = 0
# df_ms = df[(df.is_vip=='1')&(df.status_result=='0')]
# df_ms2 = df2[(df2.is_vip=='1')&(df2.status_result=='0')]
# df_ms_group = all_models.data_group(df_ms, df_ms2, df_risk_examine,  '下单日期')
# 
# df_ms_group[["进件数","人审拒绝", "客户取消","出库","待审核","进件出库率","取消率","人审拒绝率"]] 


## 信审需求（倩姐）

In [ ]:

# # df_ms_list = df_ms[(df_ms['下单日期']=='2025-08-18')&(df_ms['机审通过件']==1)]['order_id'].tolist()
# # df2[df2['机审通过件']==1].groupby('下单日期').agg({'order_id':'count'})
# # 免审订单号列表
# df_ms_list = df[(df['下单日期']=='2025-08-18')&(df['机审通过件']==1)&(df.is_vip=='1')&(df.status_result=='0')]['order_id'].tolist()
# len(df_ms_list)
# # df_ms_list
# # 机审通过件中缺少拒量的通过件，信审订单数
# # df2[(df2['下单日期']=='2025-08-18')&(df2['机审通过件']==1)&(~df2['order_id'].isin(df_ms_list))&((df2['is_vip'] == '0'))]['order_id'].count()
# # 机审通过件，不含拒量的通过件
# df2[(df2['下单日期']=='2025-08-18')&(df2['机审通过件']==1)]['order_id'].count()

### 获取机审通过订单中拒量的订单

In [ ]:
# def xmy(df, df_risk_examine):
#         '''
#         计算小蚂蚁租机的总体转移，机审强拒，人审拒绝，出库前风控强拒数据
#         :param df: 总体数据的df
#         :param df_risk_examine: 出库前风控强拒数据的df
#         :param model: 传入的分组参数
#         :return: 返回小蚂蚁租机的数据
#         '''
#         # 匹配小蚂蚁（拒绝）数据
#         df_xmy = df[df['merchant_name'].isin(['小蚂蚁租机', '兴鑫兴通讯', '人人享租', '崇胜数码', '喜卓灵租机', '喜卓灵新租机'])]
#         # 出库前强拒数据重命名
#         df_risk_examine.rename(columns={'time': 'time_risk_ex', 'status': 'status_risk_ex'}, inplace=True)
#         # 对小蚂蚁数据和出库前强拒数据进行拼接
#         df_risk_examine_all = pd.merge(df_xmy, df_risk_examine, left_on='id_card_num', right_on='id_card', how='inner')
#         # 计算出库前强拒的订单数
#         df_risk_examine_all2 = df_risk_examine_all[(df_risk_examine_all['time_risk_ex'] < df_risk_examine_all['update_time'])]
#         # 进行排序并取到最近的一个订单
#         df_risk_examine_all2 = df_risk_examine_all2.sort_values(['order_id', 'time_risk_ex'], ascending=[True, False]).groupby('order_id').head(1)
#         df_risk_examine_all2 = df_risk_examine_all2[df_risk_examine_all2['status_risk_ex'] == '1']
#         # df_risk_examine_all2_g = df_risk_examine_all2.groupby(model).agg(order_risk_ex=('order_id', 'size'))
#         # df_risk_examine_all2_g
#         # 排除出库前风控强拒的订单
#         df_xmy_new = df_xmy[~df_xmy['order_id'].isin(df_risk_examine_all2['order_id'].to_list())]
#         # 定义人审拒绝订单
#         df_xmy_new['小蚂蚁人审拒绝'] = np.where(df_xmy_new['reason'] != '系统风控拒绝转移', 1, 0)
#         print(" 小蚂蚁人审拒绝：",df_xmy_new[(df_xmy_new['下单日期']=='2025-08-18')&(df_xmy_new['小蚂蚁人审拒绝']==1)]['order_id'].to_list())
#         print(df_xmy_new[(df_xmy_new['下单日期']=='2025-08-18')&(df_xmy_new['小蚂蚁人审拒绝']==1)]['order_id'].count())
#         print("order_risk_ex:", df_risk_examine_all2[(df_risk_examine_all2['下单日期']=='2025-08-18')]['order_id'].to_list())
#         print(df_risk_examine_all2[(df_risk_examine_all2['下单日期']=='2025-08-18')]['order_id'].count())
        
# xmy(df, df_risk_examine)
# df_241205 = df[df.tips.str.contains(r'策略241205|策略241212|命中自有模型回捞策略|回捞策略250330命中', regex=True)==True]
# # 拒绝
# df_241205 = df_241205[~df_241205.merchant_name.isin(['小蚂蚁租机', '兴鑫兴通讯', '人人享租', '崇胜数码', '喜卓灵租机', '喜卓灵新租机'])]
# df_241205.loc[:, '出库前强拒_拒量'] = np.where(((df_241205.qvt_risk == '0')|(df_241205.qvt_risk.isna()))&(df_241205.qvt_result == '1'), 1, 0)
# df_241205[(df_241205['出库前强拒_拒量']==1)&(df_241205['下单日期']=='2025-08-18')]['order_id'].count()


### 信审订单，按下单日期统计

In [ ]:
# # 信审订单数
# df_credit = df2[(df2['机审通过件']==1)&((df2['is_vip'] == '0')&(df.status_result=='0'))]#  | (df['is_vip'].isna())
# # 筛选未命中发货需上锁的订单
# # df_credit_examine = df_credit_examine[~(df_credit_examine.tips.str.contains('发货需上锁', na=False))]
# # df_credit = df_credit[~(df_credit.tips.str.contains('发货需上锁', na=False))]
# # # 基于下单日期和id_card_num去重
# df_credit = df_credit.drop_duplicates(subset=['下单日期', 'id_card_num'],keep="first")
# df_credit_group = df_credit.groupby('下单日期').agg({'order_id':'count'})
# # df_creadit_list = df_credit[df_credit['下单日期']=='2025-08-18']['order_id'].tolist()
# # df_credit.tail()
# # len(df_creadit_list)
# df_credit_group

# 芝麻租物-支付宝端外渠道推广

In [ ]:
# df_tg = df[df['activity_name']=='芝麻租物-支付宝端外渠道推广']
# df_tg2 = df2[df2['activity_name']=='芝麻租物-支付宝端外渠道推广']
# df_zm_zfb = all_models.data_group(df_tg, df_tg2, df_risk_examine, '下单日期')
# Today_dw = datetime.now().strftime('%Y%m%d%H')
# # df_zm_zfb[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核", '出库']].to_excel(f'F:/需求/七月需求/支付宝端外渠道_{Today_dw}.xlsx')


# 租物  搜索的免审进件 

In [ ]:
#租物  搜索的免审进件             
# dfms = df[(df.is_vip=='1')&(df.status_result=='0')]
# dfms0 = pd.crosstab(dfms["下单日期"],dfms["归属渠道"],margins=True)
# dfms0[["芝麻租物","搜索渠道"]]


# 各渠道每日免审出库单数统计

In [ ]:
# 剔除据量的数据
# dfck_ms = dfck[~dfck.tips.str.contains(r'策略2412|命中自有模型回捞策略')==True]
# dfdfck=dfck_ms[(dfck_ms.is_vip=='1')&(dfck_ms.status_result=='0')]

# pd.crosstab(dfdfck["下单日期"],dfdfck["归属渠道"],margins=True)[['芝麻租物','搜索渠道']]

# 取消原因 & 拒绝原因

## 取消原因-总体

In [ ]:
# # 惜仪需求
# path_result = 'F:/myfile/数据提取及分析/'
# excel_name = f'取消&拒绝原因_{t_date}.xlsx'
# dfms = df2
# # 取消原因 总体
# qx_zt_df = pd.crosstab(dfms["下单日期"],dfms["取消原因"],margins=True)

# # 拒绝原因总体
# jj_zt_df = pd.crosstab(dfms["下单日期"],dfms["电审拒绝原因"],margins=True)
# #免审进件的电审拒绝原因
# dfms=df2[(df2["decision_result"].str.contains(pat="免人审",regex=False)==True)&(df2.is_vip=='1')&(df2.status_result=='0')]
# jj_ms_df = pd.crosstab(dfms["下单日期"],dfms["电审拒绝原因"],margins=True)
# #免审进件的取消原因
# dfms=df2[(df2["decision_result"].str.contains(pat="免人审",regex=False)==True)&(df2.is_vip=='1')&(df2.status_result=='0')]
# qx_ms_df = pd.crosstab(dfms["下单日期"],dfms["取消原因"],margins=True)

# with pd.ExcelWriter(path_result + excel_name, engine='xlsxwriter') as writer:  
#     qx_zt_df.to_excel(writer, sheet_name='取消原因-总体', startrow=0, index=True)
#     jj_zt_df.to_excel(writer, sheet_name='拒绝原因-总体', startrow=0, index=True) 
#     jj_ms_df.to_excel(writer, sheet_name='拒绝原因-免审', startrow=0, index=True) 
#     qx_ms_df.to_excel(writer, sheet_name='取消原因-免审', startrow=0, index=True) 

In [ ]:
#免审进件的电审拒绝原因
# dfms=df_j[df_j["决策结果"].str.contains(pat="免人审",regex=False)==True]

# path_result = 'F:/myfile/数据提取及分析/'
# excel_name = f'取消&拒绝原因_{t_date}.xlsx'

In [ ]:
# df2.loc[:, '取消原因2'] = df2.取消原因.str.split('"').str[0]
# qx_zt_df2 = pd.crosstab(df2["下单月份"],df2["取消原因2"],margins=True)
# qx_zt_df2.to_excel('F:/需求/总需求/取消原因.xlsx')

In [ ]:
# dfms = df2
# qx_zt_df = pd.crosstab(dfms["下单日期"],dfms["取消原因"],margins=True)
# 
# with pd.ExcelWriter(path_result + excel_name, engine='xlsxwriter') as writer:  
#     qx_zt_df.to_excel(writer, sheet_name='取消原因-总体', startrow=0, index=True) 

## 拒绝原因-总体

In [ ]:
# #免审进件的电审拒绝原因，免审内容存放在"决策结果"字段中
# # dfms=df_j[df_j["决策结果"].str.contains(pat="免人审",regex=False)==True]
# jj_zt_df = pd.crosstab(dfms["下单日期"],dfms["电审拒绝原因"],margins=True)
# with pd.ExcelWriter(path_result + excel_name, engine='openpyxl',mode='a') as writer:  
#     jj_zt_df.to_excel(writer, sheet_name='拒绝原因-总体', startrow=0, index=True) 


## 特定时间的拒绝原因

In [ ]:
# path="D:/需求/"
# # excel_name = f'取消&拒绝原因_{t_date}.xlsx'
# df_dsjj = df2[df2['电审拒绝原因'].notna()]
# # df_dsjj.dtypes
# # 获取下单日期'2025-06-01'到'2025-06-10'的数据
# df_dsjj6 = df_dsjj[(df_dsjj['下单日期']>='2025-06-01')&(df_dsjj['下单日期']<='2025-06-10')]
# df_ds_tab6 = df_dsjj6.groupby('电审拒绝原因').size().reset_index(name='数量').sort_values(by='数量',ascending=False)
# # 计算总数量
# total_count6 = df_ds_tab6['数量'].sum()
# # 计算比例比率
# df_ds_tab6['比例'] = (df_ds_tab6['数量'] / total_count6).apply(lambda x: format(x, ".2%"))
# # 在最后一行加入合计
# df_ds_tab6.loc['合计'] = ['总计', total_count6, '100.00%']


# # 获取下单日期'2025-07-07'到'2025-07-16'的数据
# df_dsjj7 = df_dsjj[(df_dsjj['下单日期']>='2025-07-07')&(df_dsjj['下单日期']<='2025-07-16')]

# df_ds_tab7 = df_dsjj7.groupby('电审拒绝原因').size().reset_index(name='数量').sort_values(by='数量',ascending=False)
# # 计算总数量
# total_count7 = df_ds_tab7['数量'].sum()
# # 计算比例比率
# df_ds_tab7['比例'] = (df_ds_tab7['数量'] / total_count7).apply(lambda x: format(x, ".2%"))
# # 在最后一行加入合计
# df_ds_tab7.loc['85'] = ['总计', total_count7, '100.00%']

# # 创建一个空的dataframe，用于填充空列
# empty_df = pd.DataFrame({}, columns=[f'empty_{i}' for i in range(1, 4)])


# # 将df_ds_tab6，df_ds_tab7 隔三空列 沿列方向拼接成一个dataframe
# df_ds_tab = pd.concat([df_ds_tab6, empty_df, df_ds_tab7], axis=1)
# # 写入excel
# with pd.ExcelWriter(path + f'取消原因_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_ds_tab.to_excel(writer, sheet_name='拒绝原因-特定时间', index=False)

# # df_ds_tab

### 筛选出电审拒绝原因

In [ ]:
# # 筛选出电审拒绝原因为'已在其他平台租赁"]'或'验证资料与电审不符且差距大"]'下单日期'2025-07-15'到'2025-07-16'的数据
# df_dsjj_new = df2[(df2['电审拒绝原因'].isin(['已在其他平台租赁"]', '验证资料与电审不符且差距大"]']))&((df2['下单日期']>='2025-07-15')&(df2['下单日期']<='2025-07-16'))]
# df_dsjj_new['电审拒绝原因']


### 关联分配人

In [ ]:
# sql_name = '''
#         SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
#         FROM db_digua_business.t_user_verify_order_record tuvor
#         left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
#         where tuvor.del_flag = 0 ORDER BY tuvor.update_time
#         '''
# df_name = query(sql_name)
# df_name

# df_name = df_name.sort_values(by='update_time', ascending=False).groupby('order_id').head(1)
# # 创建分配人映射关系 
# name_dict = {
#         '小张': '李巧玲',
#         '小周': '李巧凤',
#         '小南': '刘三妹',
#         '小何': '何静',
#         '小谢': '谢金凤',
#         '小咪': '杨健',
#         '小慧': '林思慧',
#         '小滢': '胡彩滢',
#         '小星': '廖丽敏',
#         '小兰': '黄兰娟',
#         '小晚': '周莹',
#         '小芳': '罗芳'
#         }
# name_list = ['李巧玲','李巧凤','刘三妹','何静','谢金凤','廖丽敏','李楠','黄兰娟','杨健','林思慧','胡彩滢','周莹','罗芳', '周念慈', '周汉鸿']
# # 匹配映射字典
# df_name.loc[:, '信审员'] = df_name.分配人.apply(lambda x: name_dict[x] if str(x).startswith('小') and str(x) in name_dict.keys() else x)
# # df_name = df_name[df_name.信审员.isin(name_list)]
# df_name

In [ ]:
# # 关联分配人
# df_dsjj_all = df_dsjj_new.merge(df_name[['order_id', '信审员']], on='order_id', how='left')
# # df_dsjj_all.dtypes
# df_dsjj_all[['order_number', '信审员', '电审拒绝原因']].rename(columns={'order_number': '订单号'})
# # 写入excel
# with pd.ExcelWriter(path + f'电审拒绝原因&信审员_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_dsjj_all[['order_number', '信审员', '电审拒绝原因']].rename(columns={'order_number': '订单号'}).to_excel(writer, sheet_name='拒绝原因-特定时间', index=False, startrow=1)

## 免审进件的电审拒绝原因

In [ ]:
# #免审进件的电审拒绝原因
# dfms=df2[df2["decision_result"].str.contains(pat="免人审",regex=False)==True]
# jj_ms_df = pd.crosstab(dfms["下单日期"],dfms["电审拒绝原因"],margins=True)
# with pd.ExcelWriter(path_result + excel_name, engine='openpyxl',mode='a') as writer:  
#     jj_ms_df.to_excel(writer, sheet_name='拒绝原因-免审', startrow=0, index=True) 

## 免审进件的取消原因

In [ ]:
# #免审进件的取消原因
# dfms=df2[df2["decision_result"].str.contains(pat="免人审",regex=False)==True]
# qx_ms_df = pd.crosstab(dfms["下单日期"],dfms["取消原因"],margins=True)
# with pd.ExcelWriter(path_result + excel_name, engine='openpyxl',mode='a') as writer:  
#     qx_ms_df.to_excel(writer, sheet_name='取消原因-免审', startrow=0, index=True) 

#  免押订单转化率 全免押

In [ ]:
# df_qmy = df[df['押金类型']=='全免押']
# df_qmy2 = df2[df2['押金类型']=='全免押']
# df_qmy_group = all_models.data_group(df_qmy, df_qmy2, df_risk_examine, '下单日期')
# 
# df_qmy_group['出库（按下单时间）'] = df_my['全免押'][:-1]	
# # df_qmy_group['出库（按下单时间）'] = df_qmy_group['出库']
# 
# df_qmy_group[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","待审核", '出库（按下单时间）', '进件出库率', '取消率', '人审转化率', '人审拒绝率']]

# 非免押（部分免押）

In [ ]:
# df_fmy = df[df['押金类型']=='部分免押']
# df_fmy2 = df2[df2['押金类型']=='部分免押']
# df_fmy_group = all_models.data_group(df_fmy, df_fmy2, df_risk_examine, '下单日期')
# df_fmy_group['待审核'] = 0
# df_fmy_group[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消", '待审核']]

In [ ]:
# S_list = ['租机宝','八派信息','CPS直播','派金花','富士康租机','九州信息','勉丫租','哈银','美仑美奂','兔丫租赁','分期乐1','分期乐2','98租超','邦客惠乘车码','推一推','硬派抖音']
# # , '单人聊天会话中的小程序消息卡片（分享）', '芝麻租物', '搜索渠道'
# df_s = df.copy()
# df2_s = df2.copy()
# df_s.loc[:, '渠道'] = np.where(df_s.归属渠道.isin(S_list), 'S渠道', 
#                         np.where(df_s.归属渠道=='单人聊天会话中的小程序消息卡片（分享）', '单人聊天会话中的小程序消息卡片（分享）', 
#                                 np.where(df_s.归属渠道=='芝麻租物', '芝麻租物',
#                                         np.where(df_s.归属渠道=='搜索渠道', '搜索渠道', '其他'))))
# df2_s.loc[:, '渠道'] = np.where(df2_s.归属渠道.isin(S_list), 'S渠道', 
#                         np.where(df2_s.归属渠道=='单人聊天会话中的小程序消息卡片（分享）', '单人聊天会话中的小程序消息卡片（分享）', 
#                                 np.where(df2_s.归属渠道=='芝麻租物', '芝麻租物',
#                                         np.where(df2_s.归属渠道=='搜索渠道', '搜索渠道', '其他'))))
# df_s_new = df_s[(df_s.渠道!='其他')&(df_s['押金类型']=='部分免押')]
# df2_s_new = df2_s[(df2_s.渠道!='其他')&(df2_s['押金类型']=='部分免押')]
# df_s_group = all_models.data_group(df_s_new, df2_s_new, df_risk_examine, '渠道')
# df_s_group[["去重订单数","前置拦截","拦截率", '进件前取消', "进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                 "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]

In [ ]:
# df_s_new = df_s[(df_s.渠道!='其他')&(df_s['押金类型']=='全免押')]
# df2_s_new = df2_s[(df2_s.渠道!='其他')&(df2_s['押金类型']=='全免押')]
# df_s_group = all_models.data_group(df_s_new, df2_s_new, df_risk_examine, '渠道')
# df_s_group[["去重订单数","前置拦截","拦截率", '进件前取消', "进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                 "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]

# 商家转化数据

In [ ]:
df_contain["待审核"] = np.where(df_contain["审核状态"]=='待审核',1,0)
df_contain["前置拦截"] = np.where(df_contain["审核状态"]=='前置拦截',1,0)
df_contain["机审强拒"] = np.where(df_contain["审核状态"]=='机审强拒',1,0)
df_contain["人审拒绝"] = np.where(df_contain["审核状态"]=='人审拒绝',1,0)
df_contain["客户取消"] = np.where(df_contain["审核状态"]=='客户取消',1,0)
df_contain["出库前风控强拒"] = np.where(df_contain["审核状态"]=='出库前风控强拒',1,0)
df_contain["无法联系"] = np.where(df_contain["审核状态"]=='无法联系',1,0)
df_contain["是否进件"] = np.where(df_contain["进件"]=='进件',1,0)
df_contain["是否出库"] = np.where(df_contain["status"].isin([2,3,4,5,6,8,15]),1,0)

df_contain["进件前取消"] = np.where(df_contain["审核状态"]=='进件前取消',1,0)

In [ ]:
# list1 = ['澄心优租', '优优2店', '北京海鸟窝科技有限公司', '租着用电脑数码', '趣智数码', '汇客好租', '致佳', '乙辉数码']
# list2 = ['小蚂蚁租机', '兴鑫兴通讯']

In [ ]:
# df_contain1 = df_contain[(df_contain.是否出库==1)&(df_contain.merchant_name.isin(list1))]
# df_contain1_group = df_contain1.groupby('merchant_name').agg({'all_rental': 'sum'})
# df_contain2 = df_contain[(df_contain.是否出库==1)&(df_contain.merchant_name.isin(list2))]
# df_contain2_group = df_contain2.groupby('merchant_name').agg({'contract_price': 'sum'})
# with pd.ExcelWriter('F:/需求/瑞彤需求/商家转化.xlsx', engine='openpyxl') as writer:
#     df_contain1_group.to_excel(writer, sheet_name='总租金')
#     df_contain2_group.to_excel(writer, sheet_name='买断价')

## 澄心优租

In [ ]:
# 澄心优租
# cxyz = all_models.merchant_names(df_contain, '澄心优租', '下单日期')
# cxyz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","订单出库率"]]

## 优优2店

In [ ]:
# 优优2店
# yy2 = all_models.merchant_names(df_contain,'优优2店', '下单日期')
# yy2 = yy2[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# yy2

In [ ]:
# df_contain_ck = df_contain[df_contain['是否出库']==1]
# df_contain_ck['机型'] == df_contain_ck['product_name'].str.extract(r'iPhone ?\d+(ProMax|Pro|Max)')[0]
# df_contain_yy2 = all_models.merchant_names(df_contain,'优优2店', '机型')[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','订单出库率']]


In [ ]:
# with pd.ExcelWriter('F:/需求/七月需求/机型转化.xlsx', engine='openpyxl') as writer:
#     df_two_all_model.to_excel(writer, sheet_name='自营二手手机型转化_汇总')
#     df_contain_yy2.to_excel(writer, sheet_name='优优2店机型转化_汇总')
#     df_two_all.to_excel(writer, sheet_name='自营二手手机机型转化_明细')
#     yy2.to_excel(writer, sheet_name='优优2点机型转化_明细')

## 北京海鸟窝科技有限公司

In [ ]:
# 北京海鸟窝科技有限公司
# hnw = all_models.merchant_names(df_contain,'北京海鸟窝科技有限公司', '下单日期')
# hnw[["去重订单数","进件数","预授权通过率","出库","进件出库率"]]

## 租着用电脑数码

In [ ]:
# 租着用电脑数码
# zzy = all_models.merchant_names(df_contain,'租着用电脑数码', '下单日期')
# zzy[["去重订单数","进件数","预授权通过率","出库","进件出库率"]]

## 格木木二奢名品

In [ ]:
# 格木木二奢名品
# gmm = all_models.merchant_names(df_contain,'格木木二奢名品', '下单日期')
# gmm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]


## 广州康基贸易有限公司

In [ ]:
# 广州康基贸易有限公司
# kjmy = all_models.merchant_names(df_contain,'广州康基贸易有限公司', '下单日期')
# kjmy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]


## 趣智数码

In [ ]:
# 趣智数码
# qzsm = all_models.merchant_names(df_contain,'趣智数码', '下单日期')
# qzsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]


## 汇客好租

In [ ]:
# 汇客好租
# hkhz = all_models.merchant_names(df_contain,'汇客好租', '下单日期')
# hkhz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]

df_hk_pc = df_contain[(df_contain.merchant_name=='汇客好租')&(df_contain.classify_id.isin([125,126,127,185]))]

In [ ]:
# # 汇客好租
# hkhz = all_models.merchant_names(df_contain,'汇客好租', '下单日期')
# hkhz = hkhz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# hkhz = clean.continuous_dates(hkhz)
# hkhz_new = hkhz.reset_index(names=['下单日期'])
# # 电脑
# df_contain_dy = df_contain[(df_contain.归属渠道=='抖音渠道')&(df_contain.classify_id.isin([125,126,127,185]))]
# hkhz_dy = all_models.merchant_names(df_contain_dy,'汇客好租', '下单日期')
# hkhz_dy = hkhz_dy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# hkhz_dy = clean.continuous_dates(hkhz_dy)
# hkhz_dy_new = hkhz_dy.reset_index(names=['下单日期'])


try:
    # 汇客好租
    hkhz = all_models.merchant_names(df_contain, '汇客好租', '下单日期')
    if hkhz.empty:
        print("warn: 没有数据")
        hkhz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
    else:
        hkhz = hkhz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        hkhz = clean.continuous_dates(hkhz)
        hkhz_new = hkhz.reset_index(names=['下单日期'])
except:
    hkhz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])

try:
    # 电脑
    df_contain_dy = df_contain[(df_contain.归属渠道=='抖音渠道')&(df_contain.classify_id.isin([125,126,127,185]))]
    hkhz_dy = all_models.merchant_names(df_contain_dy, '汇客好租', '下单日期')
    if hkhz_dy.empty:
        print("warn: 没有数据")
        hkhz_dy_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
    else:
        hkhz_dy = hkhz_dy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        hkhz_dy = clean.continuous_dates(hkhz_dy)
        hkhz_dy_new = hkhz_dy.reset_index(names=['下单日期'])
except:
    hkhz_dy_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])

hkhz_new.tail(), hkhz_dy_new.tail()

### 临时需求：获取抖音订单的已发货客户信息

In [ ]:
# # 抖音订单：order_type = 'DY_ORDER'
# # 已发货 status notin (1,2,10,12,13)其中4为租赁中
# df_hkhz_dy = df_contain[(df_contain.merchant_name=='汇客好租')&(df_contain.order_type=='DY_ORDER')]
# # 剔除status为1,2,10,12,13的订单
# df_hkhz_dy = df_hkhz_dy[~df_hkhz_dy.status.isin([1,2,10,12,13])]
# df_hkhz_dy.loc[:, '订单状态'] = np.where(df_hkhz_dy.status==4, '租赁中', '已发货')
# df_hkhz_dy.loc[:, '是否逾期'] = np.where((df_hkhz_dy.overdue==1)&(df_hkhz_dy.status==4), '逾期', '未逾期')
# df_hkhz_dy.loc[:, '订单类型'] = np.where(df_hkhz_dy.order_type=='DY_ORDER', '抖音订单', '非抖音订单')
# # df_hkhz_dy_new = df_hkhz_dy[['merchant_name','订单类型','order_number', '订单状态', 'true_name', 'user_mobile', 'id_card_num', '是否逾期', '下单日期']].rename(columns={'merchant_name': '商户名称', 'order_number': '订单号', 'true_name': '姓名', 'user_mobile': '手机号', 'id_card_num': '身份证号'})
# # df_hkhz_dy_new.to_excel(f'F:/需求/倩姐需求/汇客好租抖音订单_{t_date}.xlsx', index=False)

In [ ]:
# df_hkhz = df_contain[(df_contain.merchant_name=='汇客好租')]
# # 获取租赁中且逾期的订单
# df_hkhz_yq = df_hkhz[(df_hkhz.status==4)&(df_hkhz.overdue==1)]
# df_hkhz_yq.loc[:,'逾期金额'] = df_hkhz_yq.new_actual_money - df_hkhz_yq.已收租金 - df_hkhz_yq.已付买断尾款
# # 按下单月份分组统计逾期订单数和逾期金额
# df_hkhz_yq_group = df_hkhz_yq.groupby(df_hkhz_yq['下单月份']).agg({'order_number': 'count', '逾期金额': 'sum'}).rename(columns={'order_number': '逾期订单数'}).reset_index()
# df_hkhz_yq_group#.to_excel(f'F:/需求/倩姐需求/汇客好租逾期订单_{t_date}.xlsx', index=False)
# df_hkhz_yq[['order_number','下单日期','new_actual_money','已收租金','已付买断尾款','逾期金额']]
# df_hkhz_yq['new_actual_money'].mean()

## 小蚂蚁租机

In [ ]:
# df_contain_xmy = df_contain[(df_contain.merchant_name=='小蚂蚁租机')&(df_contain.是否出库==1)]#[['order_number']].to_excel('F:/需求/总需求/小蚂蚁出库.xlsx', index=False)
# pd.pivot_table(df_contain_xmy, values='order_number', index='下单月份', columns='overdue_type', aggfunc='count').to_excel('F:/需求/总需求/小蚂蚁出库逾期.xlsx')

In [ ]:
# 小蚂蚁租机
# xmy = all_models.merchant_names(df_contain,'小蚂蚁租机', '下单日期')
# xmy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]


In [ ]:

# merchant_names('小蚂蚁租机', '型号内存').to_excel('F:/宏生/小蚂蚁机型转化.xlsx')
# df_contain['product_name'].drop_duplicates()

## 乙辉数码

In [ ]:
# 乙辉数码
# yhsm = all_models.merchant_names(df_contain,'乙辉数码', '下单日期')
# yhsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]


In [ ]:
# xxxd = all_models.merchant_names(df_contain,'线下小店', '下单日期')
# xxxd

# 每日各渠道去重订单统计 导出

In [ ]:
# #每日各渠道去重订单统计结果表格导出
# gsqd_qcdd = pd.crosstab(df["下单日期"],df["归属渠道"],margins=True)
# 
# with pd.ExcelWriter(f'F:/myfile/p站数据/订单日报/日报相关数据输出_{t_date}.xlsx', engine='xlsxwriter') as writer:    #, mode='a'
#     # 将df写入新的sheet，名为'new_sheet'  
#     gsqd_qcdd.to_excel(writer, sheet_name='归属渠道去重订单统计', index=True)  # index=False 表示不导出行索引


In [ ]:
# lyqd_qcdd = pd.crosstab(df["下单日期"],df["来源渠道"],margins=True)
# 
# with pd.ExcelWriter(f'F:/myfile/p站数据/订单日报/日报相关数据输出_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     lyqd_qcdd.to_excel(writer, sheet_name='来源渠道去重订单统计', index=True)  # index=False 表示不导出行索引  

In [ ]:
# hdmc_qcdd = pd.crosstab(df["下单日期"],df["activity_name"],margins=True)
# 
# with pd.ExcelWriter(f'F:/myfile/p站数据/订单日报/日报相关数据输出_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     hdmc_qcdd.to_excel(writer, sheet_name='活动名称去重订单统计', index=True)


# 每日各渠道进件统计 导出

In [ ]:
# #每日各渠道进件统计结果表格导出
# gsqd_jj = pd.crosstab(df_j["下单日期"],df_j["归属渠道"],margins=True)
# 
# with pd.ExcelWriter(f'F:/myfile/p站数据/订单日报/日报相关数据输出_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     gsqd_jj.to_excel(writer, sheet_name='归属渠道进件统计', index=True)  # index=False 表示不导出行索引 


In [ ]:
# #每日各渠道进件统计结果表格导出
# lyqd_jj = pd.crosstab(df_j["下单日期"],df_j["来源渠道"],margins=True)
# 
# with pd.ExcelWriter(f'F:/myfile/p站数据/订单日报/日报相关数据输出_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     lyqd_jj.to_excel(writer, sheet_name='来源渠道进件统计', index=True)  # index=False 表示不导出行索引 


In [ ]:
# hdmc_jj = pd.crosstab(df_j["下单日期"],df_j["activity_name"],margins=True)
# 
# with pd.ExcelWriter(f'F:/myfile/p站数据/订单日报/日报相关数据输出_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     hdmc_jj.to_excel(writer, sheet_name='活动名称进件统计', index=True)

# 各归属渠道 出库 导出

In [ ]:
# # dfck_today=dfck[dfck["月份_x"]=="2024-6-30"]
# gsqd_ck = pd.crosstab(dfck["下单日期"],dfck["归属渠道"],margins=True)
# 
# 
# with pd.ExcelWriter(f'F:/myfile/p站数据/订单日报/日报相关数据输出_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     gsqd_ck.to_excel(writer, sheet_name='归属渠道出库统计', index=True)  # index=False 表示不导出行索引 


In [ ]:
# #每日各渠道进件统计结果表格导出
# lyqd_ck = pd.crosstab(dfck["下单日期"],dfck["来源渠道"],margins=True)
# 
# with pd.ExcelWriter(f'F:/myfile/p站数据/订单日报/日报相关数据输出_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     lyqd_ck.to_excel(writer, sheet_name='来源渠道出库统计', index=True)  # index=False 表示不导出行索引 


In [ ]:
# hdmc_ck = pd.crosstab(dfck["下单日期"],dfck["activity_name"],margins=True)
# 
# with pd.ExcelWriter(f'F:/myfile/p站数据/订单日报/日报相关数据输出_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     hdmc_ck.to_excel(writer, sheet_name='活动名称出库统计', index=True) 

# 总体 "租完即送"占比

In [ ]:
# #总体 "租完即送"占比
# 
# df_rent  = pd.crosstab(df_j["下单日期"],df_j["租赁方案"],margins=True)
# df_rent=df_rent.rename(columns={"租完即送":"租完即送进件","租完归还":"租完归还进件","All":"合计进件",})
# df_rent_ck=pd.crosstab(dfck["下单日期"],dfck["租赁方案"],margins=True)
# df_rent_ck=df_rent_ck.rename(columns={"租完即送":"租完即送出库","租完归还":"租完归还出库","All":"合计出库",})
# df_r=pd.merge(df_rent,df_rent_ck,left_index=True, right_index=True)
# df_r["租完即送进件占比"]=df_r["租完即送进件"]/df_r["合计进件"]
# df_r["租完即送进件占比"]=df_r["租完即送进件占比"].apply(lambda x:format(x,".2%"))
# df_r["租完归还进件占比"]=df_r["租完归还进件"]/df_r["合计进件"]
# df_r["租完归还进件占比"]=df_r["租完归还进件占比"].apply(lambda x:format(x,".2%"))
# df_r["租完即送出库占比"]=df_r["租完即送出库"]/df_r["合计出库"]
# df_r["租完即送出库占比"]=df_r["租完即送出库占比"].apply(lambda x:format(x,".2%"))
# df_r["租完归还出库占比"]=df_r["租完归还出库"]/df_r["合计出库"]
# df_r["租完归还出库占比"]=df_r["租完归还出库占比"].apply(lambda x:format(x,".2%"))
# df_r["租完即送转化率"]=df_r["租完即送出库"]/df_r["租完即送进件"]
# df_r["租完即送转化率"]=df_r["租完即送转化率"].apply(lambda x:format(x,".2%"))
# df_r["租完归还转化率"]=df_r["租完归还出库"]/df_r["租完归还进件"]
# df_r["租完归还转化率"]=df_r["租完归还转化率"].apply(lambda x:format(x,".2%"))
# 
# df_r[["租完即送进件","租完归还进件","租完归还进件占比","租完即送进件占比","租完即送出库","租完归还出库","租完归还出库占比","租完即送出库占比",'租完即送转化率','租完归还转化率']]


# 总体出库订单碎屏险购买数据

In [ ]:
# def spx(df_ck, df_buy):
#     df_bx_ck=pd.crosstab(df_ck["下单日期"],df_buy["buy_service_product"],margins=True)
#     df_bx_ck=df_bx_ck.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
#     # '服务订单状态：1、待支付；2、待确认；3、服务中；4、已失效；5、已取消；6、已退款',
#     df_ck['已取消'] = np.where(df_ck["service_status"]==5,1,0)
#     df_ck['已退款'] = np.where(df_ck["service_status"]==6,1,0)
#     df_ck['待支付'] = np.where(df_ck["service_status"]==1,1,0)
#     df_ck['待确认'] = np.where(df_ck["service_status"]==2,1,0)
#     df_ck['服务中'] = np.where(df_ck["service_status"]==3,1,0)
#     df_s3  =   df_ck.groupby(["下单日期"]).agg({'已取消':'sum','已退款':'sum', '待支付':'sum', '待确认':'sum','服务中':'sum', '碎屏险价格': 'sum'})  
# 
#     df_s2merge=pd.merge(df_bx_ck,df_s3,left_index=True, right_index=True)
#     df_s2merge["最终实际支付碎屏险出库"]=df_s2merge["待确认"]+df_s2merge["服务中"] +df_s2merge["待支付"]
#     df_s2merge["实际支付碎屏险比例"]=df_s2merge["最终实际支付碎屏险出库"]/df_s2merge["合计出库"]
#     df_s2merge["实际支付碎屏险比例"]=df_s2merge["实际支付碎屏险比例"].apply(lambda x:format(x,".2%"))
#     
#     return df_s2merge

In [ ]:
# #总体出库订单碎屏险购买数据
# spx_all = spx(dfck, dfck)[['不购买碎屏险出库','购买碎屏险出库','合计出库','已取消','已退款', '待支付', '待确认','服务中','最终实际支付碎屏险出库']]
# spx_all = spx_all.reset_index()
# # 月数据
# spx_all['月份'] = spx_all['下单日期'].dt.strftime('%Y-%m')
# spx_all_group = spx_all.groupby('月份').agg({'不购买碎屏险出库': 'sum','购买碎屏险出库': 'sum','合计出库': 'sum','已取消': 'sum','已退款': 'sum', '待支付': 'sum', '待确认': 'sum','服务中': 'sum','最终实际支付碎屏险出库': 'sum'})
# spx_all_group['实际支付碎屏险比例'] = spx_all_group['最终实际支付碎屏险出库']/spx_all_group['合计出库']
# spx_all_group['实际支付碎屏险比例'] = spx_all_group['实际支付碎屏险比例'].apply(lambda x:format(x,".2%"))
# 
# # 周数据
# spx_all1 = spx_all.reset_index().set_index('下单日期')
# week_spx_all = week_models.W_group(week_models.custom_weekly_resampler(spx_all1, 7)).drop(columns=['week_day'], inplace=False)
# week_spx_all['实际支付碎屏险比例'] = week_spx_all['最终实际支付碎屏险出库']/week_spx_all['合计出库']
# week_spx_all['实际支付碎屏险比例'] = week_spx_all['实际支付碎屏险比例'].apply(lambda x:format(x,".2%"))
# spx_all_group

# 芝麻租物出库订单碎屏险购买数据

In [ ]:
# #芝麻租物出库订单碎屏险购买数据
# dfckzw=dfck[dfck["归属渠道"]=="芝麻租物"]
# spx_zw = spx(dfckzw, dfckzw)[['不购买碎屏险出库','购买碎屏险出库','合计出库','已取消','已退款', '待支付', '待确认','服务中','最终实际支付碎屏险出库']]
# spx_zw = spx_zw.reset_index()
# # 月数据
# spx_zw['月份'] = spx_zw['下单日期'].dt.strftime('%Y-%m')
# spx_zw_group = spx_zw.groupby('月份').agg({'不购买碎屏险出库': 'sum','购买碎屏险出库': 'sum','合计出库': 'sum','已取消': 'sum','已退款': 'sum', '待支付': 'sum', '待确认': 'sum','服务中': 'sum','最终实际支付碎屏险出库': 'sum'})
# spx_zw_group['实际支付碎屏险比例'] = spx_zw_group['最终实际支付碎屏险出库']/spx_zw_group['合计出库']
# spx_zw_group['实际支付碎屏险比例'] = spx_zw_group['实际支付碎屏险比例'].apply(lambda x:format(x,".2%"))
# # 周数据
# spx_zw1 = spx_zw.reset_index().set_index('下单日期')
# week_spx_zw = week_models.W_group(week_models.custom_weekly_resampler(spx_zw1, 7)).drop(columns=['week_day'], inplace=False)
# week_spx_zw['实际支付碎屏险比例'] = week_spx_zw['最终实际支付碎屏险出库']/week_spx_zw['合计出库']
# week_spx_zw['实际支付碎屏险比例'] = week_spx_zw['实际支付碎屏险比例'].apply(lambda x:format(x,".2%"))
# spx_zw_group

In [ ]:
# dfckzw[(dfckzw.buy_service_product==2)]

# 搜索渠道出库订单碎屏险购买数据

In [ ]:
# dfckss=dfck[dfck["归属渠道"]=="搜索渠道"]
# spx_ss = spx(dfckss, dfckss)[['不购买碎屏险出库','购买碎屏险出库','合计出库','已取消','已退款', '待支付', '待确认','服务中','最终实际支付碎屏险出库']]
# spx_ss = spx_ss.reset_index()
# # 月数据
# spx_ss['月份'] = spx_ss['下单日期'].dt.strftime('%Y-%m')
# spx_ss_group = spx_ss.groupby('月份').agg({'不购买碎屏险出库': 'sum','购买碎屏险出库': 'sum','合计出库': 'sum','已取消': 'sum','已退款': 'sum', '待支付': 'sum', '待确认': 'sum','服务中': 'sum','最终实际支付碎屏险出库': 'sum'})
# spx_ss_group['实际支付碎屏险比例'] = spx_ss_group['最终实际支付碎屏险出库']/spx_ss_group['合计出库']
# spx_ss_group['实际支付碎屏险比例'] = spx_ss_group['实际支付碎屏险比例'].apply(lambda x:format(x,".2%"))
# # 周数据
# spx_ss1 = spx_ss.reset_index().set_index('下单日期')
# week_spx_ss = week_models.W_group(week_models.custom_weekly_resampler(spx_ss1, 7)).drop(columns=['week_day'], inplace=False)
# week_spx_ss['实际支付碎屏险比例'] = week_spx_ss['最终实际支付碎屏险出库']/week_spx_ss['合计出库']
# week_spx_ss['实际支付碎屏险比例'] = week_spx_ss['实际支付碎屏险比例'].apply(lambda x:format(x,".2%"))
# spx_ss_group

In [ ]:
# with pd.ExcelWriter(f'F:/需求/七月需求/碎屏险数据_month_{Today_dw}.xlsx', engine='openpyxl') as writer:
#     spx_all_group.to_excel(writer, sheet_name='总体出库碎屏险月汇总')
#     spx_zw_group.to_excel(writer, sheet_name='芝麻租物出库碎屏险月汇总')
#     spx_ss_group.to_excel(writer, sheet_name='搜索渠道出库碎屏险月汇总')
#     week_spx_all.to_excel(writer, sheet_name='总体出库碎屏险周汇总',  index=False)
#     week_spx_zw.to_excel(writer, sheet_name='芝麻租物出库碎屏险周汇总',  index=False)
#     week_spx_ss.to_excel(writer, sheet_name='搜索渠道出库碎屏险周汇总',  index=False)
#     spx_all.to_excel(writer, sheet_name='总体出库碎屏险',  index=False)
#     spx_zw.to_excel(writer, sheet_name='芝麻租物出库碎屏险',  index=False)
#     spx_ss.to_excel(writer, sheet_name='搜索渠道出库碎屏险',  index=False)

# 小蚂蚁租机碎屏险


In [ ]:
# df_xmy = df[df.merchant_name=='小蚂蚁租机']
# spx(df_xmy, df_xmy)[['购买碎屏险出库', '碎屏险价格']]#.to_excel(f'F:/需求/瑞彤需求/小蚂蚁租机碎屏险_{Today_zfb}.xlsx')

# 包含邦道渠道拒绝原因

In [ ]:
# df_bd = df[df['来源渠道'].str.contains(pat='邦道', regex=True)]
# df_bd.columns
# df_bd_jj = df_bd.query('机审强拒==1|人审拒绝==1')
# df_bd['拒绝理由'].value_counts()[:10]
# df_bd['电审拒绝原因'].drop_duplicates()

# 生活号(婷玉需求)

In [102]:
# TO_DO婷玉，支付宝需求# TO_DO日常
df_shh = df[df['activity_name'].str.contains('生活号')==True]
df_shh2 = df2[df2['activity_name'].str.contains('生活号')==True]
# df_shh_group = all_models.data_group(df_shh, df_shh2, df_risk_examine, 'channel_name')
df_shh_group = all_models.data_group_contain_hl(df_shh, df_shh2, df_risk_examine, '下单日期')
df_shh_group = df_shh_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核", '出库', '进件出库率']]

# 支付宝来源渠道转化    
df_shh_ly = df[df['来源渠道'].str.contains('生活号')==True]
df_shh_ly2 = df2[df2['来源渠道'].str.contains('生活号')==True]
df_shh_ly_group = all_models.data_group_contain_hl(df_shh_ly, df_shh_ly2, df_risk_examine, '下单日期')
df_shh_ly_group['出库（按下单时间）'] = df_shh_ly_group['出库']
df_shh_ly_group = df_shh_ly_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]


with pd.ExcelWriter(f'F:/需求/崔婷玉需求/生活号_{t_date}.xlsx', engine='openpyxl') as writer:
    df_shh_group.to_excel(writer, sheet_name='生活号')
    df_shh_ly_group.to_excel(writer, sheet_name='来源渠道')

# df_shh_group.to_excel(f'F:/需求/崔婷玉需求/生活号_{t_date}.xlsx')

In [ ]:
df_shh_ly.来源渠道.unique()

In [ ]:
df_shh_ly_group

In [ ]:
# df_shh_ck = df_shh[df_shh['是否出库']==1]
# df_shh_ck['机型'] = df_shh_ck['product_name'].str.replace(' ', '').str.extract(r'(iPhone ?\d+(ProMax|Pro|Plus)?|小米 ?\d+(Pro)?)')[0]
# # df_ck['机型'].drop_duplicates()
# # df_shh_ck = df_shh[df_shh['']]
# df_shh_ck_group = data_group(df_shh_ck,  '机型')
# df_shh_ck_group = df_shh_ck_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核", '出库']]

In [ ]:
# with pd.ExcelWriter(f'F:/需求/七月需求/生活号_{Today_dw}.xlsx', engine='openpyxl') as writer:
#     df_shh_group.to_excel(writer, sheet_name='生活号渠道')
#     df_shh_ck_group.to_excel(writer, sheet_name='生活号机型')

# vivo 小程序

In [ ]:
# df_vivo = df[df['appid']=='2021004145614858']
# df_vivo2 = df2[df2['appid']=='2021004145614858']
# vivo_month = today.strftime('%m')
# all_models.data_group(df_vivo,df_vivo2,df_risk_examine, '下单日期')[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].to_excel(f'F:/需求/瑞彤需求/vivo小程序{vivo_month}月份数据_{t_date}.xlsx')


## 租售比

In [ ]:
# df_vivo2 = dfck2[dfck2.appid=='2021004145614858']
# df_vivo2_ck = df_vivo2[df_vivo2.是否出库==1]
# # df_vivo2_ck
# df_vivo2_ck.loc[:, '残值估算毛利'] = np.where(df_vivo2_ck.下单月份<='2024-09', df_vivo2_ck.总应收 - df_vivo2_ck.采购金额, df_vivo2_ck.total_receivable - df_vivo2_ck.采购金额)
# df_vivo2_ck.loc[:, '买断估算毛利'] = df_vivo2_ck.contract_price - df_vivo2_ck.采购金额 + df_vivo2_ck.溢价费
# df_vivo2_ck_group = df_vivo2_ck.groupby('下单月份').agg({'contract_price': 'mean', 'all_deposit': 'mean', 'all_rental': 'mean', '采购金额': 'mean', '残值估算毛利': 'mean', '买断估算毛利': 'mean'}).rename(columns={'contract_price': '买断价均值', 'all_rental': '总租金均值', 'all_deposit': '总押金均值', '采购金额': '采购成本均值'})
# df_vivo2_ck_group.loc[:, '租售比'] = (df_vivo2_ck_group.总租金均值/df_vivo2_ck_group.总押金均值).map(lambda x: format(x,".2%"))
# df_vivo2_ck_group.loc[:, '买断估算毛利率'] = (df_vivo2_ck_group.买断估算毛利/df_vivo2_ck_group.采购成本均值).map(lambda x: format(x,".2%"))
# df_vivo2_ck_group.loc[:, '残值估算毛利率'] = (df_vivo2_ck_group.残值估算毛利/df_vivo2_ck_group.采购成本均值).map(lambda x: format(x,".2%"))

# with pd.ExcelWriter(f'F:/需求/瑞彤需求/vivo租售比_{t_date}.xlsx', engine='openpyxl') as writer:
#     df_vivo2_ck_group.to_excel(writer, sheet_name='vivo小程序租售比')

# 512G 带锁和不带锁的进件出库

In [ ]:
# df['product_name'] = df['product_name'].str.replace(' ', '', regex=False)
# df_iPhone = df[df['product_name'].str.contains(r'全新')]
# df_iPhone = df[df['product_name'].str.contains(r'iPhone16|iPhone15')]
# df_iPhone = df_iPhone[df_iPhone['merchant_name']!='瑞和嘉信科技']
# df_iPhone = df_iPhone[(df_iPhone['内存']=='512G')|(df_iPhone['内存']=='512GB')]
# df_iPhone['机型'] = df_iPhone['product_name'].str.extract(r'(iPhone ?\d+(ProMax|Pro|Plus)?)')[0]


In [ ]:
# df_iPhone['下单月份']=df_iPhone['下单日期'].dt.strftime('%Y-%m')
# df_iPhone['机型内存']=df_iPhone['机型']+'_'+df_iPhone['内存']
# # 
# # pd.crosstab(dfck["日期"],dfck['型号内存'],margins=True)
# df_iPhone_group = data_group(df_iPhone, '机型内存')[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# # df_iPhone_group[:5].sum()
# df_iPhone_group

In [ ]:
# max_df_iPhone = df_iPhone[(df_iPhone['max_overdue_days']>=31)&(df_iPhone['机型内存']=='iPhone15Pro_512G')]
# max_df_iPhone.groupby('机型内存').agg({'order_id': 'size'})


In [ ]:
# max_df_iPhone['max_overdue_days'].value_counts()

In [ ]:
# dfck = dfck[dfck['型号'].str.contains(r'iPhone15|iPhone16')]
# dfck =dfck[(dfck['内存_x']=='512G')|(dfck['内存_x']=='512GB')]
# dfck['机型内存'] = dfck['型号']+dfck['内存_x']
# dfck = dfck[~dfck['机型内存'].str.contains(r'99新|95|准新')]
# dfck_ys = dfck[dfck['机型内存'].str.contains(r'无锁')]
# dfck_ys = dfck_ys[~dfck_ys['快递单号'].str.contains(r'取消')]
# dfck_ys.groupby('机型内存').agg({'快递单号': 'count'})

# pd.crosstab(dfck_ys['日期'], dfck_ys['机型内存'],margins=True)

In [ ]:
# df_iPhone['product_name'].value_counts()

# 退款率

## 京东订单

### 客户取消

In [ ]:
# 归属渠道为京东的退款订单且退款原因为客户取消，客户取消状态就为已退款(df['status2']=='已退款')
df_jd = df[(df.归属渠道=='京东渠道')]


df_tkjd_tk = df_jd.groupby('下单月份')[['机审强拒', '人审拒绝', '客户取消', '无法联系', '出库前风控强拒']].sum()

df_tkjd_kh = df_jd[df_jd['客户取消']==1]
df_tkjd_kh['取消原因'] = df_tkjd_kh['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
df_tkjd_kh['取消原因'].drop_duplicates()
df_tkjd_kh_group = df_tkjd_kh.groupby(['下单月份', '取消原因']).agg(取消数=('order_number', 'nunique'))
df_tkjd_kh_group['取消率'] = df_tkjd_kh_group['取消数']/df_tkjd_tk['客户取消'].sum()
df_tkjd_kh_group['取消率'] = df_tkjd_kh_group['取消率'].apply(lambda x: format(x, '.2%'))
df_tkjd_kh_group#.to_excel(f'F:/需求/瑞彤需求/客户取消_月份_{t_date}.xjdx')

In [ ]:
df.columns

### 机审强拒,人审拒绝

In [ ]:
# df_jd = df[(df.归属渠道=='京东渠道')]
df_jdqj_group = df_jd.groupby('下单月份').agg({'机审强拒':'sum', '人审拒绝':'sum', '是否进件':'sum'}).rename(columns={'是否进件':'进件数'})
df_jdqj_group['机审拒绝率'] = df_jdqj_group['机审强拒']/df_jdqj_group['进件数']
df_jdqj_group['机审拒绝率'] = df_jdqj_group['机审拒绝率'].apply(lambda x: format(x, ".2%"))
df_jdqj_group['人审拒绝率'] = df_jdqj_group['人审拒绝']/df_jdqj_group['进件数']
df_jdqj_group['人审拒绝率'] = df_jdqj_group['人审拒绝率'].apply(lambda x: format(x, ".2%"))
df_jdqj_group

### 审核通过取消订单

In [ ]:
# 审核通过订单且订单状态为已退款(df['status2']=='已退款')&(df['风控通过件']==1)
df_jd_shtg = df_jd#[(df_jd['风控通过件']==1)&(df_jd['status2']=='已退款')]
df_jd_shtg.loc[:,'发货前取消'] = np.where((df_jd['status2']=='已退款')&(df_jd['风控通过件']==1)&(df_jd['出库前风控强拒']==0),1,0)
# 物流状态=5，退货拒签
# df.loc[:,"退货拒签"] = np.where(df['物流状态']==5, 1, 0)
df_jd_shtg.loc[:,'发货拒签'] = np.where((df_jd['物流状态']==5)&(df_jd['是否出库']==1),1,0)
df_jd_shtg_group = df_jd_shtg.groupby('下单月份').agg({'风控通过件':'sum','发货前取消':'sum', '发货拒签':'sum', '是否进件':'sum'})#.rename(columns={'发货前取消':'发货前取消数', '发货拒签':'发货拒签数'})
df_jd_shtg_group['发货前取消率'] = (df_jd_shtg_group['发货前取消']/df_jd_shtg_group['是否进件']).apply(lambda x: format(x, ".2%"))
df_jd_shtg_group['发货拒签率'] = (df_jd_shtg_group['发货拒签']/df_jd_shtg_group['是否进件']).apply(lambda x: format(x, ".2%"))
df_jd_shtg_group

In [ ]:
# 写入Excel
with pd.ExcelWriter('F:/需求/瑞彤需求/京东取消订单_月份_{t_date}.xlsx', engine='openpyxl') as writer:
    df_tkjd_kh_group.to_excel(writer, sheet_name='客户取消')
    df_jdqj_group.to_excel(writer, sheet_name='机审人审拒绝')
    df_jd_shtg_group.to_excel(writer, sheet_name='审核通过取消订单')

In [ ]:
# 归属渠道为芝麻租物的退款订单
# df_tkl = df[(df['status2']=='已退款')&(df.归属渠道=='芝麻租物')][['order_number', '下单月份']]
# df_tk_group = df_tkl.groupby('下单月份').agg(退款数=('order_number', 'nunique'))
# df_tk_group.rename(columns={'user_id': '退款数'}, inplace=True)

In [ ]:
# 总体退款订单
# df_tkls = df[(df['status2']=='已退款')] # &(df.归属渠道=='芝麻租物')
# df_tkls_tk = df_tkls.groupby('下单月份')[['机审强拒', '人审拒绝', '客户取消', '无法联系', '出库前风控强拒']].sum()


In [ ]:
# 退款订单且退款原因为客户取消
# df_tkls_kh = df_tkls[(df_tkls['客户取消']==1)]
# df_tkls_kh['取消原因'] = df_tkls_kh['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_tkls_kh['取消原因'].drop_duplicates()
# df_tkls_kh_group = df_tkls_kh.groupby(['下单月份', '取消原因']).agg(取消数=('order_number', 'nunique'))
# df_tkls_kh_group['取消率'] = df_tkls_kh_group['取消数']/df_tkls_tk['客户取消'].sum()
# df_tkls_kh_group['取消率'] = df_tkls_kh_group['取消率'].apply(lambda x: format(x, '.2%'))
# df_tkls_kh_group.to_excel(f'F:/需求/瑞彤需求/已退款取消率_月份_{t_date}.xlsx')

In [ ]:
# df_tkls_kh[df_tkls_kh.取消原因.isin(['不同意多预收租金', '租金过高/租金不合理', '不同意预收首期租金'])][['order_number', '取消原因']]#.to_excel('F:/需求/总需求/取消原因订单.xlsx', index=False)

In [ ]:
# df_ysql = df[(df['dy_order_item_json'].isna())&(~df['status'].isin([1, 13]))&(df['type']!=3)][['order_number', '下单月份']]
# df_ysq_group = df_ysql.groupby('下单月份').agg(预授权数=('order_number', 'nunique'))
# df_ysq_group.rename(columns={'user_id': '预授权数'}, inplace=True)

In [ ]:
# df_all_month = pd.merge(df_tk_group, df_ysq_group, on='下单月份',how='inner')
# df_all_month['退款率'] = df_all_month['退款数'] / df_all_month['预授权数']
# df_all_month['退款率'] = df_all_month['退款率'].apply(lambda x: format(x, '.2%'))
# pd.merge(df_tkls_tk, df_all_month, on='下单月份', how='inner')#.to_excel('F:/需求/瑞彤需求/退款分布.xlsx')

# 九州信息

In [ ]:
# df_jz = df[(df['归属渠道']=='九州信息')&(df['status2']=='租赁中')]
# df_jz[['create_date','order_number','status2','product_name','new_actual_money','back_type','true_name','channel_name','activity_name']]

In [ ]:
# df_jz1 = df[df['归属渠道']=='九州信息']
# df_jz2 = df2[df2['归属渠道']=='九州信息']
# df_jz_group = all_models.data_group(df_jz1, df_jz2, df_risk_examine, '下单日期')
# df_jz_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].to_excel(f'F:/需求/七月需求/九州信息转化_{Today_dw}.xlsx')

In [ ]:
# df_jz = df[df['归属渠道']=='八派信息']
# df_jz_group = data_group(df_jz, '下单日期')
# df_jz_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].to_excel('F:/需求/七月需求/八派信息转化.xlsx')

In [ ]:
# df_contain[df_contain["merchant_name"]=='小蚂蚁租机'][['create_time','order_id','是否进件','前置拦截','机审强拒', '进件前取消'
#                                         ,'人审拒绝','客户取消','无法联系', '出库前风控强拒','待审核', '是否出库', 'status2']].to_excel('F:/需求/小蚂蚁.xlsx')

# 各渠道排名

In [ ]:
# df_channel = all_models.data_group(df, df2, df_risk_examine, '归属渠道')
# df_channel_group = df_channel.sort_values('进件数', ascending=False).head(5)
# df_channel_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].to_excel('F:/需求/七月需求/进件前五渠道转化.xlsx')

In [ ]:
# df_channel = all_models.data_group(df, df2, df_risk_examine, ['归属渠道','下单日期'])
# df_channel_g = df_channel.groupby('归属渠道')[['进件数']].cumsum()
# df_channel_g = df_channel_g.groupby('归属渠道')[['进件数']].last().sort_values('进件数', ascending=False).head(5)
# df_channel_g.rename(columns={'进件数': '渠道总进件数'}, inplace=True)
# df_channel = df_channel.reset_index().set_index('归属渠道')
# df_channel_group = df_channel.merge(df_channel_g, on='归属渠道', how='inner').sort_values([ '渠道总进件数','归属渠道', '下单日期'], ascending=[False,True, True])
# df_channel_group
# df_channel_group[['下单日期', "去重订单数","前置拦截","拦截率","进件数",'渠道总进件数',"预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].to_excel(f'F:/需求/七月需求/进件前五渠道转化_{Today_zfb}.xlsx')

## 归属渠道出库前五

In [ ]:
df_channel = all_models.data_group(df, df2, df_risk_examine, '归属渠道')
df_channel_group = df_channel.sort_values('出库', ascending=False).head(5)
channel_top5 = df_channel_group.index.to_list()
channel_top


In [ ]:
# 找出出库量前五的渠道
# df2.purchase_amount>0存在订单取消，拒收等情况
# df = df.sort_values(by='出库', ascenging=False).groupby('归属渠道').head(5) 
# df.loc[:, '买断金额'] = df.new_actual_money-df.discount_money
df_new = df2[(df2.是否出库==1)&(df2.归属渠道.isin(channel_top5))&(df2.purchase_amount>0)].groupby('归属渠道').agg({'purchase_amount': 'sum', 'all_rental': 'sum',  'new_actual_money': 'sum'}).rename(columns={'purchase_amount': '采购价', 'all_rental': '总租金', 'new_actual_money': '签约价'})
df_new

In [ ]:
df_new.loc[:, '毛利率'] = ((df_new.签约价-df_new.采购价)/df_new.采购价).map(lambda x: format(x, '.2%'))
df_new.loc[:, '买断尾款'] = (df_new.签约价-df_new.总租金).map(lambda x: format(x, '.2f'))
df_new



# 二手手机打上标签

In [ ]:
id_list = [18, 21, 23, 30, 31, 33, 36, 51, 79, 85, 99, 107, 123, 124, 125, 132, 141, 149]
# df[(df.是否二手==1)&(df['归属渠道']!='抖音渠道')&(df.是否进件==1)&(df.下单月份=='2024-06')&(~df.merchant_id_t.isin(id_list))][['order_number', '是否进件', 'product_name', '是否二手', 'status2', 'merchant_name']]

In [ ]:
# df_2_mobile = df[(df.是否二手==1)&(df['归属渠道']!='抖音渠道')&(~df.merchant_id_t.isin(id_list))]
# df_2_mobile2 = df2[(df2.是否二手==1)&(df2['归属渠道']!='抖音渠道')&(~df.merchant_id_t.isin(id_list))]
# df_2_mobile_group = all_models.data_group(df_2_mobile, df_2_mobile2, df_risk_examine, '下单月份') classify_id
# df_2_mobile_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].to_excel(f'F:/需求/七月需求/二手手机数据1_月_{t_date}.xlsx')
# df_2_mobile_group2 = all_models.data_group(df_2_mobile, df_2_mobile2, df_risk_examine, '下单日期')
# df_2_mobile_group2[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].to_excel('F:/需求/七月需求/二手手机数据_日.xlsx')

In [ ]:
# df_2_mobile2['取消原因2'] = df_2_mobile2.取消原因#.str.split('审核结果：').str[-1]#.str.split(',').str[0]
# # pd.pivot_table(df_2_mobile2, values='order_id', index='下单月份', columns='取消原因2', aggfunc='count').to_excel('F:/需求/七月需求/二手取消原因.xlsx')
# df_2_mobile2[((df_2_mobile2.客户取消==1)|(df_2_mobile2.已退款==1))&(df_2_mobile2.下单月份>='2024-11')][['order_number', '取消原因2']].to_excel('F:/需求/七月需求/取消订单.xlsx', index=False)


In [ ]:
# df_2_mobile2_rs = df_2_mobile2[df_2_mobile2.人审拒绝==1]
# pd.pivot_table(df_2_mobile2_rs, values='order_id', index='下单月份', columns='电审拒绝原因', aggfunc='count').to_excel('F:/需求/七月需求/人审拒绝原因.xlsx')
# pd.crosstab(df_2_mobile2_rs.下单月份, df_2_mobile2_rs.电审拒绝原因, margins=True)

In [ ]:
# df_2_mobile2_ck = df_2_mobile2[df_2_mobile2.出库前风控强拒==1]
# df_2_mobile2_ck[df_2_mobile2_ck.拒绝理由==''][['order_number', '拒绝理由']]
# pd.pivot_table(df_2_mobile2_ck, values='order_id', index='下单月份', columns='拒绝理由', aggfunc='count')#.to_excel('F:/需求/七月需求/出库前风控强拒.xlsx')

In [ ]:
# 优优商家，使用merchant_name字段
# df_contain_yy = df_contain[df_contain['merchant_name'].str.contains('优优')]
# df_contain_yy_group = df_contain_yy.groupby('下单月份').agg({'是否进件': 'sum', '是否出库': 'sum'})
# df_contain_yy_group.rename(columns={'是否进件': '进件', '是否出库': '出库'}, inplace=True)
# df_contain_yy_group['进件出库率'] = df_contain_yy_group['出库'] / df_contain_yy_group['进件']
# df_contain_yy_group['进件出库率'] = df_contain_yy_group['进件出库率'].apply(lambda x: format(x, '.2%'))
# with pd.ExcelWriter('F:/需求/二手手机/二手手机进件出库.xlsx', engine='openpyxl') as writer:
#     df_contain_yy_group.to_excel(writer, sheet_name='优优商家')

In [ ]:
# 带锁手机，只有512内存的才会上锁，数据取自台账
# df_group = df.groupby('下单月份').agg({'是否进件': 'sum'})
# df_group.rename(columns={'是否进件': '自营进件'}, inplace=True)

In [ ]:
# dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存_x'].str.contains(r'512')), '带锁', '未带锁')
# dfck_group = pd.crosstab(dfck['下单月份'], dfck['是否带锁'], margins=True)
# df_ck_group = df_group.merge(dfck_group, on='下单月份', how='inner')
# df_ck_group['带锁出库率'] = df_ck_group['带锁']/df_ck_group['自营进件']
# df_ck_group['带锁出库率'] = df_ck_group['带锁出库率'].apply(lambda x: format(x, '.2%'))
# df_ck_group['未带锁出库率'] = df_ck_group['未带锁']/df_ck_group['自营进件']
# df_ck_group['未带锁出库率'] = df_ck_group['未带锁出库率'].apply(lambda x: format(x, '.2%'))
# df_ck_group.drop('All', axis=1, inplace=True)
# with pd.ExcelWriter('F:/需求/二手手机/二手手机进件出库.xlsx', engine='openpyxl', mode='a') as writer:
#     df_ck_group.to_excel(writer, sheet_name='自营')

In [ ]:
# dfck.groupby('下单月份').agg({'是否带锁': 'sum'})#to_excel('F:/需求/总需求/512带锁.xlsx')

# 匹配名单（薪华需求）

In [ ]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date
df_ml = pd.read_excel('F:/需求/薪华需求/8月总买断毛利率.xls', sheet_name='Sheet1')
df_ml_list = tuple(df_ml.订单号.to_list())
sql_ml = f'''
select distinct
tprm.order_number, tod.new_actual_money, toou.discount_money, tprm.purchase_amount
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order_details tod on tod.order_id=tprm.order_id
left join db_digua_business.t_order_out toou on toou.order_id=tprm.order_id
where tprm.order_number in {df_ml_list}
'''
df_ml_merge = clean.query(sql_ml)
df_ml_merge = df_ml_merge.sort_values(by=['order_number', 'discount_money'], ascending=False).groupby('order_number').head(1)
df_ml_merge = df_ml_merge.set_index('order_number')
df_ml_merge.loc['合计', :] = df_ml_merge.sum(axis=0)
df_ml_merge = df_ml_merge.reset_index()
df_ml_merge.loc[:, '买断金额'] = df_ml_merge.new_actual_money-df_ml_merge.discount_money
df_ml_merge.loc[:, '毛利率'] = ((df_ml_merge.买断金额-df_ml_merge.purchase_amount)/df_ml_merge.purchase_amount).map(lambda x: format(x, '.2%'))
df_ml_merge = df_ml_merge.rename(columns={'order_number': '订单号', 'new_actual_money': '买断价', 'discount_money': '优惠金额', 'purchase_amount': '采购金额'})
df_ml_merge.to_excel(f'F:/需求/薪华需求/毛利率_{t_date}.xlsx', index=False)

In [ ]:
# order = pd.read_excel('F:/需求/白名单/白名单总数据(3).xlsx')
# order = pd.read_excel('F:/需求/白名单/10月30~12月3号.xlsx')
# order['手机号码'] = order['手机号码'].astype(str)
# order = order[order['订单状态'].isna()]
# df_merge = order.merge(df[['create_time', 'order_number', 'user_mobile', 'status2']], left_on='手机号码', right_on='user_mobile', how='inner')

# df_order = df[(df['下单日期']>=(pd.Timestamp.now() - timedelta(days=8)))&(df['下单日期']<pd.Timestamp.now().strftime('%Y-%m-%d'))]
# df_xd = df_merge.groupby(['user_mobile']).agg({'order_number': 'count'})
# df_xd.rename(columns={'order_number': '下单次数'}, inplace=True)
# df_orders = pd.merge(df_merge, df_xd, on='user_mobile', how='inner')

# df_max = df_orders.groupby('user_mobile').agg({'create_time': 'max'})
# df_max.rename(columns={'create_time': 'max_create_time'}, inplace=True)

# df_orders = pd.merge(df_orders, df_max, on='user_mobile', how='inner')
# df_over = df_orders[df_orders['create_time']==df_orders['max_create_time']][['order_number', '下单次数', 'status2', 'user_mobile', 'max_create_time']]
# df_over.rename(columns={'order_number': '订单号', 'status2': '最新订单状态', 'max_create_time': '最新下单时间'}, inplace=True)

# with pd.ExcelWriter(f'F:/需求/薪华需求/匹配名单(10月30日至今)_{Today_zfb}.xlsx', engine='openpyxl') as writer:
#     df_over.to_excel(writer, sheet_name='匹配名单', index=False)

In [ ]:
# df_jjjl = pd.read_excel('F:/需求/薪华需求/进件记录.xlsx', sheet_name='Sheet2')
# df_jjjl_list = tuple(df_jjjl.user_mobile.to_list())
# sql_xd = f'''
# select
# order_number, user_mobile
# from db_digua_business.t_order 
# where create_time>=date_add(current_date, interval -13 day) and user_mobile in {df_jjjl_list}
# '''
# df_xd = query(sql_xd)
# df_xd

## 白名单转化

In [ ]:
# df_mobile = pd.read_excel('F:/需求/薪华需求/匹配名单28号.xlsx')
# df_mobile['user_mobile'] = df_mobile.user_mobile.astype('object')
# df_mobile_new = df.merge(df_mobile, on='user_mobile', how='inner')
# df_mobile_new[['user_mobile', 'true_name']]

In [ ]:
# 读取数据
# white_order1 = pd.read_excel('F:/需求/白名单/白名单总数据(1).xlsx')[['手机号','跟进人', '回访结果']]
# white_order2 = pd.read_excel('F:/需求/白名单/新白名单(1).xlsx')[['手机号','跟进人', '回访结果']]
# white_order = pd.concat([white_order1,white_order2], axis=0)
# white_order_new = white_order[white_order.跟进人.notna()]
# white_order_new = white_order_new[~white_order_new.回访结果.isin(['错误名单','错误订单','黑名单','强制买断钟','续租预期','已逾期'])]
# white_order_new['手机号'] = white_order_new['手机号'].astype(str)


In [ ]:
# df_white_new = df[df.user_mobile.isin(white_order_new.手机号.to_list())]

# df_white_new2 = df_white_new.sort_values('create_time', ascending=True).groupby('user_mobile').head(1)
# df_white_new2[['order_number', 'status2', 'user_mobile', 'create_time']].to_excel(f'F:/需求/薪华需求/匹配名单(10月30日至今)_{Today_zfb}.xlsx', sheet_name='匹配名单1', index=False)

In [ ]:
# list_mobile = [17795333204,13390520602,15678212129,13528734795,13199680013,13546603049,17345376002,13277830591,18223350117,19131602229,18364946456,13142663031,17603344066,18666275945,18706195037,13357983539,15308430470,13502672866,13305102397,15592227677,15685857792,18389690210,19524780543,18818558605,13826789364,19532635296,15066955097,15669222020,16637085587,18035119899,13066818128]


In [ ]:
# error_df = pd.read_excel('F:/需求/白名单/白名单呼叫申请明细清单_mine.xlsx', sheet_name='错误名单')
# error_df['手机号码'] = error_df['手机号码'].astype(str)
# df_white_new_o = df_white_new[df_white_new.user_mobile.isin(error_df.手机号码.to_list())]
# df_white_new_o.drop_duplicates(subset=['user_mobile'], inplace=True)
# df_white_new_o[df_white_new_o.是否前置拦截==1]

In [ ]:
# df_white_new2 = df_white_new2[~df_white_new2.user_mobile.astype(int).isin(list_mobile)]

In [ ]:
# df_white_new2[['create_time', 'order_number', 'user_mobile', 'id_card_num', '机审通过件', '是否出库']].to_excel(f'F:/需求/总需求/总体明细_{t_date}.xlsx', index=False)

In [ ]:
# df_white_new2_group = df_white_new2.groupby('下单月份').agg(数量=('order_number', 'count'),前置拦截=('是否前置拦截', 'sum'), 进件=('是否进件', 'sum'), 机审强拒=('是否机审强拒', 'sum'), 出库前强拒=('出库前风控强拒', 'sum'), 出库=('是否出库', 'sum'))
# df_white_new2_risk = risk_data.pre_reject(df_white_new2, '下单日期')
# with pd.ExcelWriter(f'F:/需求/白名单/白名单月数量_{t_date}.xlsx', engine='openpyxl') as writer:
#     df_white_new2_risk.to_excel(writer, sheet_name='前置拦截', index=False)
#     df_white_new2_group.to_excel(writer, sheet_name='月明细')

In [ ]:
# df_white_new2_jj = df_white_new2[df_white_new2.是否进件==1]
# df_white_new2_jj_detail = df_white_new2_jj[['下单日期', 'order_number', 'user_mobile', '拒绝理由', 'total_describes', 'result']]
# df_white_new2_jj_group = risk_data.model_reject(df_white_new2_jj, '下单日期')#.to_excel('F:/需求/白名单/白名单强拒.xlsx')

In [ ]:
# with pd.ExcelWriter('F:/需求/白名单/白名单强拒.xlsx', engine='openpyxl') as writer:
#     df_white_new2_jj_group.to_excel(writer, sheet_name='白名单强拒汇总', index=False)
#     df_white_new2_jj_detail.to_excel(writer, sheet_name='白名单强拒明细', index=False)

# 发货需上锁(多头)

In [ ]:
# # 获取时间段
# df_27 = df[df['下单日期']<='2024-10-27']
# df_272 = df2[df2['下单日期']<='2024-10-27']
# # 获取tips不为空且匹配到发货需上锁的数据
# df_tips = df_27[df_27['tips'].notna()]
# df_tips2 = df_272[df_272['tips'].notna()]
# 
# df_tips_ss = df_tips[df_tips['tips'].str.contains('发货需上锁')]
# # 排除掉有豁免的数据
# df_tips_ss = df_tips_ss[~df_tips_ss['tips'].str.contains('豁免')]
# df_tips_ss2 = df_tips2[(df_tips2['tips'].str.contains('发货需上锁'))]
# df_tips_ss2 = df_tips_ss2[~df_tips_ss2['tips'].str.contains('豁免')]
# df_tips_ss_group = all_models.data_group(df_tips_ss, df_tips_ss2, df_risk_examine, '下单日期')[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]

In [ ]:
# df_28 = df[df['下单日期']>'2024-10-27']
# df_282 = df2[df2['下单日期']>'2024-10-27']
# # 获取tips不为空且匹配到发货需上锁的数据
# df_tips28 = df_28[df_28['tips'].notna()]
# df_tips228 = df_282[df_282['tips'].notna()]

# df_tips_ss28 = df_tips28[df_tips28['tips'].str.contains('(策略241027豁免)')]
# df_tips_ss228 = df_tips228[(df_tips228['tips'].str.contains('(策略241027豁免)'))]
# df_tips_ss_group2 = all_models.data_group(df_tips_ss28, df_tips_ss228, df_risk_examine, '下单月份')[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# df_tips_ss_group2.to_excel('F:/需求/七月需求/多头转化.xlsx')

In [ ]:
# 命中多头
# df_tips228_new = df_tips228[(df_tips228.是否出库==1)&(df_tips228['tips'].str.contains('发货需上锁(策略241027豁免)', regex=False))]
# # df_tips228_new.loc[:, '命中多头'] = np.where(df_tips228_new.tips.str.contains('发货需上锁(策略241027豁免)', regex=False),1,0)
# df_tips228_new[['create_time', 'order_number', 'tips']].to_excel(f'F:/需求/总需求/总体命中多头_{t_date}.xlsx')

# df_tips228_new.groupby('下单日期').agg({'是否出库': 'sum', '命中多头': 'sum'})#.to_excel('F:/需求/总需求/单人回话命中多头.xlsx')

## 单人会话发货需上锁

In [ ]:
# df_dr_11 = df_dr[(df_dr.下单日期>='2024-11-01')&(df_dr.tips.notna())&(df_dr.是否出库==1)]
# df_dr_11.loc[:, '命中多头'] = np.where(df_dr_11.tips.str.contains('发货需上锁(策略241027豁免)'),1,0)
# df_dr2_11 = df_dr2[(df_dr2.下单日期>='2024-11-01')&(df_dr2.tips.notna())&(df_dr.是否出库==1)&(~df_dr2.tips.str.contains('(策略241027豁免)'))]
# all_models.data_group(df_dr_11, df_dr2_11, df_risk_examine, '下单日期')[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]


In [ ]:
# df_dr_11.groupby('下单日期').agg({'是否出库': 'sum', '命中多头': 'sum'}).to_excel('F:/需求/总需求/单人会话命中多头.xlsx')
# df_dr_11_new_detail = df_dr_11[['create_time', 'order_number', 'tips']]

# 自然周周数据保存

In [ ]:
# with pd.ExcelWriter('F:/需求/七月需求/渠道8月份至今数据转化.xlsx', engine='openpyxl') as writer:
    # week_ss_group.to_excel(writer, sheet_name='搜索渠道周转化数据', index=False)
    # week_zm_group.to_excel(writer, sheet_name='芝麻租物周转化数据', index=False)
    # week_dr_group.to_excel(writer, sheet_name='单人会话周转化数据', index=False)
    # df_ss_groups.to_excel(writer, sheet_name='搜索渠道每日转化数据')
    # df_dr_groups.to_excel(writer, sheet_name='单人会话每日转化数据')
    # df_zm_groups.to_excel(writer, sheet_name='芝麻租物每日转化数据')

# 数立月活

In [ ]:
# df_sl1 = df[df['归属渠道'].str.contains('数立月活')]
# df_sl2 = df2[df2['归属渠道'].str.contains('数立月活')]
# 
# df_sl_all = all_models.data_group(df_sl1, df_sl2, df_risk_examine, ['下单日期', '归属渠道'])
# df_sl_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].to_excel('F:/需求/丹琦需求/数立月活.xlsx')

# 二手手机保存

In [ ]:
# with pd.ExcelWriter('F:/宏生/二手手机对比.xlsx', engine='openpyxl') as writer:
#     df_ss_2_group.to_excel(writer, sheet_name='搜索渠道二手手机')
#     df_zm_2_group.to_excel(writer, sheet_name='芝麻租物二手手机')

# 迪瓜小程序导出

In [ ]:
# df_digua = df[df.appid=='2021003131606511']
# df_jj_digua = df_j[df_j.appid=='2021003131606511']
# dfck_digua = dfck[dfck.appid=='2021003131606511']
# #每日各渠道去重订单统计结果
# gsqd_digua_qcdd = pd.crosstab(df_digua["下单日期"],df_digua["归属渠道"],margins=True)
# lyqd_digua_qcdd = pd.crosstab(df_digua["下单日期"],df_digua["来源渠道"],margins=True)
# #每日各渠道进件统计结果表格导出
# gsqd_digua_jj = pd.crosstab(df_jj_digua["下单日期"],df_jj_digua["归属渠道"],margins=True)
# lyqd_digua_jj = pd.crosstab(df_jj_digua["下单日期"],df_jj_digua["来源渠道"],margins=True)
# #每日各渠道进件统计结果表格导出
# gsqd_digua_ck = pd.crosstab(dfck_digua["下单日期"],dfck_digua["归属渠道"],margins=True)
# lyqd_digua_ck = pd.crosstab(dfck_digua["下单日期"],dfck_digua["来源渠道"],margins=True)
# 
# with pd.ExcelWriter(f'F:/需求/瑞彤需求/迪瓜相关数据输出_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     gsqd_digua_qcdd.to_excel(writer, sheet_name='归属渠道去重订单统计', index=True)
#     lyqd_digua_qcdd.to_excel(writer, sheet_name='来源渠道去重订单统计', index=True)
#     gsqd_digua_jj.to_excel(writer, sheet_name='归属渠道进件统计', index=True) 
#     lyqd_digua_jj.to_excel(writer, sheet_name='来源渠道进件统计', index=True)
#     gsqd_digua_ck.to_excel(writer, sheet_name='归属渠道出库统计', index=True)
#     lyqd_digua_ck.to_excel(writer, sheet_name='来源渠道出库统计', index=True)

# 进入拒量池订单

In [ ]:
df_jl = df[df.tips.str.contains(r'策略2412|命中自有模型策略|回捞策略250330命中')==True]
# df_jl.loc[:, '买断尾款'] = df_jl.new_actual_money-df_jl.all_money
# df_jl_new = df_j
# df_jl_new['策略命中等级'] = df_jl_new['tips'].str.extract(r'(策略241205命中\(\d+\))')[0]
# df_jl_ck = df_jl_new[df_jl_new.是否出库==1]
# df_ck_241205 = df_jl_ck.merge(df_ck, left_on='order_number', right_on='订单号', how='left')[['订单号', '型号', '期数', '策略命中等级','溢价费']]
# df_ck_241205['期数'] = df_ck_241205.期数.astype(int)
# df_jl

In [ ]:
# df_jl_spx = df_jl[(df_jl.碎屏险价格==0)&(df_jl.是否出库==1)]
# df_jl_spx = df_jl[(df_jl.是否出库==1)&(df_jl.service_status==1)]
# with pd.ExcelWriter(f'F:/需求/总需求/拒量订单_{t_date}.xlsx', engine='openpyxl') as writer:
#     df_jl_spx['order_number'].to_excel(writer, sheet_name='未购买碎屏险', index=False)
# df_jl_zl = df_jl_new[(df_jl_new.租赁方案=='租完归还')&(df_jl.是否出库==1)]

# with pd.ExcelWriter('F:/需求/总需求/租完归还订单.xlsx', engine='openpyxl') as writer: 
#     df_jl_zl[['order_number', 'auto_pay_tag', '总差异化定价优惠金额', '策略命中等级']].to_excel(writer, sheet_name='租完归还', index=False)

In [ ]:
# df_ck_241205.loc[:, '应用策略'] = np.where(df_ck_241205.期数==3, 
#                                            np.where(df_ck_241205.型号.str.contains('无锁'), '正常预收规则下再加收1期', '锁机+正常预收规则下再加收1期'), np.where(df_ck_241205.型号.str.contains('无锁'), '未应用策略', '锁机'))
# df_ck_241205

In [ ]:
# sql_ya = f'''
# select
# ymos.order_id, ymos.money, ymos.part_payment, ymos.refund_date, ymos.reality_refund_date, ymos.sort
# from db_rent.ya_merchant_order_stages ymos
# '''
# df_yas = query(sql_ya)

In [ ]:
# # 获取拒量数据
# df_yas.loc[:, '实还日期'] = df_yas.reality_refund_date.dt.date
# # 获取第一期的实还日期时间
# df_yas.loc[:, 'sort1_date'] = df_yas[df_yas.sort==1]['实还日期']
# df_yas.loc[:, 'sort1_date'] = df_yas.sort1_date.ffill()
# # 给需要的字段打上标签
# df_yas.loc[:, '实还期数'] = np.where(df_yas.实还日期==df_yas.sort1_date, 1, 0)
# df_yas.loc[:, '实收金额'] = np.where(df_yas.实还期数==1, df_yas.part_payment, 0)
# df_yas.loc[:, '剩余每月租金'] = np.where(df_yas.reality_refund_date.isna(), df_yas.money, 0)
# df_yas.loc[:, '剩余期数'] = np.where(df_yas.reality_refund_date.isna(), 1, 0)
# # 根据订单id进行分组汇总
# df_yas_group = df_yas.groupby('order_id').agg({'实还期数': 'sum', '实收金额': 'sum', '剩余每月租金': 'median', '剩余期数': 'sum'})
# # 关联拒量数据
# df_jl_merge = df_jl.merge(df_yas_group, on='order_id', how='left')
# # 更新剩余每月租金和剩余期数
# df_jl_merge.loc[:, '剩余每月租金'] = np.where((df_jl_merge.是否出库==0)|(df_jl_merge.purchase_amount==0), 0, df_jl_merge.剩余每月租金)
# df_jl_merge.loc[:, '剩余期数'] = np.where((df_jl_merge.是否出库==0)|(df_jl_merge.purchase_amount==0), 0, df_jl_merge.剩余期数)
# df_jl_merge_new = df_jl_merge[['下单日期', 'order_number', '是否出库', '租赁方案', '碎屏险价格', 'purchase_amount', 'new_actual_money', '买断尾款', 'status2', '实还期数', '实收金额', '剩余每月租金', '剩余期数']]
# with pd.ExcelWriter(f'F:/需求/总需求/拒量数据明细_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_jl_merge_new.to_excel(writer, sheet_name='拒量明细',  index=False)

## R0拒量

In [ ]:
# df_merge = df.merge(df_rule_rent, on='order_id', how='left')
# df_merge.loc[:, '机审强拒_R0'] = np.where((df_merge.机审强拒==1)&(df_merge.rentOnlineDetailLevelOverall=='0'), 1, 0)
# df_merge_g = df_merge.groupby('下单日期').agg({'机审强拒_R0': 'sum'})
# df_jl_merge = df_jl.merge(df_rule_rent, on='order_id', how='left')
# df_jl_merge = df_jl_merge[df_jl_merge.rentOnlineDetailLevelOverall=='0']
# # df_jl_merge_g = df_jl_merge.groupby('下单日期').agg({'机审通过件': 'sum','风控通过件': 'sum', '客户取消': 'sum', '已退款':'sum', '无法联系': 'sum', '出库前风控强拒': 'sum', '待审核': 'sum', '是否出库': 'sum', 'new_actual_money': 'sum'}).rename(columns={'是否出库': '出库'})
# df_jl_merge_g = df_jl_merge.groupby('下单日期').agg({'机审通过件': 'sum','风控通过件': 'sum', '客户取消': 'sum', '已退款':'sum', '无法联系': 'sum', '出库前风控强拒': 'sum', '待审核': 'sum', '是否出库': 'sum'}).rename(columns={'是否出库': '出库'})

# df_jl_merge_g_1 = df_jl_merge[df_jl_merge.是否出库 == 1].groupby('下单日期').agg({'new_actual_money': 'sum'})

# df_merge_R0 = df_merge_g.merge(df_jl_merge_g, on='下单日期', how='inner')
# df_merge_R0 = df_merge_R0.merge(df_jl_merge_g_1, on='下单日期', how='inner')

# df_merge_R0.loc[:, '机审强拒_R0'] = df_merge_R0.机审强拒_R0+df_merge_R0.机审通过件
# df_merge_R0.loc[:, '客户取消'] = df_merge_R0.客户取消+df_merge_R0.已退款    
# df_merge_R0.loc[:, '出库前风控强拒'] = df_merge_R0.出库前风控强拒+df_merge_R0.无法联系                                      
# # df_merge_R0[['机审强拒_R0', '风控通过件', '客户取消', '出库前风控强拒', '待审核', '出库', 'new_actual_money']]
# df_merge_R0[['机审强拒_R0', '风控通过件', '客户取消', '待审核', '出库', '出库前风控强拒', 'new_actual_money']]

In [ ]:
# df_merge_R0 = df_merge[(df_merge.rentOnlineDetailLevelOverall=='0')&((df_merge.机审强拒==1)|(df_merge.merchant_name.isin(['小蚂蚁租机', '兴鑫兴通讯', '人人享租']))|(df_merge.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True))]
# df_merge_R0.loc[:, '小程序名称'] = '迪瓜租机'   
# df_merge_R0.loc[:, '租期类型'] = '长租'
# df_merge_R0.loc[:, '商品类型'] = np.where(df_merge_R0.是否二手==1, '二手', '全新')  
# df_merge_R0.loc[:, '拒绝理由'] = np.where(df_merge_R0.拒绝理由=='', '', '风控拒绝')

# df_merge_R0['下单日期'] = df_merge_R0['下单日期'].astype('str')
# conditions_1 = ['命中蚂蚁数控风险等级=996强拒','命中蚂蚁数控风险等级=998强拒','蚂蚁数控风险等级=979强拒','命中高风险渠道','命中高风险地址']
# df_merge_R0['拒绝理由'] = np.where(df_merge_R0.取消原因2.isin(conditions_1), '风控拒绝', df_merge_R0.拒绝理由)
# df_merge_R0['取消原因2'] = np.where(df_merge_R0.取消原因2.isin(conditions_1), np.nan, df_merge_R0.取消原因2)  
# df_merge_R0['取消原因2'] = np.where(df_merge_R0.取消原因2.str.contains('曙光')|df_merge_R0.取消原因2.str.contains('旭日'), np.nan, df_merge_R0.取消原因2)

# # df_merge_R0[['下单日期','order_id', 'appid', '小程序名称', '商品类型', '租期类型', 'zfb_open_id', 'alipay_order_id', 'out_order_no', '取消原因2', '拒绝理由']]#.to_excel('F:/R0.xlsx', index=False)
# # df_merge_R0.appid.drop_duplicates()

# df_merge_R0[['下单日期', 'appid', '小程序名称', '商品类型', '租期类型', 'zfb_open_id', 'alipay_order_id', 'out_order_no', 'new_actual_money','取消原因2', '拒绝理由']].to_excel('F:/R0.xlsx', index=False)


# 512G转256G分析（24年6月份上线）

In [ ]:
# df2_256 = df2[(df2.是否出库==1)&(df2.内存.str.contains('256', regex=False))][['下单日期', 'order_number','id_card_num', 'user_mobile', '内存']]
# df2_256.rename(columns={'下单日期': '下单日期_256', 'order_number': 'order_number_256', 'user_mobile': 'user_mobile_256', '内存':'内存_256'}, inplace=True)
# df2_512 = df2[(df2.内存.notna())&(df2.内存.str.contains('512', regex=False))][['create_time', '下单日期', 'order_number', 'id_card_num', 'user_mobile', '内存']]
# df2_512.rename(columns={'create_time': 'create_time_512', '下单日期': '下单日期_512', 'order_number': 'order_number_512', 'user_mobile': 'user_mobile_512', '内存':'内存_512'}, inplace=True)
# df2_all = df2_256.merge(df2_512, on='id_card_num', how='left')
# df2_all_new = df2_all[(df2_all.下单日期_256>=df2_all.下单日期_512)]
# df2_all_new = df2_all_new.sort_values('create_time_512', ascending=False).groupby('id_card_num').head(1)
# df2_all.shape

In [ ]:
# df2_all_new = df2_all_new[(df2_all_new.下单日期_256>=df2_all_new.下单日期_512)&((df2_all_new.下单日期_256-df2_all_new.下单日期_512)<=pd.Timedelta(days=30))] # &((df2_all.下单日期_256-df2_all.下单日期_512)<=pd.Timedelta(days=3))
# df2_all_new['order_number_256'].drop_duplicates().to_excel(f'F:/需求/总需求/512转256_{t_date}.xlsx', index=False)

# 优质客户召回

In [ ]:
# 优质客户召回字段存在于tips中
# df2_customer = df2[df2.tips.str.contains('优质客户召回', regex=False)==True]

# df2_customer.groupby('下单月份').agg(去重订单数=('order_number', 'count'), 进件数=('是否进件', 'sum'), 出库=('是否出库', 'sum'), 租中优惠券=('rent_discount_money', 'sum')).to_excel(f'F:/需求/廖志彧需求/优质客户召回_{t_date}.xlsx')

In [ ]:
# sql_task = '''
# select order_number, discount_status from db_digua_business.t_user_recall_task
# '''
# df_task = query(sql_task)

In [ ]:
# df2_customer_ck = df2_customer[df2_customer.是否出库==1]
# df_task_merge = df_task.merge(df2_customer_ck[['order_number', 'tips']], on='order_number', how='left')

In [ ]:
# df_task_merge = df_task_merge[(~df_task_merge.order_number.isna())&(df_task_merge.discount_status=='Y')]
# with pd.ExcelWriter('F:/需求/廖志彧需求/优质客户召回租中优惠券.xlsx', engine='xlsxwriter') as writer:
#     df_task_merge.to_excel(writer, sheet_name='租中优惠券', index=False)

In [ ]:
# df2_customer = df2_customer[df2_customer.是否进件==1][['下单日期', 'order_number', 'true_name', '取消原因2', '拒绝理由']].rename(columns={'order_number': '订单号', 'true_name': '姓名'})

# with pd.ExcelWriter(f'F:/需求/廖志彧需求/优质客户召回_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df2_customer.to_excel(writer, sheet_name='优质客户召回')


# 火眸分

In [ ]:
# df_wjj = df[df.审核状态=='进件前取消']
# df_wjj_hm = df_wjj.merge(df_rule, on='order_id', how='left')[['下单月份', 'order_number', 'bj_hm_score', '归属渠道']]

In [ ]:
# df_wjj_hm.to_excel('F:/需求/总需求/火眸分.xlsx', index=False)#.groupby('下单月份').agg({'bj_hm_score': 'sum'})

# 外部渠道

In [ ]:
# channel_out = ['分期乐1', '分期乐2', '八派信息', '九州信息', '富士康租机', '推一推', '线下营业厅-颜总', '勉丫租', '安逸推', '邦客惠乘车码', '亿齐走cpc', '机汤以租代购', '美仑美奂', '天翌科技']
# df_out = df[df.来源渠道.isin(channel_out)]
# df2_out = df2[df2.来源渠道.isin(channel_out)]
# all_models.data_group(df_out, df2_out, df_risk_examine, '年份').to_excel('F:/需求/瑞彤需求/外部渠道.xlsx')

# 搜索下挂-商品

In [ ]:
# df_ssxg = df[df.来源渠道=='搜索下挂-商品']
# df2_ssxg = df2[df2.来源渠道=='搜索下挂-商品']
# df_ssxg_group = all_models.data_group(df_ssxg, df2_ssxg, df_risk_examine, '下单月份')[['去重订单数', '进件数', '出库']]


In [ ]:
# df_ssxg_d = df[(df.来源渠道=='搜索下挂-商品')&(df.下单日期>='2024-09-26')&(df.下单日期<='2024-10-26')]
# df2_ssxg_d = df2[(df2.来源渠道=='搜索下挂-商品')&(df2.下单日期>='2024-09-26')&(df2.下单日期<='2024-10-26')]
# df_ssxg_d_group = all_models.data_group(df_ssxg_d, df2_ssxg_d, df_risk_examine, '下单日期')[['去重订单数', '进件数', '出库']]


In [ ]:
# with pd.ExcelWriter('F:/需求/丹琦需求/搜索下挂-商品.xlsx', engine='openpyxl') as writer:
#     df_ssxg_group.to_excel(writer, sheet_name='09-12')
#     df_ssxg_d_group.to_excel(writer, sheet_name='0926-1026')

In [ ]:
# df_ss_qc = pd.pivot_table(df_ss, values='order_number', index='月份', columns='来源渠道', aggfunc='count')
# df_ss_jj = pd.pivot_table(df_ss, values='是否进件', index='月份', columns='来源渠道', aggfunc='sum')
# df_ss_ck = pd.pivot_table(df_ss2, values='是否出库', index='月份', columns='来源渠道', aggfunc='sum')
# with pd.ExcelWriter('F:/需求/丹琦需求/搜索渠道.xlsx', engine='openpyxl') as writer:
#     df_ss_qc.to_excel(writer, sheet_name='去重订单数')
#     df_ss_jj.to_excel(writer, sheet_name='进件数')
#     df_ss_ck.to_excel(writer, sheet_name='出库数')

# 渠道数据

In [ ]:
# channel_name = ['我的小程序入口', '生活号', '小程序商家消息（服务提醒）', '支付宝直播', '支付宝 push 消息（同1014）', '支付宝客户端首页', '首页十二宫格及更多', '收藏卡片01', '繁星', '公域推广-活动页推广']
# df_channel = df[df.归属渠道.isin(channel_name)]
# df_channel2 = df2[df2.归属渠道.isin(channel_name)]
# df_channel_group = all_models.data_group(df_channel, df_channel2, df_risk_examine, '归属渠道')
# df_channel_group[['去重订单数', '进件数', '出库']].to_excel('F:/需求/支付宝直播数据/渠道数据.xlsx')

# 自有模型 2501

In [ ]:
# df_reject = df2[(df2.是否进件==1)]
# df_reject_base = df_reject.merge(df_risks, left_on=['id_card_num', '下单日期'], right_on=['id_card', '请求日期'], how='left').merge(df_base, left_on='trace_id_risk', right_on='trace_id', how='left')
# df_reject_base = df_reject_base[['下单日期', 'trace_id_risk', 'order_number', '拒绝理由', 'score_base', '归属渠道']]

In [ ]:
# df_reject_base.loc[:, '免人审'] = np.where((df_reject_base["decision_result"].str.contains(pat="免人审",regex=False)==True)&(df_reject_base.is_vip=='1')&(df_reject_base.status_result=='0'), 1, 0)
# df_reject_base.loc[:, '信审'] = np.where((df_reject_base["机审通过件"]==1)&(df_reject_base.免人审==0), 1, 0)
# df_xmy_all, df_xmy = all_models.xmy(df, df_risk_examine, '下单日期')
# df_241205 = df[df.tips.str.contains(r'策略241205|策略241212|命中自有模型回捞策略', regex=True)==True]
# df_241205 = df_241205[df_241205.merchant_name!='小蚂蚁租机']
# df_reject_base_new = df_reject_base.merge(df_xmy_all[['order_number', '小蚂蚁机审强拒']], on='order_number', how='left').merge(df_241205[['order_number', '机审通过件']], on='order_number', how='left')
# df_reject_base_new.loc[:, '机审强拒'] = np.where((df_reject_base_new.机审强拒==1)|(df_reject_base_new.小蚂蚁机审强拒==1)|(df_reject_base_new.机审通过件_y==1), 1, 0)
# df_reject_base_new.loc[:, '是否调用'] = np.where(df_reject_base_new.strategy_250210.isna(), '未调用', '已调用')
# # df_reject_base_new.loc[:, '结果'] = np.where(df_reject_base_new.免人审==1, '免人审', np.where(df_reject_base_new.机审强拒==1, '机审强拒', np.where(df_reject_base_new.出库前风控强拒==1, '出库前风控强拒', np.where(df_reject_base_new.信审==1, '信审', '未知'))))
# df_reject_base_new[['order_number', 'trace_id_risk', 'score_base', '拒绝理由', '下单日期', '是否调用', '免人审', '信审', '机审强拒', '出库前风控强拒', '归属渠道']].to_excel(f'F:/需求/总需求/自有模型2501_1_{t_date}.xlsx', index=False)

In [ ]:
# df_reject_base_new[(df_reject_base_new.下单日期=='2025-02-17')&(df_reject_base_new.信审==1)].shape

In [ ]:
# df_reject_pro = df_reject.merge(df_pro, on='order_id', how='left')
# df_reject_pro = df_reject_pro[['下单日期', 'trace_id', 'order_number', '拒绝理由', 'score_pro', '归属渠道']]

In [ ]:
# with pd.ExcelWriter('F:/需求/总需求/自有模型2501.xlsx', engine='xlsxwriter') as writer:
#     df_reject_base.to_excel(writer, sheet_name='2501_base', index=False)
#     df_reject_pro.to_excel(writer, sheet_name='2501_pro', index=False)

# 差异化定价

In [ ]:
# def cyh(df):
#     df_cy_all = df[['order_number', 'all_rental', 'new_actual_money', 'rent_grade']].rename(columns={'rent_grade': '差异化等级'})
#     df_cy_group = df_cy_all.groupby('差异化等级').agg(数量=('order_number', 'count'))
#     df_cy_group.loc[:, '总计'] = df_cy_group.数量.sum()
#     df_cy_group.loc[:, '比例'] = (df_cy_group.数量/df_cy_group.总计).map(lambda x: format(x, '.2%'))
#     return df_cy_all, df_cy_group

In [ ]:
# df_cyh = df[df.rent_grade.notna()]
# # 去重订单
# df_cyh_qc_all, df_cy_qc_group = cyh(df_cyh)
# # 进件
# df_cyh_jj = df_cyh[df_cyh.是否进件==1]
# df_cyh_jj_all, df_cy_jj_group = cyh(df_cyh_jj)
# # 出库
# df_cyh_ck = df_cyh[df_cyh.是否出库==1]
# df_cyh_ck_all, df_cy_ck_group = cyh(df_cyh_ck)
# # 拒量
# df_cyh_jl = df_cyh[df_cyh.tips.str.contains(r'策略2412|命中自有模型回捞策略')==True]
# df_cyh_jl_all, df_cy_jl_group = cyh(df_cyh_jl)
# # 拒量出库
# df_cyh_jl_ck = df_cyh_jl[df_cyh_jl.是否出库==1]
# df_cyh_jl_ck_all, df_cy_jl_ck_group = cyh(df_cyh_jl_ck)
# # 租赁中
# df_cyh_jl8 = df_cyh_jl[(df_cyh_jl.status2=='租赁中')&(df_cyh_jl.租赁方案=='租完即送')]
# df_cyh_jl8_all, df_cy_jl8_group = cyh(df_cyh_jl8)

In [ ]:
# with pd.ExcelWriter('F:/需求/廖志彧需求/差异化定价.xlsx', engine='xlsxwriter') as writer:
#     df_cy_qc_group.to_excel(writer, sheet_name='去重订单')
#     df_cy_jj_group.to_excel(writer, sheet_name='进件')
#     df_cy_ck_group.to_excel(writer, sheet_name='出库')
#     df_cy_jl_group.to_excel(writer, sheet_name='拒量')
#     df_cy_jl_ck_group.to_excel(writer, sheet_name='拒量出库')
#     df_cy_jl8_group.to_excel(writer, sheet_name='拒量出库租赁中')

# 全域租物

In [ ]:
# df_xkx = df[df.来源渠道=='侠客行全域自定义']
# df2_xkx = df2[df2.来源渠道=='侠客行全域自定义']
# df_xkx_all = all_models.data_group(df_xkx, df2_xkx, df_risk_examine, '下单日期')
# df_xkx_all = df_xkx_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# df2_xkx_g = df2_xkx[df2_xkx.拒绝理由!=''].groupby('拒绝理由').agg({'order_number': 'count'}).rename(columns={'order_number': '数量'})
# with pd.ExcelWriter('F:/需求/瑞彤需求/侠客行全域自定义.xlsx', engine='xlsxwriter') as writer:
#     df_xkx_all.to_excel(writer, sheet_name='侠客行全域自定义')
#     df2_xkx_g.to_excel(writer, sheet_name='拒绝理由')

In [ ]:
# channel_list = ['侠客行全域租物','侠客行租物搜索','创本全域租物','创本租物搜索','邦道全域租物','邦道租物搜索']
# for idx, name in enumerate(channel_list):
#     df_6 = df[df.来源渠道==name]
#     df2_6 = df2[df2.来源渠道==name]
#     df_6_group = all_models.data_group(df_6, df2_6, df_risk_examine, '下单日期')[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                 "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
#     df_6_group_reject = df_6.groupby('拒绝理由').agg({'order_number': 'count'}).rename(columns={'order_number': '数量'})
#     df_6_group_qx = df_6.groupby('取消原因2').agg({'order_number': 'count'}).rename(columns={'order_number': '数量'})
#     if idx==0:
#         with pd.ExcelWriter(f'F:/需求/瑞彤需求/全域租物_{t_date}.xlsx', engine='xlsxwriter') as writer:
#             df_6_group.to_excel(writer, sheet_name=name)
#         with pd.ExcelWriter(f'F:/需求/瑞彤需求/全域租物被拒原因_{t_date}.xlsx', engine='xlsxwriter') as writer:
#             df_6_group_reject.to_excel(writer, sheet_name=name)
#         with pd.ExcelWriter(f'F:/需求/瑞彤需求/全域租物取消原因_{t_date}.xlsx', engine='xlsxwriter') as writer:
#             df_6_group_qx.to_excel(writer, sheet_name=name)
#     else:
#         with pd.ExcelWriter(f'F:/需求/瑞彤需求/全域租物_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:
#             df_6_group.to_excel(writer, sheet_name=name)
#         with pd.ExcelWriter(f'F:/需求/瑞彤需求/全域租物被拒原因_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:
#             df_6_group_reject.to_excel(writer, sheet_name=name)
#         with pd.ExcelWriter(f'F:/需求/瑞彤需求/全域租物取消原因_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:
#             df_6_group_qx.to_excel(writer, sheet_name=name)


# df_6 = df[df.来源渠道.isin(channel_list[::2])]
# df2_6 = df2[df2.来源渠道.isin(channel_list[::2])]
# df_6_group = all_models.data_group(df_6, df2_6, df_risk_examine, '下单日期')[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#             "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# df_6_group_reject = df_6.groupby('拒绝理由').agg({'order_number': 'count'}).rename(columns={'order_number': '数量'})
# df_6_group_qx = df_6.groupby('取消原因2').agg({'order_number': 'count'}).rename(columns={'order_number': '数量'})
# with pd.ExcelWriter(f'F:/需求/瑞彤需求/全域租物_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:
#             df_6_group.to_excel(writer, sheet_name='租物汇总')
# with pd.ExcelWriter(f'F:/需求/瑞彤需求/全域租物被拒原因_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:
#             df_6_group_reject.to_excel(writer, sheet_name='租物汇总')
# with pd.ExcelWriter(f'F:/需求/瑞彤需求/全域租物取消原因_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:
#             df_6_group_qx.to_excel(writer, sheet_name='租物汇总')

# df_6 = df[df.来源渠道.isin(channel_list[1::2])]
# df2_6 = df2[df2.来源渠道.isin(channel_list[1::2])]
# df_6_group = all_models.data_group(df_6, df2_6, df_risk_examine, '下单日期')[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#             "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# df_6_group_reject = df_6.groupby('拒绝理由').agg({'order_number': 'count'}).rename(columns={'order_number': '数量'})
# df_6_group_qx = df_6.groupby('取消原因2').agg({'order_number': 'count'}).rename(columns={'order_number': '数量'})
# with pd.ExcelWriter(f'F:/需求/瑞彤需求/全域租物_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:
#             df_6_group.to_excel(writer, sheet_name='搜索汇总')
# with pd.ExcelWriter(f'F:/需求/瑞彤需求/全域租物被拒原因_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:
#             df_6_group_reject.to_excel(writer, sheet_name='搜索汇总')
# with pd.ExcelWriter(f'F:/需求/瑞彤需求/全域租物取消原因_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:
#             df_6_group_qx.to_excel(writer, sheet_name='搜索汇总')


In [ ]:
# df_6[df_6.取消原因2=='信息填写错误']['order_number'].to_excel('F:/需求/瑞彤需求/信息填写错误订单号.xlsx', index=False)

In [ ]:
def xinghao_classify(a,b):
    if 'iphone15promax' in a:
        return 'iphone15promax'
    elif 'iphone14promax' in a:
        return 'iphone14promax'
    elif 'iphone15pro' in a:
        return 'iphone15pro'    
    elif 'iphone14pro' in a:
        return 'iphone14pro' 
    elif 'iphone15plus' in a:
        return 'iphone15plus'  
    elif 'iphone15' in a:
        return 'iphone15' 
    elif 'iphone14' in a:
        return 'iphone14'
    elif 'mate60pro' in a:
        return 'mate60pro'
    elif 'iphone16promax' in a:
        return 'iphone16promax'
    elif 'iphone16pro' in a:
        return 'iphone16pro'
    elif 'iphone16plus' in a:
        return 'iphone1plus'
    elif 'iphone16' in a:
        return 'iphone16'
    elif 'iphone14plus' in a:
        return 'iphone14plus'
    elif 'iphone13promax' in a:
        return 'iphone13promax'
    elif 'mate60' in a:
        return 'mate60'
    elif 'watch' in a:
        return 'watch'
    elif 'iphone13pro' in a:
        return 'iphone13pro'
    elif 'vivox100pro' in a:
        return 'vivox100pro'
    elif '小米13pro' in a:
        return '小米13pro'
    elif '小米13' in a:
        return '小米13'
    elif 'iphone12promax' in a:
        return 'iphone12promax'
    elif 'iphone13' in a:
        return 'iphone13'
    elif '小米14pro' in a:
        return '小米14pro'
    elif '小米14' in a:
        return '小米14'
    elif 'iphone12pro' in a:
        return 'iphone12pro'
    elif 'ipad' in a:
        return 'ipad'
    elif 'iphone12' in a:
        return 'iphone12'
    elif '相机' in a:
        return '相机'
    elif '荣耀x50' in a:
        return '荣耀x50'
    elif 'findx6pro' in a:
        return 'findx6pro'
    elif 'xiaomi13' in a:
        return '小米13'
    elif 'vivoiqoo12pro' in a:
        return 'vivoiqoo12pro'
    elif 'oppofindn2' in a:
        return 'oppofindn2'
    elif 'redminote12turbo' in a:
        return 'redminote12turbo'
    elif 'iqoo11' in a:
        return 'iqoo11'
    elif 'vivox100' in a:
        return 'vivox100'
    elif 'mate50pro' in a:
        return 'mate50pro'
    elif 'x90' in a:
        return 'x90'
    elif 'vivoiqooneo8' in a:
        return 'vivoiqooneo8'
    elif '华为pura70pro' in a:
        return '华为pura70pro'
    elif '华为p60' in a:
        return '华为p60'
    elif 'vivoiqoo12' in a:
        return 'vivoiqoo12'
    elif 'x90pro' in a:
        return 'x90pro'
    else :
        return b



In [ ]:
# df_6 = df[df.来源渠道.isin(channel_list)]
# df2_6 = df2[df2.来源渠道.isin(channel_list)]
# df_6['型号1'] = df_6.product_name.str.lower().str.replace(' ','')
# tpmn.name 机型 涉及机型使用这个字段
# df_6.loc[:,"型号_new"] = df_6.apply(lambda x:xinghao_classify(x["型号1"],x['product_name']),axis=1)
# df_6.loc[:, '型号内存'] = df_6.型号_new+'_'+df_6.内存
# df2_6['型号1'] = df2_6.product_name.str.lower().str.replace(' ','')
# df2_6.loc[:,"型号_new"] = df2_6.apply(lambda x:xinghao_classify(x["型号1"],x['product_name']),axis=1)
# df2_6.loc[:, '型号内存'] = df2_6.型号_new+'_'+df2_6.内存
# all_models.data_group(df_6, df2_6, df_risk_examine, '型号内存')[['去重订单数', '进件数', '出库']].to_excel('F:/需求/瑞彤需求/机型内存.xlsx')

## 被拒原因

In [ ]:
# df_channel = df[df.来源渠道.isin(channel_list)]
# df_channel.groupby('拒绝理由').agg({'order_number': 'count'}).to_excel('F:/需求/瑞彤需求/全域租物被拒原因.xlsx')

# 八派，九洲和98租超

In [ ]:
# channel_list = ['八派信息', '九州信息', '98租超']
# for idx, name in enumerate(channel_list):
#     df_bj = df[df.归属渠道==name]
#     df_bj2 = df2[df2.归属渠道==name]
#     df_bj_group = all_models.data_group(df_bj, df_bj2, df_risk_examine, '下单日期')
#     df_bj_group = df_bj_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
#     if idx==0:
#         with pd.ExcelWriter(f'F:/需求/七月需求/八派_九州_98租超_{t_date}.xlsx', engine='xlsxwriter') as writer:
#             df_bj_group.to_excel(writer, sheet_name=name)
#     else:
#         with pd.ExcelWriter(f'F:/需求/七月需求/八派_九州_98租超_{t_date}.xlsx', engine='openpyxl', mode='a') as writer:
#             df_bj_group.to_excel(writer, sheet_name=name)

# 线下小店

In [ ]:
# df_contain_xxxd = df_contain[(df_contain.merchant_id==23)&(df_contain.status.isin([2,3,4,8]))]
# df_contain_xxxd = clean.status_node(df_contain_xxxd)
# df_contain_xxxd_detail = df_contain_xxxd[['下单日期', 'order_number', 'product_name', '归属渠道', 'status2']]
# df_contain_xxxd.loc[:, '芝麻租物出库'] = np.where((df_contain_xxxd.是否出库==1)&(df_contain_xxxd.归属渠道=='芝麻租物'), 1, 0)
# df_contain_xxxd.loc[:, '已完结订单'] = np.where(df_contain_xxxd.status2=='已完成', 1, 0)
# df_contain_xxxd_all = df_contain_xxxd.groupby('下单月份').agg({'是否出库': 'sum', '芝麻租物出库': 'sum', '已完结订单': 'sum'}).rename(columns={'是否出库': '总体出库'})
# df_contain_xxxd[df_contain_xxxd.是否出库==1][['order_number', '是否出库', '芝麻租物出库', '已完结订单', 'status2']]
# # with pd.ExcelWriter(f'F:/需求/瑞彤需求/线下小店_{t_date}.xlsx', engine='xlsxwriter') as writer:
# #     df_contain_xxxd_detail.to_excel(writer, sheet_name='订单明细')
# #     df_contain_xxxd_all.to_excel(writer, sheet_name='订单汇总')
# # df_contain_xxxd_group = df_contain_xxxd.groupby('下单月份').agg({'order_number': 'count'})
# # df_contain_xxxd[['order_number']].to_excel(f'F:/需求/瑞彤需求/线下小店_{t_date}.xlsx', index=False)
# df_contain_xxxd_all

In [ ]:
# df_contain_xxxd[(df_contain_xxxd.是否出库==1)&(df_contain_xxxd.下单月份=='2025-04')].shape

# 安卓机型

In [ ]:
# df_new = df[df.product_name.str.replace(' ', '').str.contains(r'(vivoX?(\d+|(Fold?(\d+)))?|OPPOFind ?(X?(\d+)?|华为Mate ?(\d+))?|Mate(/d+)?|iQOO(\d+)?|荣耀(Magic?(\d+))?|小米(\d+))')]
# df2_new = df2[df2.product_name.str.replace(' ', '').str.contains(r'(vivoX?(\d+|(Fold?(\d+)))?|OPPOFind ?(X?(\d+))?|华为Mate ?(\d+)?|Mate(/d+)?|iQOO(\d+)?|荣耀(Magic?(\d+))?|小米(\d+))')]
# df_all = all_models.data_group(df_new, df2_new, df_risk_examine, '下单日期')

# df_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                 "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].to_excel('F:/需求/瑞彤需求/安卓机型数据_30.xlsx')

In [ ]:
# # vivo X系列 （X100系列，X200系列，XFold3系列）
# # oppo FindX系列 （FindX8系列、 Find X7系列）
# # 华为 Mate系列  （Mate70系列、Mate60系列）
# # iQOO 系列  （iQOO12系列、iQOO13系列  ）
# # 荣耀 Magic系列 （荣耀Magic6系列、荣耀Magic7 ）
# # 小米 数字系列 （Xiaomi14系列 、 Xiaomi15系列）
# df.loc[:, '机型'] = df.product_name.str.extract(r'(vivo ?(X?(\d+|(Fold?(\d+))))?|OPPO ?(Find ?(X?(\d+)))?|华为 ?(Mate ?(\d+))?|Mate ?(/d+)?|iQOO ?(\d+)?|荣耀 ?(Magic?(\d+))?|小米 ?(\d+))')[0].drop_duplicates()
# df = df[(df.机型.notna())&(~df.机型.str.strip().isin(['华为', '荣耀', 'vivo', 'OPPO', 'iQOO', 'Mate', '小米']))]
# df2.loc[:, '机型'] = df2.product_name.str.extract(r'(vivo ?(X?(\d+|(Fold?(\d+))))?|OPPO ?(Find ?(X?(\d+)))?|华为 ?(Mate ?(\d+))?|Mate ?(/d+)?|iQOO ?(\d+)?|荣耀 ?(Magic?(\d+))?|小米 ?(\d+))')[0].drop_duplicates()
# df2 = df2[(df2.机型.notna())&(~df2.机型.str.strip().isin(['华为', '荣耀', 'vivo', 'OPPO', 'iQOO', 'Mate', '小米']))]
# df_all = all_models.data_group(df, df2, df_risk_examine, '下单日期')

# df_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                 "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]#.to_excel('F:/需求/瑞彤需求/安卓机型数据_90.xlsx')
# # df#2.机型.drop_duplicates()

# 高低风险模型

In [ ]:
# df_low = df[['下单日期', 'order_number', '是否进件', '机审强拒', '人审拒绝', '出库前风控强拒', '客户取消', '已退款', '归属渠道', '是否出库', '押金类型', 'rejected', 'extremely_low_risk_models', 'high_risk_models', 'tips', 'merchant_name', '拒绝理由']]

# df_low2 = df2[['下单日期','order_number', '是否出库', 'extremely_low_risk_models', 'high_risk_models']]


# df_low.loc[:, '机审强拒'] = np.where((df_low.merchant_name.isin(['小蚂蚁租机', '兴鑫兴通讯']))|(df_low.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')), 1, 0)
# df_low.loc[:, '强拒'] = np.where((df_low.机审强拒==1)|(df_low.人审拒绝==1)|(df_low.出库前风控强拒==1), 1, 0)
# df_low.loc[:, '取消'] = np.where((df_low.客户取消==1)|(df_low.已退款==1), 1, 0)
# # 低风险
# df_low_group  = df_low.groupby('extremely_low_risk_models').agg({'是否进件': 'sum', '强拒': 'sum', '取消': 'sum'}).rename(columns={'是否进件': '进件数'})

# df_low2_group = df_low2.groupby('extremely_low_risk_models').agg({'是否出库': 'sum'}).rename(columns={'是否出库': '出库'})
# df_low_groups = pd.concat([df_low_group, df_low2_group], axis=1)

# # 高风险
# df_low_group  = df_low.groupby('high_risk_models').agg({'是否进件': 'sum', '强拒': 'sum', '取消': 'sum'}).rename(columns={'是否进件': '进件数'})

# df_low2_group = df_low2.groupby('high_risk_models').agg({'是否出库': 'sum'}).rename(columns={'是否出库': '出库'})
# df_high_groups = pd.concat([df_low_group, df_low2_group], axis=1)

# # with pd.ExcelWriter('F:/需求/总需求/高低风险模型.xlsx', engine='xlsxwriter') as writer:
# #     df_low_groups.to_excel(writer, sheet_name='低分险模型')
# #     df_high_groups.to_excel(writer, sheet_name='高分险模型')

In [ ]:
# df_low.loc[:, '咨询订单'] = np.where((df_low.high_risk_models.isin([0, 1]))|(df_low.extremely_low_risk_models.isin([0, 1])), 1, 0)
# df_low.loc[:, '高风险'] = np.where(df_low.high_risk_models==1, 1, 0)
# df_low.loc[:, '未出库_h'] = np.where((df_low.high_risk_models==1)&(df_low.是否出库==0), 1, 0)
# df_low.loc[:, '出库_h'] = np.where((df_low.high_risk_models==1)&(df_low.是否出库==1), 1, 0)
# df_low.loc[:, '低风险'] = np.where(df_low.extremely_low_risk_models==1, 1, 0)
# df_low.loc[:, '未出库_l'] = np.where((df_low.extremely_low_risk_models==1)&(df_low.是否出库==0), 1, 0)
# df_low.loc[:, '出库_l'] = np.where((df_low.extremely_low_risk_models==1)&(df_low.是否出库==1), 1, 0)
# df_low_group = df_low.groupby('下单日期').agg({'是否进件': 'sum', '咨询订单': 'sum', '高风险': 'sum', '出库_h': 'sum', '未出库_l': 'sum', '低风险': 'sum', '未出库_h': 'sum', '出库_l': 'sum'})
# # df_low2_group = df_low2.groupby('下单日期').agg({'未出库_h': 'sum', '出库_h': 'sum', '未出库_l': 'sum', '出库_l': 'sum'})
# # df_low_concat = pd.concat([df_low_group, df_low2_group], axis=1)
# df_low_concat = df_low_group[['是否进件', '咨询订单', '高风险', '未出库_h', '出库_h', '低风险', '出库_l', '未出库_l']]
# # with pd.ExcelWriter(f'F:/需求/总需求/高低风险模型数据_{t_date}.xlsx', engine='xlsxwriter') as writer:
# #     df_low_concat.to_excel(writer, sheet_name='高低风险数据')
# df_low_concat

In [ ]:
# df_lows = df_low[df_low.拒绝理由=='命中百融借贷风险勘测强拒'][['下单日期', 'order_number', '拒绝理由']]
# df_lows.groupby('下单日期').agg({'order_number': 'count'})

# 奇哥需求

In [ ]:
# df_stagess = df2[(df2.是否出库==1)][['order_id', 'true_name', 'id_card_num', 'user_mobile', 'create_time', 'max_overdue_days']]
# df_stages.loc[:, 'reality_refund_date_new'] = pd.to_datetime(df_stages.reality_refund_date.dt.date)
# df_stages.loc[:, 'overdue_days_2'] = np.where(df_stages.reality_refund_date_new.isna(), pd.to_datetime(t_date)-df_stages.refund_date, 
#                                             np.where(df_stages.reality_refund_date_new>df_stages.refund_date, df_stages.reality_refund_date_new-df_stages.refund_date, 0))
# df_stages.loc[:, 'overdue_days_2'] = df_stages.apply(lambda x: x.overdue_days_2.days, axis=1)
# df_merge = df_stagess.merge(df_stages[['order_id', 'overdue_days_2']], on='order_id', how='left')

# df_merge_overdue = df_merge[df_merge.max_overdue_days>0].sample(frac=0.1)
# df_merge_overdue_no = df_merge[df_merge.max_overdue_days==0].sample(frac=0.9)
# df_concat = pd.concat([df_merge_overdue_no, df_merge_overdue])
# # 批量加密多列
# columns_to_encrypt = ['true_name', 'id_card_num', 'user_mobile']
# for col in columns_to_encrypt:
#     df_concat[f'{col}_MD5'] = df_concat[col].apply(clean.md5_hash)

# df_concat[['true_name_MD5', 'id_card_num_MD5', 'user_mobile_MD5', 'create_time', 'max_overdue_days', 'overdue_days_2']].to_excel('F:/需求/奇哥需求/MD5加密数据.xlsx', index=False)

In [ ]:
# df2_wyj = df2[(~df2.订单违约金.isna())&(df2.资金类目==5)&(df2.支付状态==2)]
# df2_wyj.loc[:, '违约类型'] = np.where(df2_wyj.是否出库!=1, '发货前', np.where(df2_wyj.total_describes.str.contains(r'发货|拒收|找人冒签|到货|取消订单')==True, '发货后',
#                                     np.where(df2_wyj.total_describes.str.contains(r'提前归还|提前寄回|未拆封|签收后'), '提前归还', 
#                                             np.where(df2_wyj.total_describes.str.contains(r'解锁费'), '锁机', '其他'))))
# df2_wyj.loc[:, '渠道'] = np.where(df2_wyj.归属渠道=='芝麻租物', '芝麻租物', '非芝麻租物')
# df2_wyj_g = df2_wyj.groupby(['下单月份', '渠道', '违约类型']).agg({'order_id': 'count', '订单违约金': 'sum'}).rename(columns={'order_id': '数量'}).reset_index()
# df2_wyj_m = df2_wyj.groupby('下单月份').agg({'订单违约金': 'sum'}).rename(columns={'订单违约金': '月份违约金'}).reset_index()
# df2_wyj_merge = df2_wyj_g.merge(df2_wyj_m, on='下单月份', how='left')
# df2_wyj_merge.loc[:, '比例'] = (df2_wyj_merge.订单违约金/df2_wyj_merge.月份违约金).map(lambda x: format(x, '.2%'))
# df2_wyj_merge.to_excel('F:/需求/奇哥需求/违约金数据.xlsx', index=False)

# 廖志彧需求

In [8]:
# -- 2025/8/4 需求：促买断毛利  tod.product_name, tprm.purchase_amount
# 订单的实收金额为financial_type = 0且is_transfer = 0 的订单中所有yma_type=0的订单金额（amount）之和减去yma_type=1的订单金额（amount）,ybt.status表示订单已完成
sql = '''select 
yma.order_number, yma.financial_type, yma.type, yma.amount, yma.is_transfer,
tod.new_actual_money, tod.product_name, tprm.purchase_amount, ybt.end_time, tod.order_id

-- 资方记账表
from db_rent.ya_merchant_account yma
-- 促买断任务表
left join db_rent.ya_buyout_task ybt on ybt.order_number = yma.order_number
left join db_digua_business.t_order_details tod on tod.order_id = ybt.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=ybt.order_id
where yma.financial_type = 0
and yma.is_transfer = 0
and ybt.status = 3;
'''
df_order = clean.query(sql)
df_order.shape

(36881, 10)

In [9]:
def calculate_net_amount(order_df):
    # 确保只处理financial_type=0的记录
    # order_df = order_df[(order_df['financial_type'] == 0)&(order_df['is_transfer'] == 0 )]
    # 确保amount是数值类型
    order_df['amount'] = pd.to_numeric(order_df['amount'], errors='coerce')
    
    # 计算收入(yma_type=0)和支出(yma_type=1)
    revenue = order_df[order_df['type'] == 0]['amount'].sum()
    expense = order_df[order_df['type'] == 1]['amount'].sum()
    
    # 计算实收金额
    return revenue - expense

# 按订单分组并应用计算函数,按照end_time降序排序取最新的一条
df_copy = df_order.copy()
net_amounts = df_copy.groupby('order_number').apply(calculate_net_amount).reset_index()
net_amounts.columns = ['order_number', '订单实收金额']
net_amounts
# 合并回原始DataFrame
df_new = df_copy.merge(net_amounts, on='order_number', how='left').rename(columns={'new_actual_money': '新买断价', 'purchase_amount':'采购成本'})
df_new = df_new.sort_values('end_time', ascending=False).drop_duplicates('order_number')

df_new.head()

,order_number,financial_type,type,amount,is_transfer,新买断价,product_name,采购成本,end_time,order_id,订单实收金额
29441,A20241113104753376,0,0,720.60,0,12996.70,iPhone15ProMax 全新国行 顺丰包邮 低租金高通过,8090.0,2025-11-10 17:34:02,1718777,11996.70
36868,A20250707215353362,0,0,1.00,0,12898.70,首月1元 iPhone 16 Pro Max 非监管机 全新未激活 顺丰包邮,8440.0,2025-11-10 16:41:01,3157797,12146.66
33093,A2024120315070182,0,0,695.21,0,12996.70,iPhone 16 Pro Max 全新未激活 国行正品 寄出包邮,9550.0,2025-11-10 16:30:02,1806199,12196.70
33072,A2024120315043960,0,0,347.61,0,12996.70,iPhone 16 Pro Max 全新未激活 国行正品 寄出包邮,9510.0,2025-11-10 15:53:02,1806177,12196.70
26042,A20241010104706201,0,0,720.60,0,14466.22,iPhone15ProMax 全新国行 顺丰包邮 低租金高通过,8320.0,2025-11-10 14:56:03,1576928,13066.22


In [ ]:
# df_merge.product_name.unique()

In [10]:
df_r = pd.read_excel('F:/需求/廖志彧需求/e6bee32d-4dd2-4c42-a14d-ae3008fd7a72.xlsx')
ts_list = ['']

df_r_tc = df_r[~df_r.订单号.isin(ts_list)]

# df_r_tc.to_excel('F:/需求/廖志彧需求/3月促买断_剔除特殊申请.xlsx', index=False)
# 订单的实收金额为financial_type = 0 的订单中所有yma_type=0的订单金额（amount）之和减去yma_type=1的订单金额（amount）
# 先计算出订单实收收入：financial_type = 0中所有yma_type=0的订单金额（amount）之和
# df = df[df['financial_type'] == 0]
# df.loc[:,'订单实收收入'] = df[(df.yma_type == 0)].fillna(0).groupby('order_number')['amount'].transform('sum')
# # 再计算出订单实收支出：financial_type = 0中所有yma_type的订单金额（amount）之和
# df.loc[:,'订单实收支出'] = df[df.yma_type == 1].fillna(0).groupby('order_number')['amount'].transform('sum')
# # 订单实收金额：订单实收收入-订单实收支出
# df.loc[:,'订单实收金额'] = df['订单实收收入'] - df['订单实收支出']

# df.loc[:,'订单实收收入'] = np.where((df['yma_type'] == 1)&(df['financial_type'] == 0), df['amount'], 0)

# df = df[df['financial_type'] == 0]
# revenue_df = df[(df.financial_type == 0)&(df.yma_type == 0)].fillna(0).groupby('order_number').agg({'amount': 'sum'}).reset_index()
# revenue_df.rename(columns={'amount': '订单实收收入'}, inplace=True)

# pay_df = df[(df.financial_type == 0)&(df.yma_type == 1)].fillna(0).groupby('order_number').agg({'amount': 'sum'}).reset_index()
# pay_df.rename(columns={'amount': '订单实收支出'}, inplace=True)

# df_res = df.merge(revenue_df, on='order_number', how='left').merge(pay_df, on='order_number', how='left')
# df_res.loc[:,'订单实收金额'] = df_res['订单实收收入'] - df_res['订单实收支出']

# # # 基于订单号去重

# # # 订单号
df_merge = df_r_tc.merge(df_new[['order_number', 'product_name','采购成本','订单实收金额', '新买断价']], left_on='订单号', right_on='order_number', how='left')
# # # 计算毛利
df_merge.loc[:,'毛利'] = df_merge['订单实收金额'] - df_merge['采购成本']
df_merge.loc[:,'毛利率'] = df_merge['毛利']/df_merge['采购成本']
df_merge.loc[:,'毛利率'] = df_merge['毛利率'].apply(lambda x:format(x,".2%"))
#['订单实收收入']
# df_merge = df_r_tc.merge(df[['order_number', 'product_name','purchase_amount']], left_on='订单号', right_on='order_number', how='left')
# df_merge = df_merge.rename(columns = {'purchase_amount':'采购成本'})
df_merge[df_merge.product_name.str.contains(r'iPhone|Watch|Apple|iPad')==True].to_excel('F:/需求/廖志彧需求/10月促买断_特殊申请、剔除安卓机型.xlsx', index=False)# 特殊申请、

In [ ]:
df_merge#.to_excel('F:/需求/廖志彧需求/7月促买断_剔除特殊申请、安卓机型1.xlsx', index=False)

In [ ]:
# 筛选df_merge中所有订单实收金额大于买断价的订单
df_merge[df_merge["订单实收金额"]>df_merge["买断价"]+0.001]

In [ ]:
# TO_DO日常
# df_r = pd.read_excel('F:/需求/廖志彧需求/6月促买断.xlsx')
# 特殊申请订单号
# ts_list = ['A2024110220382055']

# df_r_tc = df_r[~df_r.订单号.isin(ts_list)]
# # df_r_tc.to_excel('F:/需求/廖志彧需求/3月促买断_剔除特殊申请.xlsx', index=False)
# df_merge = df_r_tc.merge(df[['order_number', 'product_name','purchase_amount']], left_on='订单号', right_on='order_number', how='left')
# df_merge = df_merge.rename(columns = {'purchase_amount':'采购成本'})
# df_merge[df_merge.product_name.str.contains(r'iPhone|Watch|Apple')==True].to_excel('F:/需求/廖志彧需求/6月促买断_剔除特殊申请、安卓机型.xlsx', index=False)

## 促买断任务匹配分配人

In [ ]:
# 促买断任务匹配分配人直接在ybt表取，不是出库分配人
sql_ybt = '''select 
ybt.order_number, ybt.order_id, ybt.true_name, ybt.user_mobile,ybt.assign_user_name

from db_rent.ya_buyout_task ybt
# left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
# where DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2024-09-25'
# and DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2024-10-05'
;
'''
df_ybt = clean.query(sql_ybt)
df_ybt.drop_duplicates(subset=["order_number"], inplace=True)
df_ybt.shape


In [ ]:
df_dpp = pd.read_excel('F:\需求\廖志彧需求\资方数据金锤订单明细.xlsx')
# 拼接df_dpp和df_ybt、df_name
df_ans = df_dpp.merge(df_ybt, left_on='订单号', right_on='order_number', how='left').rename(columns={'true_name': '姓名','user_mobile': '手机号', 'assign_user_name': '分配人'})


# df_ans[['下单日期', '订单状态', '订单号', '姓名', '手机号', '分配人']].to_excel('F:\需求\廖志彧需求\资方数据金锤订单明细_分配人.xlsx', index=False)
# df_ans.to_excel('F:\需求\廖志彧需求\资方数据金锤订单明细_分配人.xlsx', index=False)
# df_ans

## 匹配收货人手机号和姓名

In [ ]:
sql_pp = '''
select tol.delivery_username, tol.delivery_mobile, om.order_number

from db_digua_business.t_order_logistics tol
left join db_digua_business.t_order om on tol.order_id = om.id
where DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2024-09-25'
and DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2024-10-05'
'''

df_pp = clean.query(sql_pp)


In [ ]:
# df_dpp = pd.read_excel('F:\需求\力哥需求\资方数据金锤订单明细.xlsx')
# df_pp.drop_duplicates(subset=["order_number"], inplace=True)
# df_ans = df_dpp.merge(df_pp, left_on='订单号', right_on='order_number', how='left').rename(columns={'delivery_username': '收货人','delivery_mobile': '手机号'})
# df_ans[['下单日期', '订单状态', '订单号', '收货人', '手机号']].to_excel('F:\需求\力哥需求\资方数据金锤订单明细_收货人.xlsx', index=False)

# iPhone16转化

In [ ]:
# df_16 = df[df.product_name.str.contains(r'iPhone ?16(ProMax|Pro)')==True]
# df2_16 = df2[df2.product_name.str.contains(r'iPhone ?16(ProMax|Pro)')==True]
# df_16.loc[:, '型号'] = df_16.product_name.str.extract(r'(iPhone ?16(ProMax)?)')[0]
# df_16.loc[:, '型号1'] = df_16.product_name.str.extract(r'(iPhone ?16(Pro)?)')[0]
# df2_16.loc[:, '型号'] = df2_16.product_name.str.extract(r'(iPhone ?16(ProMax)?)')[0]
# df2_16.loc[:, '型号1'] = df2_16.product_name.str.extract(r'(iPhone ?16(Pro)?)')[0]
# df_16 = df_16[df_16.归属渠道=='芝麻租物']
# df2_16 = df2_16[df2_16.归属渠道=='芝麻租物']

In [ ]:
# # 获取iPhone16ProMax(芝麻租物)
# df_16_1 = df_16[df_16.型号.str.replace(' ', '')=='iPhone16ProMax']
# df2_16_1 = df2_16[df2_16.型号.str.replace(' ', '')=='iPhone16ProMax']
# df2_16_1_group = all_models.data_group(df_16_1, df2_16_1, df_risk_examine, '下单日期')[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# # 获取iPhone16Pro
# df_16_2 = df_16[df_16.型号1.str.replace(' ', '')=='iPhone16Pro']
# df2_16_2 = df2_16[df2_16.型号1.str.replace(' ', '')=='iPhone16Pro']
# df2_16_2_group = all_models.data_group(df_16_2, df2_16_2, df_risk_examine, '下单日期')[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# # 写入Excel
# with pd.ExcelWriter(f'F:/需求/瑞彤需求/iPhone16转化_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df2_16_1_group.to_excel(writer, sheet_name='iPhone16ProMax')
#     df2_16_2_group.to_excel(writer, sheet_name='iPhone16Pro')

# 灯火联投测试

In [ ]:
# df_lt = df[(df.来源渠道.str.contains('灯火联投测试')==True)&(df.下单日期>='2025-04-15')&(~df.merchant_name.isin(['人人享租', '兴鑫兴通讯']))]
# df2_lt = df2[(df2.来源渠道.str.contains('灯火联投测试')==True)&(df2.下单日期>='2025-04-15')&(~df.merchant_name.isin(['人人享租', '兴鑫兴通讯']))]
# df_lt_all = all_models.data_group(df_lt, df2_lt, df_risk_examine, '下单日期')
# # 不包含小蚂蚁
# df_lt_all = df_lt_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]

# # 包含小蚂蚁
# df_lt_all2 = all_models.merchant_names(df_contain[(df_contain.来源渠道.str.contains('灯火联投测试')==True)&(df_contain.下单日期>='2025-04-15')],'小蚂蚁租机', '下单日期')
# df_lt_all2 = df_lt_all2[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','订单出库率']]
# # 取消原因-不包含小蚂蚁
# df_lt_g = df2_lt[df2_lt.是否进件==1].groupby('取消原因2').agg({'order_number': 'count'}).rename(columns={'order_number': '数量'})
# # 取消原因-包含小蚂蚁
# # df_lt_g2 = df_lt[df_lt.是否进件==1].groupby('取消原因2').agg({'order_number': 'count'}).rename(columns={'order_number': '数量'})
# # 拒绝理由-不包含小蚂蚁
# df_lt_gr = df2_lt[df2_lt.是否进件==1].groupby('拒绝理由').agg({'order_number': 'count'}).rename(columns={'order_number': '数量'})
# # 拒绝理由-包含小蚂蚁
# # df_lt_gr2 = df_lt[df_lt.是否进件==1].groupby('拒绝理由').agg({'order_number': 'count'}).rename(columns={'order_number': '数量'})

# # 拒量
# df_lt_jl = df_lt[df_lt.是否拒量==1]
# df_jl_group = df_lt_jl.groupby('下单日期').agg({'是否进件': 'sum', '人审拒绝': 'sum', '风控通过件': 'sum', '客户取消': 'sum', '无法联系': 'sum', '出库前风控强拒': 'sum', '待审核': 'sum', '是否出库': 'sum', '已退款': 'sum'}).rename(columns={'是否进件': '进件数', '是否出库': '出库'})
# df_jl_group.loc[:, '客户取消'] = df_jl_group.客户取消+df_jl_group.已退款
# df_jl_group.loc[:, '风控通过率'] = (df_jl_group.风控通过件/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
# df_jl_group.loc[:, '进件出库率'] = (df_jl_group.出库/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
# df_jl_group.loc[:, '取消比例'] = (df_jl_group.客户取消/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
# df_jl_group.loc[:, '电审拒绝率'] = (df_jl_group.人审拒绝/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
# df_jl_group.loc[:, '出库前强拒比例'] = (df_jl_group.出库前风控强拒/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
# df_jl_group.loc[:, '无法联系占比'] = (df_jl_group.无法联系/df_jl_group.进件数).map(lambda x: format(x, '.2%'))

# df_lt_qd = df_lt[df_lt.是否进件==1][['order_number', '来源渠道', 'activity_name']]

# with pd.ExcelWriter(f'F:/需求/瑞彤需求/灯火联投测试_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_lt_all.to_excel(writer, sheet_name='转化数据_不包含小蚂蚁')
#     df_lt_all2.to_excel(writer, sheet_name='转化数据_小蚂蚁')
#     df_jl_group.to_excel(writer, sheet_name='转化数据_拒量')
#     df_lt_g.to_excel(writer, sheet_name='取消原因_不包含小蚂蚁')
#     # df_lt_g2.to_excel(writer, sheet_name='取消原因_包含小蚂蚁')
#     df_lt_gr.to_excel(writer, sheet_name='拒绝理由_不包含小蚂蚁')
#     # df_lt_gr2.to_excel(writer, sheet_name='拒绝理由_包含小蚂蚁')
#     df_lt_qd.to_excel(writer, sheet_name='渠道明细', index=False)

# CPS渠道

In [ ]:
# cps_list = ['租机宝','八派信息','CPS直播','派金花','富士康租机','九州信息','勉丫租','哈银','美仑美奂','兔丫租赁','分期乐1','分期乐2','98租超','邦客惠乘车码','推一推','硬派抖音']
# df_cps = df[df.归属渠道.isin(cps_list)]
# df2_cps = df2[df2.归属渠道.isin(cps_list)]
# df_cps_all = all_models.data_group(df_cps, df2_cps, df_risk_examine, ['下单月份', '归属渠道'])

# df_cps_all = df_cps_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                 "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# with pd.ExcelWriter(f'F:/需求/瑞彤需求/CPS渠道转化_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_cps_all.to_excel(writer, sheet_name='cps')

# 仕程需求

In [ ]:
# sql_zqyq = '''
# select order_id, add_time, sort, agent_user_name, status from db_rent.ya_agent_task where overdue_type=0
# '''
# df_zqyq = query(sql_zqyq)
# df_zqyq.loc[:, 'year'] = df_zqyq.add_time.dt.year

In [ ]:
# df_fp = pd.read_excel('F:/无标题(2).xlsx')
# df_fp = df_fp[~df_fp.agent_user_name.isna()]
# df_fp.loc[:, 'sort'] = df_fp.sort.str.strip('[]')
# df_fp.loc[:, '还款'] = np.where(df_fp.status==3, 1, 0)
# df_fp.loc[:, '未还款'] = np.where(df_fp.status!=3, 1, 0)
# 根据资方业务员名称和期数分组，计算总任务、还款、未还款，计算催回率
# df_fp_g = df_fp.groupby(['agent_user_name', 'sort']).agg({'order_id': 'size', '还款': 'sum', '未还款': 'sum'}).reset_index()
# df_fp_g = df_fp_g.rename(columns={'agent_user_name': '名称', 'sort': '期数', 'order_id': '总任务'})
# df_fp_g.loc[:, '催回率'] = (df_fp_g.还款/df_fp_g.总任务).map(lambda x: format(x, '.2%'))
# 筛选出2、3、4期
# df_fp_new = df_fp[df_fp.sort.isin(['2', '3', '4'])]

In [ ]:
# df_fp_new_merge = df_fp_new.merge(df, on='order_id', how='left')
# 回捞策略标签
# df_fp_new_merge.loc[:, '是否拒量'] = np.where(df_fp_new_merge.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)
# # df_fp_new_merge[['order_id', '是否拒量', '归属渠道']].value_counts()
# # df_fp_new_merge.groupby(['sort', '归属渠道', '未还款']).agg({'order_id': 'count', '是否拒量': 'sum'})


In [ ]:
# df_zqyq.loc[:, 'add_date'] = df_zqyq.add_time.dt.strftime('%Y-%m')
# 筛选2024年10月到2025年3月的订单
# df_zqyq_new = df_zqyq[df_zqyq.add_date.isin(['2024-10', '2024-11', '2024-12', '2025-01', '2025-02', '2025-03'])]
# # df_zqyq_new = df_zqyq_new[df_zqyq_new.agent_user_name!=' ']
# 过滤掉agent_user_name为空或者包含'AI机器人外呼'的行
# df_zqyq_new = df_zqyq_new[~((df_zqyq_new.agent_user_name.isna())|(df_zqyq_new.agent_user_name.str.contains('AI机器人外呼')==True))]
# df_zqyq_new.loc[:, 'sort_len'] = df_zqyq_new.sort.apply(lambda x: len(ast.literal_eval(x)))
# 筛选出sort_len为1的行
# df_zqyq_new = df_zqyq_new[df_zqyq_new.sort_len==1]
# 将sort列去除方括号，并转换为整数
# df_zqyq_new.loc[:, 'sort_new'] = df_zqyq_new.sort.apply(lambda x: int(x.strip('[]')))
# 按 add_time 排序并分组，取每组的第一行
# df_zqyq_new = df_zqyq_new.sort_values(by='add_time').groupby(['order_id', 'sort']).head(1)

# df_zqyq_new.loc[:, '还款'] = np.where(df_zqyq_new.status==3, 1, 0)
# df_zqyq_new.loc[:, '未还款'] = np.where(df_zqyq_new.status!=3, 1, 0)
# 按日期和期数分组,计算总任务、还款、未还款，计算催回率
# df_zqyq_new_g = df_zqyq_new.groupby(['add_date', 'sort_new']).agg({'order_id': 'size', '还款': 'sum', '未还款': 'sum'}).reset_index()
# df_zqyq_new_g = df_zqyq_new_g.rename(columns={'agent_user_name': '名称', 'sort_new': '期数', 'order_id': '总任务'})
# df_zqyq_new_g.loc[:, '催回率'] = (df_zqyq_new_g.还款/df_zqyq_new_g.总任务).map(lambda x: format(x, '.2%'))
# # df_zqyq_new_g.sort_values(by=['add_date', '期数']).to_excel('F:/嘻嘻嘻.xlsx', index=False)

In [ ]:
# df_zqyq_new_merge = df_zqyq_new.merge(df[['order_id', '归属渠道', 'tips']], on='order_id', how='left')
# # df_zqyq_new_merge = df_zqyq_new_merge[df_zqyq_new_merge.sort_new.isin([2, 3, 4])]
# df_zqyq_new_merge.loc[:, '渠道'] = np.where(df_zqyq_new_merge.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, '拒量', 
#                                             np.where(df_zqyq_new_merge.归属渠道.isin(['八派信息','CPS直播','派金花','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','分期乐1','分期乐2','98租超','推一推','硬派抖音']), 'CPS渠道', df_zqyq_new_merge.归属渠道))
# 筛选出拒量、CPS渠道、搜索渠道、单人聊天会话中的小程序消息卡片（分享）、芝麻租物、CPS渠道的订单
# df_zqyq_new_merge_new = df_zqyq_new_merge[df_zqyq_new_merge.渠道.isin(['芝麻租物','搜索渠道','单人聊天会话中的小程序消息卡片（分享）','CPS渠道', '拒量'])]
# # df_zqyq_new_merge.groupby(['渠道', 'sort']).agg({'order_id': 'count'}).sort_values(['渠道', 'sort', 'order_id'], ascending=[False, True, False])

# df_zqyq_new_merge_new.loc[:, '还款'] = np.where(df_zqyq_new_merge_new.status==3, 1, 0)
# df_zqyq_new_merge_new.loc[:, '未还款'] = np.where(df_zqyq_new_merge_new.status!=3, 1, 0)
# df_zqyq_new_merge_new_g = df_zqyq_new_merge_new.groupby(['sort_new']).agg({'order_id': 'size', '还款': 'sum', '未还款': 'sum'}).reset_index()
# df_zqyq_new_merge_new_g = df_zqyq_new_merge_new_g.rename(columns={'agent_user_name': '名称', 'sort_new': '期数', 'order_id': '总任务'})
# df_zqyq_new_merge_new_g.loc[:, '催回率'] = (df_zqyq_new_merge_new_g.还款/df_zqyq_new_merge_new_g.总任务).map(lambda x: format(x, '.2%'))


# # df_zqyq_new_merge_new_g
# # with pd.ExcelWriter('F:/渠道催回.xlsx', engine='xlsxwriter') as writer:
# #     df_zqyq_new_merge_new_g.to_excel(writer, sheet_name='催回率', index=False)
# #     df_zqyq_new_merge_new_jg.to_excel(writer, sheet_name='未还款的拒量情况')

In [ ]:
# list_sort = ['2','3','4','5','6','7-12', '12以上']
# # 总体
# 期数重新分类命名
# df_zqyq_new_merge_new.loc[:, 'sort_new2'] = np.where(df_zqyq_new_merge_new.sort_new.isin([7, 8, 9, 10, 11, 12]), '7-12', np.where(df_zqyq_new_merge_new.sort_new>12, '12以上', df_zqyq_new_merge_new.sort_new))
'''
values='order_id'表示透视表的值是order_id列的值。
columns=df_zqyq_new_merge_new.sort_new2表示透视表的列是sort_new2列的值。
index=df_zqyq_new_merge_new.add_date表示透视表的索引是add_date列的值。
aggfunc='count'表示对每个单元格的值进行计数。
最后，将透视表的列按照list_sort的顺序重新排列
'''
# df_zqyq_new_merge_new_pt = pd.pivot_table(df_zqyq_new_merge_new, values='order_id', columns=df_zqyq_new_merge_new.sort_new2, index=df_zqyq_new_merge_new.add_date, aggfunc='count')
# df_zqyq_new_merge_new_pt = df_zqyq_new_merge_new_pt[list_sort]
# # 还款
# df_zqyq_new_merge_new_hk = df_zqyq_new_merge_new[df_zqyq_new_merge_new.还款==1]
# df_zqyq_new_merge_new_hk_pt = pd.pivot_table(df_zqyq_new_merge_new_hk, values='order_id', columns=df_zqyq_new_merge_new_hk.sort_new2, index=df_zqyq_new_merge_new_hk.add_date, aggfunc='count')
# df_zqyq_new_merge_new_hk_pt = df_zqyq_new_merge_new_hk_pt[list_sort]
# pd.concat([df_zqyq_new_merge_new_pt, df_zqyq_new_merge_new_hk_pt], axis=1)

In [ ]:
# df_zqyq_new_merge_new = df_zqyq_new_merge_new[df_zqyq_new_merge_new.sort_new.isin([2, 3, 4])]
# df_zqyq_new_merge_new.loc[:, '是否拒量'] = np.where(df_zqyq_new_merge_new.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)
# df_zqyq_new_merge_new.loc[:, '新渠道'] = np.where(df_zqyq_new_merge_new.是否拒量==1, '拒量', np.where(df_zqyq_new_merge_new.归属渠道=='芝麻租物', '芝麻租物', 
#                                             np.where(df_zqyq_new_merge_new.归属渠道=='搜索渠道', '搜索渠道',  
#                                             np.where(df_zqyq_new_merge_new.归属渠道=='单人聊天会话中的小程序消息卡片（分享）', '单人聊天会话中的小程序消息卡片（分享）',
#                                                         np.where(df_zqyq_new_merge_new.归属渠道.isin(['八派信息','CPS直播','派金花','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','分期乐1','分期乐2','98租超','推一推','硬派抖音']), 'S量', '其他')))))
# df_zqyq_new_merge_new_jg = df_zqyq_new_merge_new.groupby(['add_date', '渠道']).agg({'order_id': 'count', '是否拒量': 'sum'})
# df_zqyq_new_merge_new_jg
# df_zqyq_new_merge_new_qpt = pd.pivot_table(df_zqyq_new_merge_new, values='order_id', columns=df_zqyq_new_merge_new.sort_new, index=[df_zqyq_new_merge_new.add_date, df_zqyq_new_merge_new.新渠道], aggfunc='count')
# df_zqyq_new_merge_new_qpt.sort_values(by=['add_date', '新渠道'], ascending=[True, False])

In [ ]:
# df_zqyq_new_merge_new_hk = df_zqyq_new_merge_new[df_zqyq_new_merge_new.还款==1]
# df_zqyq_new_merge_new_qpt = pd.pivot_table(df_zqyq_new_merge_new_hk, values='order_id', columns=df_zqyq_new_merge_new_hk.sort_new, index=[df_zqyq_new_merge_new_hk.add_date, df_zqyq_new_merge_new_hk.新渠道], aggfunc='count')
# df_zqyq_new_merge_new_qpt.sort_values(by=['add_date', '新渠道'], ascending=[True, False])

# 支付宝订单

In [ ]:
# df_zfb = df[df.order_type=='ZFB_ORDER']
# df2_zfb = df2[df2.order_type=='ZFB_ORDER']
# df_zfb_all = all_models.data_group(df_zfb, df2_zfb, df_risk_examine, '下单月份')
# df_zfb_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                 "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].to_excel('F:/需求/七月需求/支付宝转化数据.xlsx')


# 电脑

In [ ]:
# 125：游戏本
# 126：商务本
# 127：笔记本
# 185：台式机
df_pc = df[df.classify_id.isin([125, 126, 127, 185])]
df2_pc = df2[df2.classify_id.isin([125, 126, 127, 185])]
# all_models.data_group(df_pc, df2_pc, df_risk_examine, '下单日期')[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                                                                     "出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]

## 宏生需求支付宝订单

### 租物

In [ ]:
df[df.classify_id.isin([125, 126, 127, 185])].机型.unique()

In [ ]:
df_zfb = df[df.order_type=='ZFB_ORDER']
df_zfb2 = df2[df2.order_type == 'ZFB_ORDER']
df_zm = df_zfb[(df_zfb['归属渠道']=='芝麻租物') & (df_zfb.classify_id.isin([125, 126, 127, 185]))]
df_zm2 = df_zfb2[(df_zfb2['归属渠道']=='芝麻租物')& (df_zfb2.classify_id.isin([125, 126, 127, 185]))]
df_zm_group = all_models.data_group(df_zm, df_zm2, df_risk_examine, '机型')

# df_zm_group['出库（按下单时间）'] = df_weekday['芝麻租物'][:-1]
df_zm_group['出库（按下单时间）'] = df_zm_group['出库']
df_zm_group['出库（按转化时间）'] = df_weekday_zh['芝麻租物'][:-1]
df_zm_group = df_zm_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_zm_group_new = df_zm_group.reset_index()
df_zm_group#.tail()

### 非租物

In [ ]:
df_fzm = df_zfb[(df_zfb['归属渠道']!='芝麻租物') & (df_zfb.classify_id.isin([125, 126, 127, 185]))]
df_fzm2 = df_zfb2[(df_zfb2['归属渠道']!='芝麻租物')& (df_zfb2.classify_id.isin([125, 126, 127, 185]))]
df_fzm_group = all_models.data_group(df_fzm, df_fzm2, df_risk_examine, '机型')

# df_fzm_group['出库（按下单时间）'] = df_weekday['芝麻租物'][:-1]
df_fzm_group['出库（按下单时间）'] = df_fzm_group['出库']
df_fzm_group['出库（按转化时间）'] = df_weekday_zh['芝麻租物'][:-1]
df_fzm_group = df_fzm_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_fzm_group_new = df_fzm_group.reset_index()
df_fzm_group#.tail()

## 京东订单

In [ ]:
df_jd = df[(df['归属渠道']=='京东渠道')&(df.classify_id.isin([125, 126, 127, 185]))]
df_jd2 = df2[(df2['归属渠道']=='京东渠道')&(df2.classify_id.isin([125, 126, 127, 185]))]
# df_jd = df_jd[df_jd.下单日期>='2025-05-20']
# df_jd2 = df_jd2[df_jd2.下单日期>='2025-05-20']
df_jd_group = all_models.data_group(df_jd, df_jd2, df_risk_examine, '机型')


df_jd_group['出库（按下单时间）'] = df_jd_group['出库']
try:
    df_jd_group['出库（按转化时间）'] = df_weekday_zh['京东渠道'][:-1]
except:
    df_jd_group['出库（按转化时间）'] = 0
df_jd_group = df_jd_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_jd_group_new = df_jd_group.reset_index()
# df_jd_group_new = df_no_drop_jd_g.merge(df_jd_group_new, on='下单日期', how='inner')
df_jd_group_new#.tail()

In [ ]:
# path = 'F:/宏生/笔记本转化数据.xlsx'
# with pd.ExcelWriter(path, engine='openpyxl') as writer:
#     df_zm_group_new.to_excel(writer, sheet_name='芝麻租物', index=False)
#     df_fzm_group_new.to_excel(writer, sheet_name='非芝麻租物', index=False)
#     df_jd_group_new.to_excel(writer, sheet_name='京东渠道', index=False)

# 宏生需求

In [ ]:
df_contain["待审核"] = np.where(df_contain["审核状态"]=='待审核',1,0)
df_contain["前置拦截"] = np.where(df_contain["审核状态"]=='前置拦截',1,0)
df_contain["机审强拒"] = np.where(df_contain["审核状态"]=='机审强拒',1,0)
df_contain["人审拒绝"] = np.where(df_contain["审核状态"]=='人审拒绝',1,0)
df_contain["客户取消"] = np.where(df_contain["审核状态"]=='客户取消',1,0)
df_contain["出库前风控强拒"] = np.where(df_contain["审核状态"]=='出库前风控强拒',1,0)
df_contain["无法联系"] = np.where(df_contain["审核状态"]=='无法联系',1,0)
df_contain["是否进件"] = np.where(df_contain["进件"]=='进件',1,0)
df_contain["是否出库"] = np.where(df_contain["status"].isin([2,3,4,5,6,8,15]),1,0)

df_contain["进件前取消"] = np.where(df_contain["审核状态"]=='进件前取消',1,0)

In [ ]:
df_rrxz_ck = df_contain[(df_contain.merchant_name=='人人享租')&(df_contain.是否出库==1)]

